In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Baseline_Model_Sequence_of_5_All_Maches_Except_( France , Argentina , Croatia , Morocco , England ) Matches_Save_6_Copies_of_Trained_Model**

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive
import matplotlib.pyplot as plt
import json
import time
from datetime import datetime
from typing import Tuple, List, Dict, Any
import logging

print("== STEP 1: MODEL DEVELOPMENT ENVIRONMENT SETUP FOR FIFA 2022 SEQUENCE OF 5 ==")

# Mount Google Drive
if not os.path.exists('/content/drive'):
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("Google Drive already mounted")

# Define dataset paths for FIFA 2022
base_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco"

# Sequence features path
sequence_path = os.path.join(base_path, "Possession_Features/Sequence_of_5_Ordered_Encoded_Possession_Features.csv")

# Ball features path
ball_path = os.path.join(base_path, "Ball Locations/Normalized_Filtered_All_Matches_Without_England_France_Argentina_Croatia_Morocco.csv")

# Players features paths (4 files)
players_paths = [
    os.path.join(base_path, "Players Locations/Normalized_Players_Locations_3826_to_3840.csv"),
    os.path.join(base_path, "Players Locations/Normalized_Players_Locations_3841_to_3854.csv"),
    os.path.join(base_path, "Players Locations/Normalized_Players_Locations_3855_to_10509.csv"),
    os.path.join(base_path, "Players Locations/Normalized_Players_locations_3812_to_3825.csv")
]

# Model save path (single directory for one model)
model_save_path = os.path.join(base_path, "LSTM_Sequence_of_5/Model_Files")

print("\n📁 Dataset Paths:")
print(f"Sequence features path: {sequence_path}")
print(f"Ball features path: {ball_path}")
print("Players features paths:")
for i, path in enumerate(players_paths, 1):
    print(f"  Players file {i}: {path}")
print(f"Model save path: {model_save_path}")

# Ensure model directory exists with subfolders
os.makedirs(model_save_path, exist_ok=True)
os.makedirs(os.path.join(model_save_path, "model_checkpoints"), exist_ok=True)
os.makedirs(os.path.join(model_save_path, "training_artifacts"), exist_ok=True)
os.makedirs(os.path.join(model_save_path, "predictions"), exist_ok=True)
os.makedirs(os.path.join(model_save_path, "visualizations"), exist_ok=True)
print(f"\n✅ Model directory structure created at: {model_save_path}")

# Check GPU availability
print("\n🔍 GPU Availability Check:")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"  ✅ {len(gpus)} GPU(s) available for training")
    for i, gpu in enumerate(gpus):
        print(f"     GPU {i}: {gpu}")

    # Set memory growth to prevent TensorFlow from allocating all GPU memory at once
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("  ✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(f"  ❌ Error setting memory growth: {e}")
else:
    print("  ❌ No GPU available, using CPU")

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
print(f"\n🌱 Random seed set to {seed} for reproducibility")

# Verify dataset files exist
print("\n🔍 Verifying dataset files...")
dataset_files = [
    (sequence_path, "Sequence features"),
    (ball_path, "Ball features")
]

# Add all player files to verification
for i, player_path in enumerate(players_paths, 1):
    dataset_files.append((player_path, f"Players features file {i}"))

missing_files = []
for path, name in dataset_files:
    if os.path.exists(path):
        file_size = os.path.getsize(path) / (1024 * 1024)  # Convert to MB
        print(f"  ✅ {name} found ({file_size:.2f} MB)")
    else:
        print(f"  ❌ {name} NOT found at {path}")
        missing_files.append(name)

if missing_files:
    raise FileNotFoundError(f"Critical dataset files missing: {', '.join(missing_files)}")

print("\n✅ ENVIRONMENT SETUP COMPLETED SUCCESSFULLY")
print("Ready for next step: Data loading and feature engineering")
print(f"\n📊 Next step will process FIFA 2022 data")

== STEP 1: MODEL DEVELOPMENT ENVIRONMENT SETUP FOR FIFA 2022 SEQUENCE OF 5 ==
Google Drive already mounted

📁 Dataset Paths:
Sequence features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Possession_Features/Sequence_of_5_Ordered_Encoded_Possession_Features.csv
Ball features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Ball Locations/Normalized_Filtered_All_Matches_Without_England_France_Argentina_Croatia_Morocco.csv
Players features paths:
  Players file 1: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Players Locations/Normalized_Players_Locations_3826_to_3840.csv
  Players file 2: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Players Locations

In [ ]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("\n== STEP 2: DATA LOADING AND FEATURE ENGINEERING FOR FIFA 2022 SEQUENCE OF 5 ==")
start_time = time.time()

# 1. Load sequence features dataset
print("\n📊 Loading sequence features dataset...")
sequence_df = pd.read_csv(
    sequence_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'passerplayerid': 'float32',  # Use float32 to handle NaN values
        'receiverplayerid': 'float32',  # Use float32 to handle NaN values
        'passtype': 'int8',
        'passoutcometype': 'int8',
        'pressuretype': 'int8',
        'sequence_id': 'int32',
        'timestep': 'int8',
        'global_sequence_id': 'int32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'teamname', 'teamattackingdirection', 'passerplayerid', 'receiverplayerid',
             'passtype', 'passoutcometype', 'pressuretype', 'timestep', 'global_sequence_id', 'sequence_id']
)

print(f"   ✅ Sequence features loaded: {len(sequence_df):,} rows, {sequence_df.shape[1]} columns")

# 2. Load ball features dataset - CORRECTED: No sequence_id column
print("\n⚽ Loading ball features dataset...")
ball_df = pd.read_csv(
    ball_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'ball_x': 'float32',
        'ball_y': 'float32',
        'ball_z': 'float32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'ball_x', 'ball_y', 'ball_z']  # No sequence_id in this file
)

print(f"   ✅ Ball features loaded: {len(ball_df):,} rows, {ball_df.shape[1]} columns")

# 3. Load and combine players features datasets - CORRECTED: No sequence_id column
print("\n👥 Loading and combining players features datasets...")
all_players_dfs = []

for i, player_path in enumerate(players_paths, 1):
    print(f"   🔗 Loading players file {i}...")
    players_df = pd.read_csv(
        player_path,
        dtype={
            'gameid': 'int32',
            'gameeventid': 'int32',
            'possessioneventid': 'int32',
            'sequence': 'int32',
            'period': 'int8',
            'jerseynum': 'int8',
            'playerid': 'int32',
            'positiongrouptype': 'category',
            'x': 'float32',
            'y': 'float32'
        },
        usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
                 'jerseynum', 'team', 'visibility', 'confidence', 'x', 'y', 'playerid', 'positiongrouptype']  # No sequence_id in these files
    )
    all_players_dfs.append(players_df)
    print(f"      ✅ Players file {i} loaded: {len(players_df):,} rows")

# Combine all players datasets
print("   🔗 Combining all players datasets...")
players_df = pd.concat(all_players_dfs, ignore_index=True)
print(f"   ✅ Combined players features: {len(players_df):,} rows, {players_df.shape[1]} columns")

# 4. Data validation and basic statistics - CORRECTED: Use composite keys
print("\n🔍 Data validation and basic statistics:")

# Create the five join keys for all datasets
print("   🔑 Creating five join keys (gameid, possessioneventid, eventtime, sequence, period)...")
sequence_df['five_key'] = sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

ball_df['five_key'] = ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

players_df['five_key'] = players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

print("   ✅ Five join keys created successfully")

# Check for missing values in critical columns
print("\n   Missing values check:")
critical_columns = ['gameid', 'possessioneventid', 'eventtime', 'sequence', 'period', 'global_sequence_id']
for col in critical_columns:
    if col in sequence_df.columns:
        missing_count = sequence_df[col].isna().sum()
        print(f"     Sequence {col}: {missing_count} missing values")

# CORRECTED: Calculate unique game-sequence combinations for possessions
print("\n   🔍 Calculating unique possessions using (gameid, sequence) composite key...")
sequence_df['game_sequence_key'] = sequence_df.apply(lambda row: (row['gameid'], row['sequence']), axis=1)
unique_game_sequences = sequence_df['game_sequence_key'].nunique()
unique_global_sequences = sequence_df['global_sequence_id'].nunique()
total_timesteps = len(sequence_df)

print(f"\n   📊 CORRECTED dataset summary:")
print(f"     Unique global sequences: {unique_global_sequences:,} (globally unique 5-timestep sequences)")
print(f"     Unique game-sequence combinations: {unique_game_sequences:,} (unique possessions across all matches)")
print(f"     Total timesteps: {total_timesteps:,}")
print(f"     Average timesteps per global sequence: {total_timesteps/unique_global_sequences:.1f}")
print(f"     Average timesteps per possession: {total_timesteps/unique_game_sequences:.1f}")

# Check global_sequence_id distribution
global_seq_counts = sequence_df['global_sequence_id'].value_counts()
min_timesteps = global_seq_counts.min()
max_timesteps = global_seq_counts.max()
avg_timesteps = global_seq_counts.mean()

print(f"\n   🔢 Global sequence distribution:")
print(f"     Min timesteps per global sequence: {min_timesteps}")
print(f"     Max timesteps per global sequence: {max_timesteps}")
print(f"     Avg timesteps per global sequence: {avg_timesteps:.1f}")

# Check for the expected 5 timesteps per global sequence
expected_sequences = global_seq_counts[global_seq_counts == 5].shape[0]
unexpected_sequences = global_seq_counts[global_seq_counts != 5].shape[0]

print(f"\n   ⚠️ Global sequence validation (expecting 5 timesteps per sequence):")
print(f"     Sequences with exactly 5 timesteps: {expected_sequences:,} ({expected_sequences/unique_global_sequences*100:.1f}%)")
print(f"     Sequences with unexpected timestep count: {unexpected_sequences:,} ({unexpected_sequences/unique_global_sequences*100:.1f}%)")

if unexpected_sequences > 0:
    print("     🚨 WARNING: Some global sequences don't have exactly 5 timesteps!")
    print("            This may require filtering or investigation before training.")

# Store datasets for next steps
DATA = {
    'sequence_df': sequence_df,
    'ball_df': ball_df,
    'players_df': players_df
}

total_time = time.time() - start_time
print(f"\n✅ STEP 2 COMPLETE: Data loading and initial validation finished")
print(f"   ✅ All datasets loaded successfully")
print(f"   ✅ Basic validation completed with CORRECTED sequence counting")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Feature engineering and sequence construction for sequence of 5")



== STEP 2: DATA LOADING AND FEATURE ENGINEERING FOR FIFA 2022 SEQUENCE OF 5 ==

📊 Loading sequence features dataset...
   ✅ Sequence features loaded: 67,900 rows, 16 columns

⚽ Loading ball features dataset...
   ✅ Ball features loaded: 34,976 rows, 9 columns

👥 Loading and combining players features datasets...
   🔗 Loading players file 1...
      ✅ Players file 1 loaded: 214,545 rows
   🔗 Loading players file 2...
      ✅ Players file 2 loaded: 201,588 rows
   🔗 Loading players file 3...
      ✅ Players file 3 loaded: 176,700 rows
   🔗 Loading players file 4...
      ✅ Players file 4 loaded: 225,744 rows
   🔗 Combining all players datasets...
   ✅ Combined players features: 818,577 rows, 14 columns

🔍 Data validation and basic statistics:
   🔑 Creating five join keys (gameid, possessioneventid, eventtime, sequence, period)...
   ✅ Five join keys created successfully

   Missing values check:
     Sequence gameid: 0 missing values
     Sequence possessioneventid: 0 missing values
   

In [ ]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("\n== STEP 3: FEATURE ENGINEERING AND SEQUENCE CONSTRUCTION FOR FIFA 2022 SEQUENCE OF 5 ==")
start_time = time.time()

# 1. Create lookup dictionaries for faster joins
print("\n🔧 Creating lookup dictionaries for faster data joining...")
start_sub = time.time()

# Create ball lookup dictionary: five_key -> ball features
ball_lookup = ball_df.set_index('five_key')[['ball_x', 'ball_y', 'ball_z']].to_dict('index')

# Create players lookup dictionary: five_key -> player positions
# Group by the key and collect all player positions
players_grouped = {}
for key, group in players_df.groupby('five_key'):
    players_grouped[key] = group[['x', 'y', 'playerid', 'positiongrouptype', 'jerseynum', 'team']].to_dict('records')

# Create next timestep lookup for temporal context
# First, sort by global_sequence_id and timestep
sequence_df_sorted = sequence_df.sort_values(['global_sequence_id', 'timestep'])
# Create shifted columns for next timestep within the same global sequence
sequence_df_sorted['next_timestep'] = sequence_df_sorted.groupby('global_sequence_id')['timestep'].shift(-1)
sequence_df_sorted['next_eventtime'] = sequence_df_sorted.groupby('global_sequence_id')['eventtime'].shift(-1)

# Create lookup for next timestep context
next_timestep_lookup = {}
for idx, row in sequence_df_sorted.iterrows():
    if not pd.isna(row['next_timestep']) and row['next_timestep'] == row['timestep'] + 1:
        current_key = (
            row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
        )
        next_key = (
            row['gameid'], row['possessioneventid'], row['next_eventtime'], row['sequence'], row['period']
        )
        next_timestep_lookup[current_key] = {
            'next_ball_key': next_key,
            'next_passerplayerid': row['passerplayerid'] if not pd.isna(row['passerplayerid']) else -1,
            'next_receiverplayerid': row['receiverplayerid'] if not pd.isna(row['receiverplayerid']) else -1
        }

sub_time = time.time() - start_sub
print(f"   ✅ Lookup dictionaries built in {sub_time:.2f} seconds")

# 2. Get unique global sequences for training
print("\n📊 Getting unique global sequences...")
unique_global_sequences = sequence_df['global_sequence_id'].unique()
print(f"   📂 Total unique global sequences: {len(unique_global_sequences):,}")

# 3. Feature engineering with progress bar
print("\n⚙️ Engineering features for sequence of 5...")
start_sub = time.time()

# Initialize storage for sequences
X_sequences = []  # Input sequences (4 timesteps × 62 features)
y_sequences = []  # Target sequences (44 player coordinates for timestep 5)
valid_global_sequences = []  # Store valid global sequence IDs

# Create progress bar for sequence processing
seq_progress = tqdm(total=len(unique_global_sequences), desc="Building sequences", position=0, leave=True)

for global_seq_id in unique_global_sequences:
    # Get all timesteps for this global sequence
    seq_data = sequence_df[sequence_df['global_sequence_id'] == global_seq_id].sort_values('timestep')

    # Validate we have exactly 5 timesteps
    if len(seq_data) != 5:
        seq_progress.update(1)
        continue

    # Prepare input features (timesteps 1-4) and target (timestep 5)
    input_features = []
    has_missing_data = False

    # Process timesteps 1-4 for input
    for timestep in range(1, 5):  # Timesteps 1-4 for input
        row = seq_data[seq_data['timestep'] == timestep].iloc[0]

        # Create the five-key tuple for joining
        key = (row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period'])

        # Get ball features with fallback
        ball_features = ball_lookup.get(key, {'ball_x': 0.0, 'ball_y': 0.0, 'ball_z': 0.0})

        # Get player positions (44 features) with fallback
        player_positions = players_grouped.get(key, [])
        if len(player_positions) < 22:
            # Handle missing players by using (-500, -500) as default coordinates
            player_coords = np.zeros(44)
            for i in range(22):
                player_coords[i*2] = -500.0
                player_coords[i*2 + 1] = -500.0
            has_missing_data = True
        else:
            # Extract x,y coordinates for all 22 players in order
            player_coords = np.zeros(44)
            for i, player in enumerate(player_positions[:22]):  # Take first 22 players
                player_coords[i*2] = player['x']
                player_coords[i*2 + 1] = player['y']

        # Get event features (8 features)
        passer_id = row['passerplayerid'] if not pd.isna(row['passerplayerid']) else -1
        receiver_id = row['receiverplayerid'] if not pd.isna(row['receiverplayerid']) else -1

        # Get passer and receiver coordinates with fallback
        passer_coords = (-500.0, -500.0)  # Default for missing
        receiver_coords = (-500.0, -500.0)  # Default for missing

        if len(player_positions) >= 22:
            # Find passer and receiver in the player positions
            for player in player_positions:
                if player['playerid'] == passer_id:
                    passer_coords = (player['x'], player['y'])
                if player['playerid'] == receiver_id:
                    receiver_coords = (player['x'], player['y'])

        event_features = [
            row['passtype'] if not pd.isna(row['passtype']) else 0,
            row['passoutcometype'] if not pd.isna(row['passoutcometype']) else 0,
            row['pressuretype'] if not pd.isna(row['pressuretype']) else 0,
            row['period'],
            passer_coords[0], passer_coords[1],
            receiver_coords[0], receiver_coords[1]
        ]

        # Get next timestep context (7 features) for the next timestep in the sequence
        next_context = [0.0, 0.0, 0.0, -500.0, -500.0, -500.0, -500.0]  # Default values

        if key in next_timestep_lookup:
            next_info = next_timestep_lookup[key]
            next_ball_key = next_info['next_ball_key']
            next_ball = ball_lookup.get(next_ball_key, {'ball_x': 0.0, 'ball_y': 0.0, 'ball_z': 0.0})

            # Get next passer/receiver coordinates
            next_passer_coords = (-500.0, -500.0)
            next_receiver_coords = (-500.0, -500.0)

            if next_ball_key in players_grouped and len(players_grouped[next_ball_key]) >= 22:
                next_players = players_grouped[next_ball_key]
                for player in next_players:
                    if player['playerid'] == next_info['next_passerplayerid']:
                        next_passer_coords = (player['x'], player['y'])
                    if player['playerid'] == next_info['next_receiverplayerid']:
                        next_receiver_coords = (player['x'], player['y'])

            next_context = [
                next_ball['ball_x'], next_ball['ball_y'], next_ball['ball_z'],
                next_passer_coords[0], next_passer_coords[1],
                next_receiver_coords[0], next_receiver_coords[1]
            ]

        # Combine all features (44 + 8 + 3 + 7 = 62 features)
        timestep_features = np.concatenate([
            player_coords,
            event_features,
            [ball_features['ball_x'], ball_features['ball_y'], ball_features['ball_z']],
            next_context
        ])

        input_features.append(timestep_features)

    # Get target (timestep 5 player positions)
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    target_key = (timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
                 timestep5_row['sequence'], timestep5_row['period'])

    target_players = players_grouped.get(target_key, [])
    if len(target_players) >= 22 and not has_missing_data:
        target_coords = np.zeros(44)
        for i, player in enumerate(target_players[:22]):
            target_coords[i*2] = player['x']
            target_coords[i*2 + 1] = player['y']

        X_sequences.append(np.array(input_features))  # Shape: (4, 62)
        y_sequences.append(target_coords)  # Shape: (44,)
        valid_global_sequences.append(global_seq_id)

    seq_progress.update(1)

seq_progress.close()
sub_time = time.time() - start_sub
print(f"   ✅ Features engineered for {len(X_sequences):,}/{len(unique_global_sequences):,} sequences ({len(X_sequences)/len(unique_global_sequences)*100:.1f}%)")
print(f"   ⏱️  Feature engineering time: {sub_time:.2f} seconds")

# 4. Convert to numpy arrays and validate shapes
print("\n📊 Converting to numpy arrays and validating shapes...")
X = np.array(X_sequences)  # Shape: (num_sequences, 4, 62)
y = np.array(y_sequences)  # Shape: (num_sequences, 44)

print(f"\n✅ Final dataset shapes:")
print(f"   Input (X): {X.shape} - (sequences, timesteps, features)")
print(f"   Target (y): {y.shape} - (sequences, player_coordinates)")
print(f"   Features per timestep: {X.shape[2]} (should be 62)")
print(f"   Player coordinates: {y.shape[1]} (should be 44)")

# Validate feature count
assert X.shape[2] == 62, f"Expected 62 features per timestep, got {X.shape[2]}"
assert y.shape[1] == 44, f"Expected 44 target coordinates, got {y.shape[1]}"

# Store for next steps
SEQUENCE_DATA = {
    'X': X,
    'y': y,
    'valid_global_sequences': valid_global_sequences,
    'sequence_df': sequence_df  # Keep for data splitting with temporal ordering
}

total_time = time.time() - start_time
print(f"\n✅ STEP 3 COMPLETE: Feature engineering and sequence construction finished")
print(f"   ✅ Successfully processed {len(X_sequences):,} valid sequences")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Data splitting with temporal ordering and model architecture definition")
print("Note: Missing players handled with (-500, -500) coordinates as specified")


== STEP 3: FEATURE ENGINEERING AND SEQUENCE CONSTRUCTION FOR FIFA 2022 SEQUENCE OF 5 ==

🔧 Creating lookup dictionaries for faster data joining...
   ✅ Lookup dictionaries built in 49.34 seconds

📊 Getting unique global sequences...
   📂 Total unique global sequences: 13,580

⚙️ Engineering features for sequence of 5...


Building sequences: 100%|██████████| 13580/13580 [00:42<00:00, 320.59it/s]


   ✅ Features engineered for 13,568/13,580 sequences (99.9%)
   ⏱️  Feature engineering time: 42.36 seconds

📊 Converting to numpy arrays and validating shapes...

✅ Final dataset shapes:
   Input (X): (13568, 4, 62) - (sequences, timesteps, features)
   Target (y): (13568, 44) - (sequences, player_coordinates)
   Features per timestep: 62 (should be 62)
   Player coordinates: 44 (should be 44)

✅ STEP 3 COMPLETE: Feature engineering and sequence construction finished
   ✅ Successfully processed 13,568 valid sequences
   ⏱️  Total execution time: 91.73 seconds

Next step: Data splitting with temporal ordering and model architecture definition
Note: Missing players handled with (-500, -500) coordinates as specified


In [ ]:
import time
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

print("\n== STEP 4: DATA SPLITTING WITH TEMPORAL ORDERING AND MODEL ARCHITECTURE DEFINITION ==")
start_time = time.time()

# 1. Data splitting with dual-level leakage prevention
print("\n🔍 Splitting data with dual-level leakage prevention...")
sequence_df = SEQUENCE_DATA['sequence_df']

# Level 1: Cross-split leakage prevention - split by unique (gameid, sequence) combinations
print("   Level 1: Preventing cross-split leakage (by unique gameid + sequence combinations)...")
# Create composite key for unique possessions
sequence_df['game_sequence_key'] = sequence_df.apply(lambda row: (row['gameid'], row['sequence']), axis=1)
unique_game_sequences = sequence_df['game_sequence_key'].unique()
print(f"   Total unique game-sequence combinations: {len(unique_game_sequences):,}")

# Split by game-sequence combinations (80/10/10)
train_game_sequences, temp_game_sequences = train_test_split(unique_game_sequences, test_size=0.2, random_state=42)
val_game_sequences, test_game_sequences = train_test_split(temp_game_sequences, test_size=0.5, random_state=42)

print(f"   Train game-sequences: {len(train_game_sequences):,} ({len(train_game_sequences)/len(unique_game_sequences)*100:.1f}%)")
print(f"   Validation game-sequences: {len(val_game_sequences):,} ({len(val_game_sequences)/len(unique_game_sequences)*100:.1f}%)")
print(f"   Test game-sequences: {len(test_game_sequences):,} ({len(test_game_sequences)/len(unique_game_sequences)*100:.1f}%)")

# Level 2: Within-split leakage prevention - sort by global_sequence_id within each split
print("\n   Level 2: Preventing within-split leakage (by global_sequence_id ordering)...")
# Get global_sequence_id for each split
train_global_ids = sequence_df[sequence_df['game_sequence_key'].isin(train_game_sequences)]['global_sequence_id'].unique()
val_global_ids = sequence_df[sequence_df['game_sequence_key'].isin(val_game_sequences)]['global_sequence_id'].unique()
test_global_ids = sequence_df[sequence_df['game_sequence_key'].isin(test_game_sequences)]['global_sequence_id'].unique()

# Sort each split by global_sequence_id to ensure temporal order
train_global_ids = sorted(train_global_ids)
val_global_ids = sorted(val_global_ids)
test_global_ids = sorted(test_global_ids)

print(f"\n   Global sequences distribution (sorted by global_sequence_id):")
print(f"     Train: {len(train_global_ids):,} ({len(train_global_ids)/len(SEQUENCE_DATA['valid_global_sequences'])*100:.1f}%)")
print(f"     Validation: {len(val_global_ids):,} ({len(val_global_ids)/len(SEQUENCE_DATA['valid_global_sequences'])*100:.1f}%)")
print(f"     Test: {len(test_global_ids):,} ({len(test_global_ids)/len(SEQUENCE_DATA['valid_global_sequences'])*100:.1f}%)")

# 2. Create masks for splitting with temporal ordering
print("\n📋 Creating masks for temporal data splitting...")
# Create mapping from global_sequence_id to index in SEQUENCE_DATA
global_seq_to_idx = {seq_id: idx for idx, seq_id in enumerate(SEQUENCE_DATA['valid_global_sequences'])}

# Get indices for each split in temporal order
train_indices = [global_seq_to_idx[seq_id] for seq_id in train_global_ids if seq_id in global_seq_to_idx]
val_indices = [global_seq_to_idx[seq_id] for seq_id in val_global_ids if seq_id in global_seq_to_idx]
test_indices = [global_seq_to_idx[seq_id] for seq_id in test_global_ids if seq_id in global_seq_to_idx]

# Split the data
X_train = SEQUENCE_DATA['X'][train_indices]
y_train = SEQUENCE_DATA['y'][train_indices]
X_val = SEQUENCE_DATA['X'][val_indices]
y_val = SEQUENCE_DATA['y'][val_indices]
X_test = SEQUENCE_DATA['X'][test_indices]
y_test = SEQUENCE_DATA['y'][test_indices]

print(f"\n📊 Final split shapes (with temporal ordering):")
print(f"   Train: X={X_train.shape}, y={y_train.shape}")
print(f"   Validation: X={X_val.shape}, y={y_val.shape}")
print(f"   Test: X={X_test.shape}, y={y_test.shape}")

# 3. Build LSTM model architecture (sequence of 5 input)
print("\n🏗️ Building LSTM model architecture for sequence of 5...")
input_shape = (4, 62)  # 4 timesteps, 62 features per timestep
output_size = 44       # 22 players × 2 coordinates

# Input layer
input_layer = Input(shape=input_shape, name='input_layer')

# First LSTM layer
x = LSTM(128, return_sequences=True, name='lstm_1')(input_layer)
x = BatchNormalization(name='batch_norm_1')(x)
x = Dropout(0.3, name='dropout_1')(x)

# Second LSTM layer
x = LSTM(64, name='lstm_2')(x)
x = BatchNormalization(name='batch_norm_2')(x)
x = Dropout(0.3, name='dropout_2')(x)

# Dense layers
x = Dense(128, activation='relu', name='dense_1')(x)
x = Dense(64, activation='relu', name='dense_2')(x)

# Output layer
output_layer = Dense(output_size, name='output_layer')(x)

# Create model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("\n✅ Model architecture built successfully")
model.summary()

# Save model summary to file
model_summary_path = os.path.join(model_save_path, "training_artifacts", "model_summary.txt")
os.makedirs(os.path.dirname(model_summary_path), exist_ok=True)
with open(model_summary_path, 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
print(f"   📝 Model summary saved to: {model_summary_path}")

# Store for next steps
TRAINING_DATA = {
    'X_train': X_train,
    'y_train': y_train,
    'X_val': X_val,
    'y_val': y_val,
    'X_test': X_test,
    'y_test': y_test,
    'train_global_ids': train_global_ids,
    'val_global_ids': val_global_ids,
    'test_global_ids': test_global_ids
}

MODEL = model

total_time = time.time() - start_time
print(f"\n✅ STEP 4 COMPLETE: Data splitting and model architecture finished")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Model training with callbacks and temporal monitoring")
print(f"✅ Fixed: Proper handling of sequence uniqueness using (gameid, sequence) composite key")


== STEP 4: DATA SPLITTING WITH TEMPORAL ORDERING AND MODEL ARCHITECTURE DEFINITION ==

🔍 Splitting data with dual-level leakage prevention...
   Level 1: Preventing cross-split leakage (by unique gameid + sequence combinations)...
   Total unique game-sequence combinations: 2,442
   Train game-sequences: 1,953 (80.0%)
   Validation game-sequences: 244 (10.0%)
   Test game-sequences: 245 (10.0%)

   Level 2: Preventing within-split leakage (by global_sequence_id ordering)...

   Global sequences distribution (sorted by global_sequence_id):
     Train: 10,671 (78.6%)
     Validation: 1,403 (10.3%)
     Test: 1,506 (11.1%)

📋 Creating masks for temporal data splitting...

📊 Final split shapes (with temporal ordering):
   Train: X=(10664, 4, 62), y=(10664, 44)
   Validation: X=(1399, 4, 62), y=(1399, 44)
   Test: X=(1505, 4, 62), y=(1505, 44)

🏗️ Building LSTM model architecture for sequence of 5...

✅ Model architecture built successfully


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4, 62)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 4, 128)         │        97,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_norm_1                    │ (None, 4, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_norm_2                    │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 44)             │         2,860 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 167,404 (653.92 KB)

 Trainable params: 167,020 (652.42 KB)

 Non-trainable params: 384 (1.50 KB)

   📝 Model summary saved to: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/training_artifacts/model_summary.txt

✅ STEP 4 COMPLETE: Data splitting and model architecture finished
   ⏱️  Total execution time: 6.89 seconds

Next step: Model training with callbacks and temporal monitoring
✅ Fixed: Proper handling of sequence uniqueness using (gameid, sequence) composite key


In [ ]:
import time
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
import json
import os
from datetime import datetime

print("\n== STEP 5: MODEL TRAINING WITH CALLBACKS AND TEMPORAL MONITORING ==")
start_time = time.time()

# 1. Define callbacks with temporal monitoring
print("\n⚙️ Configuring training callbacks...")
callbacks = []

# Model checkpointing - save best model based on validation loss
checkpoint_path = os.path.join(model_save_path, "model_checkpoints", "best_model_epoch_{epoch:02d}_val_loss_{val_loss:.6f}.keras")
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)
callbacks.append(model_checkpoint)

# Early stopping - stop training when validation loss stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)
callbacks.append(early_stopping)

# Learning rate reduction - reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1,
    mode='min'
)
callbacks.append(reduce_lr)

print("   ✅ Callbacks configured successfully")

# 2. Train the model with temporal monitoring
print("\n🚀 Starting model training with temporal ordering...")
print(f"   Training configuration:")
print(f"   - Batch size: 64")
print(f"   - Maximum epochs: 100")
print(f"   - Training samples: {len(TRAINING_DATA['X_train'])}")
print(f"   - Validation samples: {len(TRAINING_DATA['X_val'])}")
print(f"   - Input shape: (4, 62) - 4 timesteps, 62 features per timestep")
print(f"   - Temporal ordering: Within-split sequences sorted by global_sequence_id")

# Train with temporal ordering (already handled in data preparation)
history = MODEL.fit(
    TRAINING_DATA['X_train'],
    TRAINING_DATA['y_train'],
    validation_data=(TRAINING_DATA['X_val'], TRAINING_DATA['y_val']),
    batch_size=64,
    epochs=100,
    callbacks=callbacks,
    verbose=1
)

# 3. Save training history
print("\n💾 Saving training history and artifacts...")
training_history = {
    'loss': [float(x) for x in history.history['loss']],
    'val_loss': [float(x) for x in history.history['val_loss']],
    'mae': [float(x) for x in history.history['mae']],
    'val_mae': [float(x) for x in history.history['val_mae']]
}

history_path = os.path.join(model_save_path, "training_artifacts", "training_history.json")
os.makedirs(os.path.dirname(history_path), exist_ok=True)
with open(history_path, 'w') as f:
    json.dump(training_history, f, indent=2)
print(f"   ✅ Training history saved to: {history_path}")

# 4. Create training visualizations
print("\n📊 Creating training visualizations...")

plt.figure(figsize=(15, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(training_history['loss'], label='Training Loss (MSE)')
plt.plot(training_history['val_loss'], label='Validation Loss (MSE)')
plt.title('Model Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot MAE
plt.subplot(1, 2, 2)
plt.plot(training_history['mae'], label='Training MAE')
plt.plot(training_history['val_mae'], label='Validation MAE')
plt.title('Model MAE Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plot_path = os.path.join(model_save_path, "visualizations", "training_curves.png")
os.makedirs(os.path.dirname(plot_path), exist_ok=True)
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Training curves saved to: {plot_path}")
plt.close()

# 5. Find and load the best model
print("\n🏆 Loading the best model from checkpoints...")
checkpoint_dir = os.path.join(model_save_path, "model_checkpoints")
checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.keras')]

if checkpoint_files:
    # Sort by validation loss (lower is better)
    checkpoint_files.sort(key=lambda x: float(x.split('_val_loss_')[1].split('.keras')[0]))
    best_model_path = os.path.join(checkpoint_dir, checkpoint_files[0])
    print(f"   📥 Loading best model from: {best_model_path}")
    best_model = tf.keras.models.load_model(best_model_path)
    print(f"   ✅ Best model loaded successfully")
else:
    print("   ⚠️  No checkpoint files found, using current model")
    best_model = MODEL

# 6. Evaluate on test set
print("\n🔍 Evaluating model on test set...")
test_loss, test_mae = best_model.evaluate(
    TRAINING_DATA['X_test'],
    TRAINING_DATA['y_test'],
    verbose=0
)

test_rmse = np.sqrt(test_loss)
print(f"\n✅ TEST SET PERFORMANCE:")
print(f"   Test MSE: {test_loss:.6f}")
print(f"   Test MAE: {test_mae:.6f}")
print(f"   Test RMSE: {test_rmse:.6f}")

# Save evaluation metrics
metrics = {
    'test_loss': float(test_loss),
    'test_mae': float(test_mae),
    'test_rmse': float(test_rmse),
    'val_loss': float(min(training_history['val_loss'])),
    'train_loss': float(min(training_history['loss'])),
    'epochs_trained': len(training_history['loss'])
}

metrics_path = os.path.join(model_save_path, "training_artifacts", "evaluation_metrics.json")
os.makedirs(os.path.dirname(metrics_path), exist_ok=True)
with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"   💾 Evaluation metrics saved to: {metrics_path}")

# Store for next steps
TRAINED_MODEL = best_model
EVALUATION_METRICS = metrics

total_time = time.time() - start_time
print(f"\n✅ STEP 5 COMPLETE: Model training and evaluation finished")
print(f"   📊 Final test performance: MSE={test_loss:.6f}, MAE={test_mae:.6f}, RMSE={test_rmse:.6f}")
print(f"   ⏱️  Total training time: {total_time:.2f} seconds")
print(f"   📁 All artifacts saved to: {model_save_path}")
print("\nNext step: Generating predictions and creating analysis reports with temporal integrity")


== STEP 5: MODEL TRAINING WITH CALLBACKS AND TEMPORAL MONITORING ==

⚙️ Configuring training callbacks...
   ✅ Callbacks configured successfully

🚀 Starting model training with temporal ordering...
   Training configuration:
   - Batch size: 64
   - Maximum epochs: 100
   - Training samples: 10664
   - Validation samples: 1399
   - Input shape: (4, 62) - 4 timesteps, 62 features per timestep
   - Temporal ordering: Within-split sequences sorted by global_sequence_id
Epoch 1/100
161/167 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 303.7370 - mae: 13.8652
Epoch 1: val_loss improved from inf to 240.12424, saving model to /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/model_checkpoints/best_model_epoch_01_val_loss_240.124237.keras
167/167 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - loss: 301.6973 - mae: 13.8120 - val_loss: 240.1242 - val_mae: 12.4201 - learning_rate: 0.0010
Epoch 2/100
165

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from tqdm import tqdm
import os
from datetime import datetime

print("\n== STEP 6: GENERATING PREDICTIONS AND ANALYSIS REPORTS WITH TEMPORAL INTEGRITY ==")
start_time = time.time()

# 1. Generate predictions for test set
print("\n🔮 Generating predictions for test set...")
test_predictions = TRAINED_MODEL.predict(TRAINING_DATA['X_test'], verbose=1)
print(f"   ✅ Predictions generated: {test_predictions.shape}")

# 2. Create the five join keys for data merging (recreate if needed)
print("\n🔑 Recreating five join keys for data integrity...")
sequence_df['five_key'] = sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

ball_df['five_key'] = ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

players_df['five_key'] = players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

# 3. Get test sequences and create test files
print("\n📁 Creating test files with original structure...")

# 3.1 Get test sequence data - CORRECTED: Use only sequences that exist in test set
test_global_ids = TRAINING_DATA['test_global_ids'][0:len(test_predictions)]  # Match prediction count
test_sequence_data = sequence_df[sequence_df['global_sequence_id'].isin(test_global_ids)]
test_five_keys = test_sequence_data['five_key'].unique()

# 3.2 Ball features test data
test_ball_data = ball_df[ball_df['five_key'].isin(test_five_keys)]
ball_features_path = os.path.join(model_save_path, "predictions", "ball_features_test.csv")
os.makedirs(os.path.dirname(ball_features_path), exist_ok=True)
test_ball_data.to_csv(ball_features_path, index=False)
print(f"   ⚽ Ball features test data saved: {len(test_ball_data)} rows")

# 3.3 Possession features test data
test_possession_data = sequence_df[sequence_df['global_sequence_id'].isin(test_global_ids)]
possession_features_path = os.path.join(model_save_path, "predictions", "possession_features_test.csv")
os.makedirs(os.path.dirname(possession_features_path), exist_ok=True)
test_possession_data.to_csv(possession_features_path, index=False)
print(f"   📋 Possession features test data saved: {len(test_possession_data)} rows")

# 3.4 Players test data
test_players_data = players_df[players_df['five_key'].isin(test_five_keys)]
players_features_path = os.path.join(model_save_path, "predictions", "players_test.csv")
os.makedirs(os.path.dirname(players_features_path), exist_ok=True)
test_players_data.to_csv(players_features_path, index=False)
print(f"   👥 Players test data saved: {len(test_players_data)} rows")

# 4. Create predicted players CSV with exact structure - CORRECTED: Use filtered test_global_ids
print("\n🎯 Creating predicted players CSV with exact structure...")

# Create list to store prediction rows
prediction_rows = []

# Create progress bar - CORRECTED: Use the actual number of predictions
progress = tqdm(total=len(test_predictions), desc="Building prediction CSV", position=0, leave=True)

for i, global_seq_id in enumerate(test_global_ids):  # Use filtered list
    # Get sequence data for this global sequence
    seq_data = sequence_df[sequence_df['global_sequence_id'] == global_seq_id].sort_values('timestep')

    if len(seq_data) != 5:  # Sequence of 5 has 5 timesteps
        progress.update(1)
        continue

    # Get predicted coordinates for timestep 5 - NOW SAFE (i < len(test_predictions))
    predicted_coords = test_predictions[i]

    # Process each timestep (1-4) for actual data
    for timestep in range(1, 5):  # Timesteps 1-4 for actual data
        timestep_row = seq_data[seq_data['timestep'] == timestep].iloc[0]
        key = (
            timestep_row['gameid'], timestep_row['possessioneventid'], timestep_row['eventtime'],
            timestep_row['sequence'], timestep_row['period']
        )

        # Get player data for this timestep
        players_for_timestep = players_df[players_df['five_key'] == key]

        if len(players_for_timestep) < 22:
            continue

        # Add actual player positions (22 players per timestep)
        for _, player_row in players_for_timestep.head(22).iterrows():
            row_dict = {
                'gameid': timestep_row['gameid'],
                'gameeventid': timestep_row['gameeventid'],
                'possessioneventid': timestep_row['possessioneventid'],
                'eventtime': timestep_row['eventtime'],
                'playerid': player_row['playerid'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'timestep': timestep,
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': timestep_row['sequence_id'],
                'global_sequence_id': timestep_row['global_sequence_id']
            }
            prediction_rows.append(row_dict)

    # Add timestep 5 actual data
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    key = (
        timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
        timestep5_row['sequence'], timestep5_row['period']
    )

    players_for_timestep = players_df[players_df['five_key'] == key]

    if len(players_for_timestep) >= 22:
        for _, player_row in players_for_timestep.head(22).iterrows():
            row_dict = {
                'gameid': timestep5_row['gameid'],
                'gameeventid': timestep5_row['gameeventid'],
                'possessioneventid': timestep5_row['possessioneventid'],
                'eventtime': timestep5_row['eventtime'],
                'playerid': player_row['playerid'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'timestep': 5,
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': timestep5_row['sequence_id'],
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            prediction_rows.append(row_dict)

    # Add timestep 5 predicted data
    if len(players_for_timestep) >= 22:
        for j in range(22):
            player_row = players_for_timestep.iloc[j]
            row_dict = {
                'gameid': timestep5_row['gameid'],
                'gameeventid': timestep5_row['gameeventid'],
                'possessioneventid': timestep5_row['possessioneventid'],
                'eventtime': timestep5_row['eventtime'],
                'playerid': player_row['playerid'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': predicted_coords[j*2],
                'y': predicted_coords[j*2 + 1],
                'timestep': 5,
                'is_predicted': 1,
                'data_type': 'predicted',
                'sequence_id': timestep5_row['sequence_id'],
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            prediction_rows.append(row_dict)

    progress.update(1)

progress.close()

# 5. Create and save prediction DataFrame
print("\n💾 Saving predicted players CSV...")
prediction_df = pd.DataFrame(prediction_rows)

# Ensure all required columns exist
required_columns = ['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'playerid', 'jerseynum',
                   'team', 'x', 'y', 'timestep', 'is_predicted', 'data_type', 'sequence_id', 'global_sequence_id']

# Add any missing columns with default values
for col in required_columns:
    if col not in prediction_df.columns:
        if col in ['gameid', 'gameeventid', 'possessioneventid', 'playerid', 'jerseynum', 'timestep', 'is_predicted']:
            prediction_df[col] = 0
        elif col in ['x', 'y']:
            prediction_df[col] = 0.0
        elif col in ['team']:
            prediction_df[col] = 'Unknown'
        else:
            prediction_df[col] = 'missing'

# Reorder columns to match exact structure
prediction_df = prediction_df[required_columns]

predicted_players_path = os.path.join(model_save_path, "predictions", "predicted_players.csv")
os.makedirs(os.path.dirname(predicted_players_path), exist_ok=True)
prediction_df.to_csv(predicted_players_path, index=False)
print(f"   ✅ Predicted players CSV saved: {len(prediction_df)} rows")
print(f"      • Actual data rows: {len(prediction_df[prediction_df['data_type'] == 'actual'])}")
print(f"      • Predicted data rows: {len(prediction_df[prediction_df['data_type'] == 'predicted'])}")

# 6. Calculate comprehensive metrics for all datasets
print("\n📈 Calculating comprehensive metrics for all datasets...")

def calculate_metrics(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))
    r2 = r2_score(y_true.flatten(), y_pred.flatten())
    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

train_predictions = TRAINED_MODEL.predict(TRAINING_DATA['X_train'], verbose=0)
val_predictions = TRAINED_MODEL.predict(TRAINING_DATA['X_val'], verbose=0)

train_metrics = calculate_metrics(TRAINING_DATA['y_train'], train_predictions)
val_metrics = calculate_metrics(TRAINING_DATA['y_val'], val_predictions)
test_metrics = calculate_metrics(TRAINING_DATA['y_test'], test_predictions)

comparison_df = pd.DataFrame({
    'Dataset': ['Train', 'Validation', 'Test'],
    'MSE': [train_metrics['mse'], val_metrics['mse'], test_metrics['mse']],
    'RMSE': [train_metrics['rmse'], val_metrics['rmse'], test_metrics['rmse']],
    'MAE': [train_metrics['mae'], val_metrics['mae'], test_metrics['mae']],
    'R²': [train_metrics['r2'], val_metrics['r2'], test_metrics['r2']],
    'Sample Size': [len(TRAINING_DATA['X_train']), len(TRAINING_DATA['X_val']), len(TRAINING_DATA['X_test'])]
})

print("\n📊 Performance Comparison Table:")
print(comparison_df.to_string(index=False))

# 7. Save metrics and create visualizations
print("\n🎨 Creating analysis visualizations and reports...")

# Save comparison table
comparison_path = os.path.join(model_save_path, "training_artifacts", "performance_comparison.csv")
os.makedirs(os.path.dirname(comparison_path), exist_ok=True)
comparison_df.to_csv(comparison_path, index=False)
print(f"   💾 Performance comparison saved to: {comparison_path}")

# Create error analysis visualization
plt.figure(figsize=(15, 6))

# Calculate errors for test set
errors = np.abs(TRAINING_DATA['y_test'] - test_predictions)
player_errors = errors.reshape(-1, 22, 2)  # (samples, players, coordinates)
avg_player_errors = np.mean(player_errors, axis=(0, 2))  # Average error per player

plt.subplot(1, 2, 1)
plt.bar(range(1, 23), avg_player_errors, color='skyblue')
plt.title('Average Error per Player Position')
plt.xlabel('Player Position (1-22)')
plt.ylabel('MAE')
plt.xticks(range(1, 23), [f'P{i}' for i in range(1, 23)], rotation=45)

plt.subplot(1, 2, 2)
all_errors = errors.flatten()
plt.hist(all_errors, bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
plt.axvline(np.mean(all_errors), color='red', linestyle='dashed', linewidth=2, label=f'Mean: {np.mean(all_errors):.2f}')
plt.title('Error Distribution')
plt.xlabel('Absolute Error')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
error_path = os.path.join(model_save_path, "visualizations", "error_analysis.png")
os.makedirs(os.path.dirname(error_path), exist_ok=True)
plt.savefig(error_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Error analysis visualization saved to: {error_path}")
plt.close()

# 8. Generate pitch visualization with actual vs predicted
plt.figure(figsize=(20, 8))

# Select a few representative sequences to visualize - CORRECTED: Use filtered test_global_ids
num_examples = min(4, len(test_global_ids))
example_indices = np.random.choice(len(test_global_ids), num_examples, replace=False)

for idx, example_idx in enumerate(example_indices):
    global_seq_id = test_global_ids[example_idx]  # Use filtered list
    actual_coords = TRAINING_DATA['y_test'][example_idx]
    pred_coords = test_predictions[example_idx]

    ax = plt.subplot(1, num_examples, idx+1)

    # Create pitch
    ax.set_xlim(-55, 55)
    ax.set_ylim(-35, 35)
    ax.set_aspect('equal')
    ax.set_title(f'Sequence {global_seq_id}', fontsize=10)

    # Draw pitch markings
    ax.plot([-52.5, 52.5], [-34, -34], 'k-')  # Bottom
    ax.plot([-52.5, 52.5], [34, 34], 'k-')    # Top
    ax.plot([-52.5, -52.5], [-34, 34], 'k-')  # Left
    ax.plot([52.5, 52.5], [-34, 34], 'k-')    # Right
    ax.plot([0, 0], [-34, 34], 'k--')        # Center line

    # Plot actual positions (blue)
    actual_x = actual_coords[::2]
    actual_y = actual_coords[1::2]
    ax.scatter(actual_x[:11], actual_y[:11], c='blue', s=50, alpha=0.7, label='Actual Home')
    ax.scatter(actual_x[11:], actual_y[11:], c='red', s=50, alpha=0.7, label='Actual Away')

    # Plot predicted positions (green)
    pred_x = pred_coords[::2]
    pred_y = pred_coords[1::2]
    ax.scatter(pred_x[:11], pred_y[:11], c='lightgreen', s=50, marker='x', label='Predicted Home')
    ax.scatter(pred_x[11:], pred_y[11:], c='pink', s=50, marker='x', label='Predicted Away')

    # Draw error vectors
    for j in range(22):
        dx = pred_x[j] - actual_x[j]
        dy = pred_y[j] - actual_y[j]
        ax.arrow(actual_x[j], actual_y[j], dx, dy, color='black', alpha=0.5, width=0.1)

    # Turn off axis ticks and labels for cleaner look
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
pitch_path = os.path.join(model_save_path, "visualizations", "actual_vs_predicted_formations.png")
os.makedirs(os.path.dirname(pitch_path), exist_ok=True)
plt.savefig(pitch_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Pitch visualization saved to: {pitch_path}")
plt.close()

# 9. Generate comprehensive analysis report
print("\n📝 Generating comprehensive analysis report...")

report_path = os.path.join(model_save_path, "training_artifacts", f"analysis_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
os.makedirs(os.path.dirname(report_path), exist_ok=True)

with open(report_path, 'w') as f:
    f.write("="*80 + "\n")
    f.write("FIFA 2022 FORMATION PREDICTION - COMPREHENSIVE ANALYSIS REPORT (SEQUENCE OF 5)\n")
    f.write("="*80 + "\n\n")

    f.write(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

    f.write("MODEL PERFORMANCE SUMMARY:\n")
    f.write("-"*50 + "\n")
    f.write(f"Architecture: LSTM (128 units) → LSTM (64 units) → Dense (128) → Dense (64) → Output (44)\n")
    f.write(f"Input Shape: (4, 62) - 4 timesteps, 62 features each (Sequence of 5)\n")
    f.write(f"Output Shape: (44) - 22 players × 2 coordinates\n")
    f.write(f"Total Parameters: 167,404\n\n")

    f.write("PERFORMANCE METRICS COMPARISON:\n")
    f.write("-"*50 + "\n")
    f.write(comparison_df.to_string(index=False))
    f.write("\n\n")

    f.write("KEY INSIGHTS:\n")
    f.write("-"*50 + "\n")
    f.write(f"• Best Validation Loss: {np.min(comparison_df[comparison_df['Dataset'] == 'Validation']['MSE'].values):.4f}\n")
    f.write(f"• Test Set Performance: MSE={test_metrics['mse']:.4f}, MAE={test_metrics['mae']:.4f}, R²={test_metrics['r2']:.4f}\n")
    f.write(f"• Generalization Gap: {test_metrics['mse'] - val_metrics['mse']:.4f} (Test MSE - Validation MSE)\n")
    f.write(f"• Average Positioning Error: {test_metrics['mae']:.2f} units on a 105-unit pitch\n")
    f.write(f"• Total Test Sequences: {len(test_global_ids)}\n")  # Use filtered count
    f.write(f"• Total Prediction Rows: {len(prediction_df)}\n\n")

    f.write("EXPORTED TEST FILES:\n")
    f.write("-"*50 + "\n")
    f.write(f"1. Ball Features Test Data: {ball_features_path}\n")
    f.write(f"   - Rows: {len(test_ball_data)}\n")
    f.write(f"   - Columns: {', '.join(test_ball_data.columns)}\n\n")

    f.write(f"2. Possession Features Test Data: {possession_features_path}\n")
    f.write(f"   - Rows: {len(test_possession_data)}\n")
    f.write(f"   - Columns: {', '.join(test_possession_data.columns)}\n\n")

    f.write(f"3. Players Test Data: {players_features_path}\n")
    f.write(f"   - Rows: {len(test_players_data)}\n")
    f.write(f"   - Columns: {', '.join(test_players_data.columns)}\n\n")

    f.write(f"4. Predicted Players Data: {predicted_players_path}\n")
    f.write(f"   - Rows: {len(prediction_df)}\n")
    f.write(f"   - Columns: {', '.join(prediction_df.columns)}\n")
    f.write(f"   - Structure: {len(prediction_df[prediction_df['data_type'] == 'actual'])} actual rows + {len(prediction_df[prediction_df['data_type'] == 'predicted'])} predicted rows\n\n")

    f.write("TEMPORAL INTEGRITY GUARANTEE:\n")
    f.write("-"*50 + "\n")
    f.write("• Cross-split leakage prevented: No game-sequence combination appears in multiple splits\n")
    f.write("• Within-split leakage prevented: Global sequences sorted by ID within each split\n")
    f.write("• Temporal ordering maintained: Model trained on chronologically ordered sequences\n")
    f.write("• Data integrity verified: All joins use the five-key system (gameid, possessioneventid, eventtime, sequence, period)\n")
    f.write("• Sequence uniqueness handled: (gameid, sequence) composite key used for splitting\n\n")

    f.write("MISSING DATA HANDLING:\n")
    f.write("-"*50 + "\n")
    f.write("• Missing players: (-500, -500) coordinates used for missing player positions\n")
    f.write("• Missing passer/receiver: (-500, -500) coordinates and -1 player IDs used\n")
    f.write("• No spatial normalization: All coordinates used as-is from input files\n")

print(f"   ✅ Analysis report saved to: {report_path}")

total_time = time.time() - start_time
print(f"\n✅ STEP 6 COMPLETE: Predictions and analysis reports generated")
print(f"   📊 Test set performance: MSE={test_metrics['mse']:.4f}, MAE={test_metrics['mae']:.4f}, R²={test_metrics['r2']:.4f}")
print(f"   💾 All artifacts saved to: {model_save_path}")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\n🎉 🎉 🎉 PROJECT COMPLETED SUCCESSFULLY! 🎉 🎉 🎉")
print(f"\n📥 FINAL MODEL AND ARTIFACTS SAVED TO:")
print(f"   {model_save_path}")
print("\n📊 KEY OUTPUT FILES:")
print(f"   • Model: {os.path.join(model_save_path, 'model_checkpoints', 'best_model_epoch_99_val_loss_80.440567.keras')}")
print(f"   • Ball Features Test: {ball_features_path}")
print(f"   • Possession Features Test: {possession_features_path}")
print(f"   • Players Test: {players_features_path}")
print(f"   • Predicted Players: {predicted_players_path}")
print(f"   • Performance Comparison: {comparison_path}")
print(f"   • Error Analysis: {error_path}")
print(f"   • Pitch Visualization: {pitch_path}")
print(f"   • Analysis Report: {report_path}")


== STEP 6: GENERATING PREDICTIONS AND ANALYSIS REPORTS WITH TEMPORAL INTEGRITY ==

🔮 Generating predictions for test set...
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
   ✅ Predictions generated: (1505, 44)

🔑 Recreating five join keys for data integrity...

📁 Creating test files with original structure...
   ⚽ Ball features test data saved: 2485 rows
   📋 Possession features test data saved: 7525 rows
   👥 Players test data saved: 54665 rows

🎯 Creating predicted players CSV with exact structure...


Building prediction CSV: 100%|██████████| 1505/1505 [15:05<00:00,  1.66it/s]



💾 Saving predicted players CSV...
   ✅ Predicted players CSV saved: 198528 rows
      • Actual data rows: 165440
      • Predicted data rows: 33088

📈 Calculating comprehensive metrics for all datasets...

📊 Performance Comparison Table:
   Dataset       MSE     RMSE      MAE       R²  Sample Size
     Train 66.278518 8.141162 6.073396 0.804403        10664
Validation 80.440566 8.968867 6.635464 0.760449         1399
      Test 82.441127 9.079710 6.763831 0.758274         1505

🎨 Creating analysis visualizations and reports...
   💾 Performance comparison saved to: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/training_artifacts/performance_comparison.csv
   ✅ Error analysis visualization saved to: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/visualizations/err

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from tqdm import tqdm
import os
from datetime import datetime

print("\n== STEP 6: GENERATING PREDICTIONS AND ANALYSIS REPORTS WITH TEMPORAL INTEGRITY ==")
start_time = time.time()

# 1. Generate predictions for test set
print("\n🔮 Generating predictions for test set...")
test_predictions = TRAINED_MODEL.predict(TRAINING_DATA['X_test'], verbose=1)
print(f"   ✅ Predictions generated: {test_predictions.shape}")

# 2. Create the five join keys for data merging (recreate if needed)
print("\n🔑 Recreating five join keys for data integrity...")
sequence_df['five_key'] = sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

ball_df['five_key'] = ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

players_df['five_key'] = players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

# 3. Get test sequences and create test files
print("\n📁 Creating test files with original structure...")

# 3.1 Get test sequence data - CORRECTED: Use only sequences that exist in test set
test_global_ids = TRAINING_DATA['test_global_ids'][0:len(test_predictions)]  # Match prediction count
test_sequence_data = sequence_df[sequence_df['global_sequence_id'].isin(test_global_ids)]
test_five_keys = test_sequence_data['five_key'].unique()

# 3.2 Ball features test data
test_ball_data = ball_df[ball_df['five_key'].isin(test_five_keys)]
ball_features_path = os.path.join(model_save_path, "predictions", "ball_features_test.csv")
os.makedirs(os.path.dirname(ball_features_path), exist_ok=True)
test_ball_data.to_csv(ball_features_path, index=False)
print(f"   ⚽ Ball features test data saved: {len(test_ball_data)} rows")

# 3.3 Possession features test data
test_possession_data = sequence_df[sequence_df['global_sequence_id'].isin(test_global_ids)]
possession_features_path = os.path.join(model_save_path, "predictions", "possession_features_test.csv")
os.makedirs(os.path.dirname(possession_features_path), exist_ok=True)
test_possession_data.to_csv(possession_features_path, index=False)
print(f"   📋 Possession features test data saved: {len(test_possession_data)} rows")

# 3.4 Players test data
test_players_data = players_df[players_df['five_key'].isin(test_five_keys)]
players_features_path = os.path.join(model_save_path, "predictions", "players_test.csv")
os.makedirs(os.path.dirname(players_features_path), exist_ok=True)
test_players_data.to_csv(players_features_path, index=False)
print(f"   👥 Players test data saved: {len(test_players_data)} rows")

# 4. Create predicted players CSV with complete structure - CORRECTED
print("\n🎯 Creating predicted players CSV with complete structure including sequence column...")

# Create list to store prediction rows
prediction_rows = []

# Create progress bar - CORRECTED: Use the actual number of predictions
progress = tqdm(total=len(test_predictions), desc="Building prediction CSV", position=0, leave=True)

for i, global_seq_id in enumerate(test_global_ids):  # Use filtered list
    # Get sequence data for this global sequence
    seq_data = sequence_df[sequence_df['global_sequence_id'] == global_seq_id].sort_values('timestep')

    if len(seq_data) != 5:  # Sequence of 5 has 5 timesteps
        progress.update(1)
        continue

    # Get predicted coordinates for timestep 5 - NOW SAFE (i < len(test_predictions))
    predicted_coords = test_predictions[i]

    # Process each timestep (1-4) for actual data
    for timestep in range(1, 5):  # Timesteps 1-4 for actual data
        timestep_row = seq_data[seq_data['timestep'] == timestep].iloc[0]
        key = (
            timestep_row['gameid'], timestep_row['possessioneventid'], timestep_row['eventtime'],
            timestep_row['sequence'], timestep_row['period']
        )

        # Get player data for this timestep - CORRECTED: Include all columns
        players_for_timestep = players_df[players_df['five_key'] == key]

        if len(players_for_timestep) < 22:
            continue

        # Add actual player positions (22 players per timestep) with ALL required columns
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row for this player's event
            matching_seq_row = sequence_df[
                (sequence_df['gameid'] == player_row['gameid']) &
                (sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (sequence_df['eventtime'] == player_row['eventtime']) &
                (sequence_df['sequence'] == player_row['sequence']) &
                (sequence_df['period'] == player_row['period'])
            ].iloc[0] if not sequence_df.empty else timestep_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],  # Using eventtime as starttime
                'endtime': matching_seq_row['eventtime'],    # Using eventtime as endtime
                'duration': 0.0,  # Default duration
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': timestep,
                'global_sequence_id': timestep_row['global_sequence_id']
            }
            prediction_rows.append(row_dict)

    # Add timestep 5 actual data
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    key = (
        timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
        timestep5_row['sequence'], timestep5_row['period']
    )

    players_for_timestep = players_df[players_df['five_key'] == key]

    if len(players_for_timestep) >= 22:
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row
            matching_seq_row = sequence_df[
                (sequence_df['gameid'] == player_row['gameid']) &
                (sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (sequence_df['eventtime'] == player_row['eventtime']) &
                (sequence_df['sequence'] == player_row['sequence']) &
                (sequence_df['period'] == player_row['period'])
            ].iloc[0] if not sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            prediction_rows.append(row_dict)

    # Add timestep 5 predicted data
    if len(players_for_timestep) >= 22:
        for j in range(22):
            player_row = players_for_timestep.iloc[j]
            # Get matching sequence row
            matching_seq_row = sequence_df[
                (sequence_df['gameid'] == player_row['gameid']) &
                (sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (sequence_df['eventtime'] == player_row['eventtime']) &
                (sequence_df['sequence'] == player_row['sequence']) &
                (sequence_df['period'] == player_row['period'])
            ].iloc[0] if not sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': predicted_coords[j*2],
                'y': predicted_coords[j*2 + 1],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 1,
                'data_type': 'predicted',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            prediction_rows.append(row_dict)

    progress.update(1)

progress.close()

# 5. Create and save prediction DataFrame with ALL required columns
print("\n💾 Saving predicted players CSV with complete column structure...")
prediction_df = pd.DataFrame(prediction_rows)

# Define EXACT column order as requested
required_columns = [
    'gameid', 'gameeventid', 'possessioneventid', 'starttime', 'endtime', 'duration', 'eventtime', 'sequence',
    'playerid', 'positiongrouptype', 'jerseynum', 'team', 'x', 'y', 'visibility', 'confidence',
    'possessioneventtype', 'teamattackingdirection', 'period', 'teamname',
    'is_predicted', 'data_type', 'sequence_id', 'timestep', 'global_sequence_id'
]

# Ensure all required columns exist with proper defaults
for col in required_columns:
    if col not in prediction_df.columns:
        if col in ['gameid', 'gameeventid', 'possessioneventid', 'playerid', 'jerseynum', 'period', 'sequence', 'sequence_id', 'timestep', 'global_sequence_id', 'is_predicted']:
            prediction_df[col] = 0
        elif col in ['x', 'y', 'starttime', 'endtime', 'duration']:
            prediction_df[col] = 0.0
        elif col in ['positiongrouptype', 'team', 'visibility', 'confidence', 'possessioneventtype', 'teamattackingdirection', 'teamname', 'data_type']:
            prediction_df[col] = 'Unknown'
        else:
            prediction_df[col] = 'missing'

# Reorder columns to EXACT required structure
prediction_df = prediction_df[required_columns]

predicted_players_path = os.path.join(model_save_path, "predictions", "predicted_players.csv")
os.makedirs(os.path.dirname(predicted_players_path), exist_ok=True)
prediction_df.to_csv(predicted_players_path, index=False)
print(f"   ✅ Predicted players CSV saved: {len(prediction_df)} rows")
print(f"      • Actual data rows: {len(prediction_df[prediction_df['data_type'] == 'actual'])}")
print(f"      • Predicted data rows: {len(prediction_df[prediction_df['data_type'] == 'predicted'])}")
print(f"      • Columns included: {', '.join(prediction_df.columns)}")

# 6-9. The rest of the code remains the same (metrics, visualizations, report generation)
# ... [same as previous code for metrics, visualizations, and report generation]

total_time = time.time() - start_time
print(f"\n✅ STEP 6 COMPLETE: Predictions and analysis reports generated")
print(f"   📊 Test set performance: MSE={test_metrics['mse']:.4f}, MAE={test_metrics['mae']:.4f}, R²={test_metrics['r2']:.4f}")
print(f"   💾 All artifacts saved to: {model_save_path}")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\n🎉 🎉 🎉 PROJECT COMPLETED SUCCESSFULLY! 🎉 🎉 🎉")
print(f"\n📥 FINAL MODEL AND ARTIFACTS SAVED TO:")
print(f"   {model_save_path}")
print("\n📊 KEY OUTPUT FILES:")
print(f"   • Model: {os.path.join(model_save_path, 'model_checkpoints', 'best_model_epoch_99_val_loss_80.440567.keras')}")
print(f"   • Ball Features Test: {ball_features_path}")
print(f"   • Possession Features Test: {possession_features_path}")
print(f"   • Players Test: {players_features_path}")
print(f"   • Predicted Players: {predicted_players_path} (with complete 25-column structure)")
print(f"   • Performance Comparison: {comparison_path}")
print(f"   • Error Analysis: {error_path}")
print(f"   • Pitch Visualization: {pitch_path}")
print(f"   • Analysis Report: {report_path}")


== STEP 6: GENERATING PREDICTIONS AND ANALYSIS REPORTS WITH TEMPORAL INTEGRITY ==

🔮 Generating predictions for test set...
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
   ✅ Predictions generated: (1505, 44)

🔑 Recreating five join keys for data integrity...

📁 Creating test files with original structure...
   ⚽ Ball features test data saved: 2485 rows
   📋 Possession features test data saved: 7525 rows
   👥 Players test data saved: 54665 rows

🎯 Creating predicted players CSV with complete structure including sequence column...


Building prediction CSV: 100%|██████████| 1505/1505 [19:11<00:00,  1.31it/s]



💾 Saving predicted players CSV with complete column structure...
   ✅ Predicted players CSV saved: 198528 rows
      • Actual data rows: 165440
      • Predicted data rows: 33088
      • Columns included: gameid, gameeventid, possessioneventid, starttime, endtime, duration, eventtime, sequence, playerid, positiongrouptype, jerseynum, team, x, y, visibility, confidence, possessioneventtype, teamattackingdirection, period, teamname, is_predicted, data_type, sequence_id, timestep, global_sequence_id

✅ STEP 6 COMPLETE: Predictions and analysis reports generated
   📊 Test set performance: MSE=82.4411, MAE=6.7638, R²=0.7583
   💾 All artifacts saved to: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files
   ⏱️  Total execution time: 1171.16 seconds

🎉 🎉 🎉 PROJECT COMPLETED SUCCESSFULLY! 🎉 🎉 🎉

📥 FINAL MODEL AND ARTIFACTS SAVED TO:
   /content/drive/MyDrive/Pass2Formation_Methodology/F

# **Baseline_Model_Test_on_France_Team_Only**

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive
import matplotlib.pyplot as plt
import json
import time
from datetime import datetime
import logging

print("== STEP 1: ENVIRONMENT SETUP & MODEL LOADING FOR FRANCE TEAM EVALUATION ==")

# Mount Google Drive
if not os.path.exists('/content/drive'):
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("Google Drive already mounted")

# Define dataset paths for France team
base_path_france = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/France"

# France data file paths
ball_france_path = os.path.join(base_path_france, "Ball_Features/Ball_Normalized_Filtered_France_Team_Only.csv")
players_france_path = os.path.join(base_path_france, "Players_Features/Normalized_Ordered_France_Team_Only.csv")
possession_france_path = os.path.join(base_path_france, "Possession_Features/France_Team_Only_Sequence_of_5_Possession_Features.csv")

# Output save path for France test results
output_base_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/France_Test"

print("\n📁 France Data File Paths:")
print(f"Ball features path: {ball_france_path}")
print(f"Players features path: {players_france_path}")
print(f"Possession features path: {possession_france_path}")
print(f"Output save path: {output_base_path}")

# Create output directory structure
os.makedirs(output_base_path, exist_ok=True)
os.makedirs(os.path.join(output_base_path, "model_validation"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "predictions"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "visualizations"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "analysis"), exist_ok=True)
print(f"\n✅ Output directory structure created at: {output_base_path}")

# Check GPU availability
print("\n🔍 GPU Availability Check:")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"  ✅ {len(gpus)} GPU(s) available for inference")
    for i, gpu in enumerate(gpus):
        print(f"     GPU {i}: {gpu}")

    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("  ✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(f"  ❌ Error setting memory growth: {e}")
else:
    print("  ❌ No GPU available, using CPU for inference")

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
print(f"\n🌱 Random seed set to {seed} for reproducibility")

# Load pre-trained model from previous task
print("\n🧠 Loading pre-trained model from baseline model...")
model_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/model_checkpoints/best_model_epoch_99_val_loss_80.440567.keras"

try:
    france_model = tf.keras.models.load_model(model_path)
    print(f"   ✅ Model loaded successfully from: {model_path}")

    # Verify model architecture
    print("\n✅ Model architecture verification:")
    print(f"   Input shape: {france_model.input_shape}")
    print(f"   Output shape: {france_model.output_shape}")
    print(f"   Total parameters: {france_model.count_params():,}")

    # Save model summary
    model_summary_path = os.path.join(output_base_path, "model_validation", "loaded_model_summary.txt")
    with open(model_summary_path, 'w') as f:
        france_model.summary(print_fn=lambda x: f.write(x + '\n'))
    print(f"   📝 Model summary saved to: {model_summary_path}")

except Exception as e:
    print(f"   ❌ Error loading model: {e}")
    raise

# Verify model can handle expected input shape
expected_input_shape = (None, 4, 62)  # batch_size, timesteps, features
if france_model.input_shape != expected_input_shape:
    print(f"   ⚠️  WARNING: Model input shape {france_model.input_shape} doesn't match expected {expected_input_shape}")
    print("   This may cause errors during inference with France data")

# Verify output shape
expected_output_shape = (None, 44)  # batch_size, player coordinates
if france_model.output_shape != expected_output_shape:
    print(f"   ⚠️  WARNING: Model output shape {france_model.output_shape} doesn't match expected {expected_output_shape}")

print("\n✅ STEP 1 COMPLETE: Environment setup and model loading finished")
print("Ready for next step: France data loading and validation")
print(f"\n📊 Next step will process France team data using identical logic to training task")
print("All spatial coordinates used as-is (no normalization applied)")
print("Missing players handled with (-500, -500) coordinates as in training")
print("Batch size for inference: 64 (same as training)")

== STEP 1: ENVIRONMENT SETUP & MODEL LOADING FOR FRANCE TEAM EVALUATION ==
Google Drive already mounted

📁 France Data File Paths:
Ball features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/France/Ball_Features/Ball_Normalized_Filtered_France_Team_Only.csv
Players features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/France/Players_Features/Normalized_Ordered_France_Team_Only.csv
Possession features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/France/Possession_Features/France_Team_Only_Sequence_of_5_Possession_Features.csv
Output save path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/France_Test

✅ Output directory structure created at: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model

   📝 Model summary saved to: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/France_Test/model_validation/loaded_model_summary.txt

✅ STEP 1 COMPLETE: Environment setup and model loading finished
Ready for next step: France data loading and validation

📊 Next step will process France team data using identical logic to training task
All spatial coordinates used as-is (no normalization applied)
Missing players handled with (-500, -500) coordinates as in training
Batch size for inference: 64 (same as training)


In [ ]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("\n== STEP 2: FRANCE DATA LOADING AND VALIDATION ==")
start_time = time.time()

# 1. Load France possession features dataset
print("\n📊 Loading France possession features dataset...")
france_sequence_df = pd.read_csv(
    possession_france_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'passerplayerid': 'float32',  # Use float32 to handle NaN values
        'receiverplayerid': 'float32',  # Use float32 to handle NaN values
        'passtype': 'int8',
        'passoutcometype': 'int8',
        'pressuretype': 'int8',
        'sequence_id': 'int32',
        'timestep': 'int8',
        'global_sequence_id': 'int32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'teamname', 'teamattackingdirection', 'passerplayerid', 'receiverplayerid',
             'passtype', 'passoutcometype', 'pressuretype', 'timestep', 'global_sequence_id', 'sequence_id']
)

print(f"   ✅ France possession features loaded: {len(france_sequence_df):,} rows, {france_sequence_df.shape[1]} columns")

# 2. Load France ball features dataset
print("\n⚽ Loading France ball features dataset...")
france_ball_df = pd.read_csv(
    ball_france_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'ball_x': 'float32',
        'ball_y': 'float32',
        'ball_z': 'float32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'ball_x', 'ball_y', 'ball_z']  # No sequence_id in this file
)

print(f"   ✅ France ball features loaded: {len(france_ball_df):,} rows, {france_ball_df.shape[1]} columns")

# 3. Load France players features dataset
print("\n👥 Loading France players features dataset...")
france_players_df = pd.read_csv(
    players_france_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'jerseynum': 'int8',
        'playerid': 'int32',
        'positiongrouptype': 'category',
        'x': 'float32',
        'y': 'float32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'jerseynum', 'team', 'visibility', 'confidence', 'x', 'y', 'playerid', 'positiongrouptype']  # No sequence_id in this file
)

print(f"   ✅ France players features loaded: {len(france_players_df):,} rows, {france_players_df.shape[1]} columns")

# 4. Data validation and basic statistics (identical to training logic)
print("\n🔍 Data validation and basic statistics:")

# Create the five join keys for all datasets
print("   🔑 Creating five join keys (gameid, possessioneventid, eventtime, sequence, period)...")
france_sequence_df['five_key'] = france_sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

france_ball_df['five_key'] = france_ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

france_players_df['five_key'] = france_players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

print("   ✅ Five join keys created successfully")

# Check for missing values in critical columns
print("\n   Missing values check:")
critical_columns = ['gameid', 'possessioneventid', 'eventtime', 'sequence', 'period', 'global_sequence_id']
for col in critical_columns:
    if col in france_sequence_df.columns:
        missing_count = france_sequence_df[col].isna().sum()
        print(f"     France Sequence {col}: {missing_count} missing values")

# CORRECTED: Calculate unique game-sequence combinations for France possessions
print("\n   🔍 Calculating unique France possessions using (gameid, sequence) composite key...")
france_sequence_df['game_sequence_key'] = france_sequence_df.apply(lambda row: (row['gameid'], row['sequence']), axis=1)
unique_france_game_sequences = france_sequence_df['game_sequence_key'].nunique()
unique_france_global_sequences = france_sequence_df['global_sequence_id'].nunique()
total_france_timesteps = len(france_sequence_df)

print(f"\n   📊 France dataset summary:")
print(f"     Unique global sequences: {unique_france_global_sequences:,} (globally unique 5-timestep sequences)")
print(f"     Unique game-sequence combinations: {unique_france_game_sequences:,} (unique France possessions)")
print(f"     Total timesteps: {total_france_timesteps:,}")
print(f"     Average timesteps per global sequence: {total_france_timesteps/unique_france_global_sequences:.1f}")
print(f"     Average timesteps per possession: {total_france_timesteps/unique_france_game_sequences:.1f}")

# Check global_sequence_id distribution
france_global_seq_counts = france_sequence_df['global_sequence_id'].value_counts()
min_timesteps = france_global_seq_counts.min()
max_timesteps = france_global_seq_counts.max()
avg_timesteps = france_global_seq_counts.mean()

print(f"\n   🔢 France global sequence distribution:")
print(f"     Min timesteps per global sequence: {min_timesteps}")
print(f"     Max timesteps per global sequence: {max_timesteps}")
print(f"     Avg timesteps per global sequence: {avg_timesteps:.1f}")

# Check for the expected 5 timesteps per global sequence
france_expected_sequences = france_global_seq_counts[france_global_seq_counts == 5].shape[0]
france_unexpected_sequences = france_global_seq_counts[france_global_seq_counts != 5].shape[0]

print(f"\n   ⚠️ France global sequence validation (expecting 5 timesteps per sequence):")
print(f"     Sequences with exactly 5 timesteps: {france_expected_sequences:,} ({france_expected_sequences/unique_france_global_sequences*100:.1f}%)")
print(f"     Sequences with unexpected timestep count: {france_unexpected_sequences:,} ({france_unexpected_sequences/unique_france_global_sequences*100:.1f}%)")

if france_unexpected_sequences > 0:
    print("     🚨 WARNING: Some France global sequences don't have exactly 5 timesteps!")
    print("            This may require filtering before inference.")

# Store France datasets for next steps
FRANCE_DATA = {
    'sequence_df': france_sequence_df,
    'ball_df': france_ball_df,
    'players_df': france_players_df
}

total_time = time.time() - start_time
print(f"\n✅ STEP 2 COMPLETE: France data loading and validation finished")
print(f"   ✅ All France datasets loaded successfully")
print(f"   ✅ Basic validation completed with CORRECTED sequence counting")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Feature engineering and sequence construction for France data")
print("Note: All spatial coordinates used as-is (no normalization applied)")
print("✅ Using identical logic to training task for feature extraction")


== STEP 2: FRANCE DATA LOADING AND VALIDATION ==

📊 Loading France possession features dataset...
   ✅ France possession features loaded: 8,085 rows, 16 columns

⚽ Loading France ball features dataset...
   ✅ France ball features loaded: 3,559 rows, 9 columns

👥 Loading France players features dataset...
   ✅ France players features loaded: 83,908 rows, 14 columns

🔍 Data validation and basic statistics:
   🔑 Creating five join keys (gameid, possessioneventid, eventtime, sequence, period)...
   ✅ Five join keys created successfully

   Missing values check:
     France Sequence gameid: 0 missing values
     France Sequence possessioneventid: 0 missing values
     France Sequence eventtime: 0 missing values
     France Sequence sequence: 0 missing values
     France Sequence period: 0 missing values
     France Sequence global_sequence_id: 0 missing values

   🔍 Calculating unique France possessions using (gameid, sequence) composite key...

   📊 France dataset summary:
     Unique glo

In [ ]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("\n== STEP 3: FEATURE ENGINEERING AND SEQUENCE CONSTRUCTION FOR FRANCE DATA ==")
start_time = time.time()

# 1. Create lookup dictionaries for faster joins (identical to training logic)
print("\n🔧 Creating lookup dictionaries for faster data joining...")
start_sub = time.time()

# Create ball lookup dictionary: five_key -> ball features
france_ball_lookup = FRANCE_DATA['ball_df'].set_index('five_key')[['ball_x', 'ball_y', 'ball_z']].to_dict('index')

# Create players lookup dictionary: five_key -> player positions
france_players_grouped = {}
for key, group in FRANCE_DATA['players_df'].groupby('five_key'):
    france_players_grouped[key] = group[['x', 'y', 'playerid', 'positiongrouptype', 'jerseynum', 'team']].to_dict('records')

# Create next timestep lookup for temporal context
# First, sort by global_sequence_id and timestep
france_sequence_df_sorted = FRANCE_DATA['sequence_df'].sort_values(['global_sequence_id', 'timestep'])
# Create shifted columns for next timestep within the same global sequence
france_sequence_df_sorted['next_timestep'] = france_sequence_df_sorted.groupby('global_sequence_id')['timestep'].shift(-1)
france_sequence_df_sorted['next_eventtime'] = france_sequence_df_sorted.groupby('global_sequence_id')['eventtime'].shift(-1)

# Create lookup for next timestep context
france_next_timestep_lookup = {}
for idx, row in france_sequence_df_sorted.iterrows():
    if not pd.isna(row['next_timestep']) and row['next_timestep'] == row['timestep'] + 1:
        current_key = (
            row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
        )
        next_key = (
            row['gameid'], row['possessioneventid'], row['next_eventtime'], row['sequence'], row['period']
        )
        france_next_timestep_lookup[current_key] = {
            'next_ball_key': next_key,
            'next_passerplayerid': row['passerplayerid'] if not pd.isna(row['passerplayerid']) else -1,
            'next_receiverplayerid': row['receiverplayerid'] if not pd.isna(row['receiverplayerid']) else -1
        }

sub_time = time.time() - start_sub
print(f"   ✅ Lookup dictionaries built in {sub_time:.2f} seconds")

# 2. Get unique global sequences for France data (already validated to have exactly 5 timesteps)
print("\n📊 Getting unique France global sequences...")
unique_france_global_sequences = FRANCE_DATA['sequence_df']['global_sequence_id'].unique()
print(f"   📂 Total unique France global sequences: {len(unique_france_global_sequences):,}")

# 3. Feature engineering with validation - CORRECTED: Hard check sequence count matching
print("\n⚙️ Engineering features for France sequence of 5...")
start_sub = time.time()

# Initialize storage for France sequences
X_france_sequences = []  # Input sequences (4 timesteps × 62 features)
y_france_sequences = []  # Target sequences (44 player coordinates for timestep 5)
valid_france_global_sequences = []  # Store valid global sequence IDs

# Create progress bar for sequence processing
seq_progress = tqdm(total=len(unique_france_global_sequences), desc="Building France sequences", position=0, leave=True)

# Track global sequences that will be processed
processed_global_sequences = []

for global_seq_id in unique_france_global_sequences:
    # Get all timesteps for this global sequence
    seq_data = FRANCE_DATA['sequence_df'][FRANCE_DATA['sequence_df']['global_sequence_id'] == global_seq_id].sort_values('timestep')

    # Validate we have exactly 5 timesteps
    if len(seq_data) != 5:
        seq_progress.update(1)
        continue

    # Prepare input features (timesteps 1-4) and target (timestep 5)
    input_features = []
    has_missing_data = False

    # Process timesteps 1-4 for input
    for timestep in range(1, 5):  # Timesteps 1-4 for input
        row = seq_data[seq_data['timestep'] == timestep].iloc[0]

        # Create the five-key tuple for joining
        key = (row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period'])

        # Get ball features with fallback
        ball_features = france_ball_lookup.get(key, {'ball_x': 0.0, 'ball_y': 0.0, 'ball_z': 0.0})

        # Get player positions (44 features) with fallback
        player_positions = france_players_grouped.get(key, [])
        if len(player_positions) < 22:
            # Handle missing players by using (-500, -500) as default coordinates
            player_coords = np.zeros(44)
            for i in range(22):
                player_coords[i*2] = -500.0
                player_coords[i*2 + 1] = -500.0
            has_missing_data = True
        else:
            # Extract x,y coordinates for all 22 players in order
            player_coords = np.zeros(44)
            for i, player in enumerate(player_positions[:22]):  # Take first 22 players
                player_coords[i*2] = player['x']
                player_coords[i*2 + 1] = player['y']

        # Get event features (8 features)
        passer_id = row['passerplayerid'] if not pd.isna(row['passerplayerid']) else -1
        receiver_id = row['receiverplayerid'] if not pd.isna(row['receiverplayerid']) else -1

        # Get passer and receiver coordinates with fallback
        passer_coords = (-500.0, -500.0)  # Default for missing
        receiver_coords = (-500.0, -500.0)  # Default for missing

        if len(player_positions) >= 22:
            # Find passer and receiver in the player positions
            for player in player_positions:
                if player['playerid'] == passer_id:
                    passer_coords = (player['x'], player['y'])
                if player['playerid'] == receiver_id:
                    receiver_coords = (player['x'], player['y'])

        event_features = [
            row['passtype'] if not pd.isna(row['passtype']) else 0,
            row['passoutcometype'] if not pd.isna(row['passoutcometype']) else 0,
            row['pressuretype'] if not pd.isna(row['pressuretype']) else 0,
            row['period'],
            passer_coords[0], passer_coords[1],
            receiver_coords[0], receiver_coords[1]
        ]

        # Get next timestep context (7 features) for the next timestep in the sequence
        next_context = [0.0, 0.0, 0.0, -500.0, -500.0, -500.0, -500.0]  # Default values

        if key in france_next_timestep_lookup:
            next_info = france_next_timestep_lookup[key]
            next_ball_key = next_info['next_ball_key']
            next_ball = france_ball_lookup.get(next_ball_key, {'ball_x': 0.0, 'ball_y': 0.0, 'ball_z': 0.0})

            # Get next passer/receiver coordinates
            next_passer_coords = (-500.0, -500.0)
            next_receiver_coords = (-500.0, -500.0)

            if next_ball_key in france_players_grouped and len(france_players_grouped[next_ball_key]) >= 22:
                next_players = france_players_grouped[next_ball_key]
                for player in next_players:
                    if player['playerid'] == next_info['next_passerplayerid']:
                        next_passer_coords = (player['x'], player['y'])
                    if player['playerid'] == next_info['next_receiverplayerid']:
                        next_receiver_coords = (player['x'], player['y'])

            next_context = [
                next_ball['ball_x'], next_ball['ball_y'], next_ball['ball_z'],
                next_passer_coords[0], next_passer_coords[1],
                next_receiver_coords[0], next_receiver_coords[1]
            ]

        # Combine all features (44 + 8 + 3 + 7 = 62 features)
        timestep_features = np.concatenate([
            player_coords,
            event_features,
            [ball_features['ball_x'], ball_features['ball_y'], ball_features['ball_z']],
            next_context
        ])

        input_features.append(timestep_features)

    # Get target (timestep 5 player positions)
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    target_key = (timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
                 timestep5_row['sequence'], timestep5_row['period'])

    target_players = france_players_grouped.get(target_key, [])
    if len(target_players) >= 22 and not has_missing_data:
        target_coords = np.zeros(44)
        for i, player in enumerate(target_players[:22]):
            target_coords[i*2] = player['x']
            target_coords[i*2 + 1] = player['y']

        X_france_sequences.append(np.array(input_features))  # Shape: (4, 62)
        y_france_sequences.append(target_coords)  # Shape: (44,)
        valid_france_global_sequences.append(global_seq_id)
        processed_global_sequences.append(global_seq_id)

    seq_progress.update(1)

seq_progress.close()
sub_time = time.time() - start_sub
print(f"   ✅ Features engineered for {len(X_france_sequences):,}/{len(unique_france_global_sequences):,} France sequences ({len(X_france_sequences)/len(unique_france_global_sequences)*100:.1f}%)")
print(f"   ⏱️  Feature engineering time: {sub_time:.2f} seconds")

# 4. Convert to numpy arrays and validate shapes - CORRECTED: Hard validation
print("\n📊 Converting to numpy arrays and validating shapes...")
X_france = np.array(X_france_sequences)  # Shape: (num_sequences, 4, 62)
y_france = np.array(y_france_sequences)  # Shape: (num_sequences, 44)

print(f"\n✅ Final France dataset shapes:")
print(f"   Input (X_france): {X_france.shape} - (sequences, timesteps, features)")
print(f"   Target (y_france): {y_france.shape} - (sequences, player_coordinates)")
print(f"   Features per timestep: {X_france.shape[2]} (should be 62)")
print(f"   Player coordinates: {y_france.shape[1]} (should be 44)")

# HARD VALIDATION: Ensure we processed the expected number of sequences
expected_sequences = 1617  # From Step 2 validation
actual_sequences = len(X_france_sequences)
print(f"\n🔍 HARD SEQUENCE VALIDATION:")
print(f"   Expected global sequences: {expected_sequences:,}")
print(f"   Actually processed: {actual_sequences:,}")
print(f"   Processing rate: {actual_sequences/expected_sequences*100:.1f}%")

if actual_sequences < expected_sequences * 0.95:  # Less than 95% processed
    print("   ⚠️  WARNING: Significant sequence loss during feature engineering!")
    print(f"   Lost {expected_sequences - actual_sequences:,} sequences")
    print("   Check for missing player data or other filtering issues")

# Validate feature count
assert X_france.shape[2] == 62, f"Expected 62 features per timestep, got {X_france.shape[2]}"
assert y_france.shape[1] == 44, f"Expected 44 target coordinates, got {y_france.shape[1]}"

# Store for next steps
FRANCE_SEQUENCE_DATA = {
    'X': X_france,
    'y': y_france,
    'valid_global_sequences': valid_france_global_sequences,
    'sequence_df': FRANCE_DATA['sequence_df'],
    'processed_global_sequences': processed_global_sequences
}

total_time = time.time() - start_time
print(f"\n✅ STEP 3 COMPLETE: France feature engineering and sequence construction finished")
print(f"   ✅ Successfully processed {len(X_france_sequences):,} valid France sequences")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Model inference and prediction generation for France data")
print("Note: Using batch size 64 for inference (same as training)")
print("✅ Applied identical feature engineering logic as training task")
print("✅ Hard validation ensures sequence count consistency")


== STEP 3: FEATURE ENGINEERING AND SEQUENCE CONSTRUCTION FOR FRANCE DATA ==

🔧 Creating lookup dictionaries for faster data joining...
   ✅ Lookup dictionaries built in 7.45 seconds

📊 Getting unique France global sequences...
   📂 Total unique France global sequences: 1,617

⚙️ Engineering features for France sequence of 5...


Building France sequences: 100%|██████████| 1617/1617 [00:04<00:00, 337.00it/s]

   ✅ Features engineered for 1,617/1,617 France sequences (100.0%)
   ⏱️  Feature engineering time: 4.80 seconds

📊 Converting to numpy arrays and validating shapes...

✅ Final France dataset shapes:
   Input (X_france): (1617, 4, 62) - (sequences, timesteps, features)
   Target (y_france): (1617, 44) - (sequences, player_coordinates)
   Features per timestep: 62 (should be 62)
   Player coordinates: 44 (should be 44)

🔍 HARD SEQUENCE VALIDATION:
   Expected global sequences: 1,617
   Actually processed: 1,617
   Processing rate: 100.0%

✅ STEP 3 COMPLETE: France feature engineering and sequence construction finished
   ✅ Successfully processed 1,617 valid France sequences
   ⏱️  Total execution time: 12.26 seconds

Next step: Model inference and prediction generation for France data
Note: Using batch size 64 for inference (same as training)
✅ Applied identical feature engineering logic as training task
✅ Hard validation ensures sequence count consistency


In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from tqdm import tqdm
import os
from datetime import datetime

print("\n== STEP 4: MODEL INFERENCE AND PREDICTION GENERATION FOR FRANCE DATA ==")
start_time = time.time()

# 1. Generate predictions for France data using the pre-trained model
print("\n🔮 Generating predictions for France data...")
print(f"   Model input shape: {france_model.input_shape}")
print(f"   France data shape: {FRANCE_SEQUENCE_DATA['X'].shape}")
print(f"   Batch size for inference: 64 (same as training)")

france_predictions = france_model.predict(
    FRANCE_SEQUENCE_DATA['X'],
    batch_size=64,  # Same batch size as training
    verbose=1
)

print(f"   ✅ Predictions generated: {france_predictions.shape}")

# 2. Create the five join keys for data merging (recreate if needed)
print("\n🔑 Recreating five join keys for France data integrity...")
france_sequence_df = FRANCE_DATA['sequence_df']
france_ball_df = FRANCE_DATA['ball_df']
france_players_df = FRANCE_DATA['players_df']

france_sequence_df['five_key'] = france_sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

france_ball_df['five_key'] = france_ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

france_players_df['five_key'] = france_players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

print("   ✅ Five join keys recreated successfully")

# 3. Get France test sequences and create test files
print("\n📁 Creating France test files with original structure...")

# 3.1 Get processed sequence data
france_test_global_ids = FRANCE_SEQUENCE_DATA['processed_global_sequences']
france_test_sequence_data = france_sequence_df[france_sequence_df['global_sequence_id'].isin(france_test_global_ids)]
france_test_five_keys = france_test_sequence_data['five_key'].unique()

# 3.2 Ball features test data
france_test_ball_data = france_ball_df[france_ball_df['five_key'].isin(france_test_five_keys)]
ball_france_path = os.path.join(output_base_path, "predictions", "ball_features_france_test.csv")
os.makedirs(os.path.dirname(ball_france_path), exist_ok=True)
france_test_ball_data.to_csv(ball_france_path, index=False)
print(f"   ⚽ Ball features France test data saved: {len(france_test_ball_data)} rows")

# 3.3 Possession features test data
france_test_possession_data = france_sequence_df[france_sequence_df['global_sequence_id'].isin(france_test_global_ids)]
possession_france_path = os.path.join(output_base_path, "predictions", "possession_features_france_test.csv")
os.makedirs(os.path.dirname(possession_france_path), exist_ok=True)
france_test_possession_data.to_csv(possession_france_path, index=False)
print(f"   📋 Possession features France test data saved: {len(france_test_possession_data)} rows")

# 3.4 Players test data
france_test_players_data = france_players_df[france_players_df['five_key'].isin(france_test_five_keys)]
players_france_path = os.path.join(output_base_path, "predictions", "players_france_test.csv")
os.makedirs(os.path.dirname(players_france_path), exist_ok=True)
france_test_players_data.to_csv(players_france_path, index=False)
print(f"   👥 Players France test data saved: {len(france_test_players_data)} rows")

# 4. Create predicted players CSV with complete structure
print("\n🎯 Creating predicted players CSV with complete structure including sequence column...")

# Create list to store prediction rows
france_prediction_rows = []

# Create progress bar
progress = tqdm(total=len(france_test_global_ids), desc="Building France prediction CSV", position=0, leave=True)

for i, global_seq_id in enumerate(france_test_global_ids):
    # Get sequence data for this global sequence
    seq_data = france_sequence_df[france_sequence_df['global_sequence_id'] == global_seq_id].sort_values('timestep')

    if len(seq_data) != 5:  # Sequence of 5 has 5 timesteps
        progress.update(1)
        continue

    # Get predicted coordinates for timestep 5
    predicted_coords = france_predictions[i]

    # Process each timestep (1-4) for actual data
    for timestep in range(1, 5):  # Timesteps 1-4 for actual data
        timestep_row = seq_data[seq_data['timestep'] == timestep].iloc[0]
        key = (
            timestep_row['gameid'], timestep_row['possessioneventid'], timestep_row['eventtime'],
            timestep_row['sequence'], timestep_row['period']
        )

        # Get player data for this timestep
        players_for_timestep = france_players_df[france_players_df['five_key'] == key]

        if len(players_for_timestep) < 22:
            continue

        # Add actual player positions (22 players per timestep) with ALL required columns
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row for this player's event
            matching_seq_row = france_sequence_df[
                (france_sequence_df['gameid'] == player_row['gameid']) &
                (france_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (france_sequence_df['eventtime'] == player_row['eventtime']) &
                (france_sequence_df['sequence'] == player_row['sequence']) &
                (france_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not france_sequence_df.empty else timestep_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],  # Using eventtime as starttime
                'endtime': matching_seq_row['eventtime'],    # Using eventtime as endtime
                'duration': 0.0,  # Default duration
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': timestep,
                'global_sequence_id': timestep_row['global_sequence_id']
            }
            france_prediction_rows.append(row_dict)

    # Add timestep 5 actual data
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    key = (
        timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
        timestep5_row['sequence'], timestep5_row['period']
    )

    players_for_timestep = france_players_df[france_players_df['five_key'] == key]

    if len(players_for_timestep) >= 22:
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row
            matching_seq_row = france_sequence_df[
                (france_sequence_df['gameid'] == player_row['gameid']) &
                (france_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (france_sequence_df['eventtime'] == player_row['eventtime']) &
                (france_sequence_df['sequence'] == player_row['sequence']) &
                (france_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not france_sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            france_prediction_rows.append(row_dict)

    # Add timestep 5 predicted data
    if len(players_for_timestep) >= 22:
        for j in range(22):
            player_row = players_for_timestep.iloc[j]
            # Get matching sequence row
            matching_seq_row = france_sequence_df[
                (france_sequence_df['gameid'] == player_row['gameid']) &
                (france_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (france_sequence_df['eventtime'] == player_row['eventtime']) &
                (france_sequence_df['sequence'] == player_row['sequence']) &
                (france_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not france_sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': predicted_coords[j*2],
                'y': predicted_coords[j*2 + 1],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 1,
                'data_type': 'predicted',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            france_prediction_rows.append(row_dict)

    progress.update(1)

progress.close()

# 5. Create and save prediction DataFrame with ALL required columns
print("\n💾 Saving predicted players CSV with complete column structure...")
france_prediction_df = pd.DataFrame(france_prediction_rows)

# Define EXACT column order as requested
required_columns = [
    'gameid', 'gameeventid', 'possessioneventid', 'starttime', 'endtime', 'duration', 'eventtime', 'sequence',
    'playerid', 'positiongrouptype', 'jerseynum', 'team', 'x', 'y', 'visibility', 'confidence',
    'possessioneventtype', 'teamattackingdirection', 'period', 'teamname',
    'is_predicted', 'data_type', 'sequence_id', 'timestep', 'global_sequence_id'
]

# Ensure all required columns exist with proper defaults
for col in required_columns:
    if col not in france_prediction_df.columns:
        if col in ['gameid', 'gameeventid', 'possessioneventid', 'playerid', 'jerseynum', 'period', 'sequence', 'sequence_id', 'timestep', 'global_sequence_id', 'is_predicted']:
            france_prediction_df[col] = 0
        elif col in ['x', 'y', 'starttime', 'endtime', 'duration']:
            france_prediction_df[col] = 0.0
        elif col in ['positiongrouptype', 'team', 'visibility', 'confidence', 'possessioneventtype', 'teamattackingdirection', 'teamname', 'data_type']:
            france_prediction_df[col] = 'Unknown'
        else:
            france_prediction_df[col] = 'missing'

# Reorder columns to EXACT required structure
france_prediction_df = france_prediction_df[required_columns]

predicted_players_france_path = os.path.join(output_base_path, "predictions", "predicted_players_france.csv")
os.makedirs(os.path.dirname(predicted_players_france_path), exist_ok=True)
france_prediction_df.to_csv(predicted_players_france_path, index=False)
print(f"   ✅ Predicted players France CSV saved: {len(france_prediction_df)} rows")
print(f"      • Actual data rows: {len(france_prediction_df[france_prediction_df['data_type'] == 'actual'])}")
print(f"      • Predicted data rows: {len(france_prediction_df[france_prediction_df['data_type'] == 'predicted'])}")
print(f"      • Columns included: {', '.join(france_prediction_df.columns)}")

# 6. Calculate comprehensive metrics for France data
print("\n📈 Calculating comprehensive metrics for France data...")

def calculate_metrics(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))
    r2 = r2_score(y_true.flatten(), y_pred.flatten())
    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

france_metrics = calculate_metrics(FRANCE_SEQUENCE_DATA['y'], france_predictions)

print("\n📊 France Performance Metrics:")
print(f"   Test MSE: {france_metrics['mse']:.6f}")
print(f"   Test MAE: {france_metrics['mae']:.6f}")
print(f"   Test RMSE: {france_metrics['rmse']:.6f}")
print(f"   Test R²: {france_metrics['r2']:.6f}")

# 7. Save metrics and create visualizations
print("\n🎨 Creating analysis visualizations and reports...")

# Save performance metrics
metrics_path = os.path.join(output_base_path, "model_validation", "performance_metrics.json")
os.makedirs(os.path.dirname(metrics_path), exist_ok=True)
with open(metrics_path, 'w') as f:
    json.dump(france_metrics, f, indent=2)
print(f"   💾 Performance metrics saved to: {metrics_path}")

# Create error analysis visualization
plt.figure(figsize=(15, 6))

# Calculate errors for France data
errors = np.abs(FRANCE_SEQUENCE_DATA['y'] - france_predictions)
player_errors = errors.reshape(-1, 22, 2)  # (samples, players, coordinates)
avg_player_errors = np.mean(player_errors, axis=(0, 2))  # Average error per player

plt.subplot(1, 2, 1)
plt.bar(range(1, 23), avg_player_errors, color='skyblue')
plt.title('Average Error per Player Position (France Data)')
plt.xlabel('Player Position (1-22)')
plt.ylabel('MAE')
plt.xticks(range(1, 23), [f'P{i}' for i in range(1, 23)], rotation=45)

plt.subplot(1, 2, 2)
all_errors = errors.flatten()
plt.hist(all_errors, bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
plt.axvline(np.mean(all_errors), color='red', linestyle='dashed', linewidth=2, label=f'Mean: {np.mean(all_errors):.2f}')
plt.title('Error Distribution (France Data)')
plt.xlabel('Absolute Error')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
error_path = os.path.join(output_base_path, "visualizations", "france_error_analysis.png")
os.makedirs(os.path.dirname(error_path), exist_ok=True)
plt.savefig(error_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Error analysis visualization saved to: {error_path}")
plt.close()

# 8. Generate pitch visualization with actual vs predicted for France data
plt.figure(figsize=(20, 8))

# Select a few representative France sequences to visualize
num_examples = min(4, len(france_test_global_ids))
example_indices = np.random.choice(len(france_test_global_ids), num_examples, replace=False)

for idx, example_idx in enumerate(example_indices):
    global_seq_id = france_test_global_ids[example_idx]
    actual_coords = FRANCE_SEQUENCE_DATA['y'][example_idx]
    pred_coords = france_predictions[example_idx]

    ax = plt.subplot(1, num_examples, idx+1)

    # Create pitch
    ax.set_xlim(-55, 55)
    ax.set_ylim(-35, 35)
    ax.set_aspect('equal')
    ax.set_title(f'France Sequence {global_seq_id}', fontsize=10)

    # Draw pitch markings
    ax.plot([-52.5, 52.5], [-34, -34], 'k-')  # Bottom
    ax.plot([-52.5, 52.5], [34, 34], 'k-')    # Top
    ax.plot([-52.5, -52.5], [-34, 34], 'k-')  # Left
    ax.plot([52.5, 52.5], [-34, 34], 'k-')    # Right
    ax.plot([0, 0], [-34, 34], 'k--')        # Center line

    # Plot actual positions (blue)
    actual_x = actual_coords[::2]
    actual_y = actual_coords[1::2]
    ax.scatter(actual_x[:11], actual_y[:11], c='blue', s=50, alpha=0.7, label='Actual Home')
    ax.scatter(actual_x[11:], actual_y[11:], c='red', s=50, alpha=0.7, label='Actual Away')

    # Plot predicted positions (green)
    pred_x = pred_coords[::2]
    pred_y = pred_coords[1::2]
    ax.scatter(pred_x[:11], pred_y[:11], c='lightgreen', s=50, marker='x', label='Predicted Home')
    ax.scatter(pred_x[11:], pred_y[11:], c='pink', s=50, marker='x', label='Predicted Away')

    # Draw error vectors
    for j in range(22):
        dx = pred_x[j] - actual_x[j]
        dy = pred_y[j] - actual_y[j]
        ax.arrow(actual_x[j], actual_y[j], dx, dy, color='black', alpha=0.5, width=0.1)

    # Turn off axis ticks and labels for cleaner look
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
pitch_path = os.path.join(output_base_path, "visualizations", "france_actual_vs_predicted_formations.png")
os.makedirs(os.path.dirname(pitch_path), exist_ok=True)
plt.savefig(pitch_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Pitch visualization saved to: {pitch_path}")
plt.close()

# 9. Generate comprehensive analysis report for France data
print("\n📝 Generating comprehensive analysis report for France data...")

report_path = os.path.join(output_base_path, "analysis", f"france_team_analysis_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
os.makedirs(os.path.dirname(report_path), exist_ok=True)

with open(report_path, 'w') as f:
    f.write("="*80 + "\n")
    f.write("FRANCE TEAM FORMATION PREDICTION - COMPREHENSIVE ANALYSIS REPORT\n")
    f.write("="*80 + "\n\n")

    f.write(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

    f.write("MODEL INFORMATION:\n")
    f.write("-"*50 + "\n")
    f.write(f"Pre-trained model loaded from: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/model_checkpoints/best_model_epoch_99_val_loss_80.440567.keras\n")
    f.write(f"Model input shape: (None, 4, 62)\n")
    f.write(f"Model output shape: (None, 44)\n")
    f.write(f"Total parameters: 167,404\n")
    f.write(f"Batch size used for inference: 64\n\n")

    f.write("FRANCE DATA SUMMARY:\n")
    f.write("-"*50 + "\n")
    f.write(f"Total unique global sequences: 1,617\n")
    f.write(f"Total timesteps processed: 8,085\n")
    f.write(f"Average timesteps per possession: 32.7\n")
    f.write(f"Sequences successfully processed: 1,617 (100.0%)\n\n")

    f.write("PERFORMANCE METRICS:\n")
    f.write("-"*50 + "\n")
    f.write(f"Test MSE: {france_metrics['mse']:.4f}\n")
    f.write(f"Test MAE: {france_metrics['mae']:.4f}\n")
    f.write(f"Test RMSE: {france_metrics['rmse']:.4f}\n")
    f.write(f"Test R²: {france_metrics['r2']:.4f}\n")
    f.write(f"Average positioning error: {france_metrics['mae']:.2f} coordinate units\n")
    f.write(f"Spatial interpretation: ~{(france_metrics['mae'] * 0.68):.2f} meter average error per player on 105m pitch\n\n")

    f.write("EXPORTED FILES:\n")
    f.write("-"*50 + "\n")
    f.write(f"1. Ball Features France Test: {ball_france_path}\n")
    f.write(f"   - Rows: {len(france_test_ball_data)}\n")
    f.write(f"   - Columns: {', '.join(france_test_ball_data.columns)}\n\n")
    f.write(f"2. Possession Features France Test: {possession_france_path}\n")
    f.write(f"   - Rows: {len(france_test_possession_data)}\n")
    f.write(f"   - Columns: {', '.join(france_test_possession_data.columns)}\n\n")
    f.write(f"3. Players France Test: {players_france_path}\n")
    f.write(f"   - Rows: {len(france_test_players_data)}\n")
    f.write(f"   - Columns: {', '.join(france_test_players_data.columns)}\n\n")
    f.write(f"4. Predicted Players France: {predicted_players_france_path}\n")
    f.write(f"   - Rows: {len(france_prediction_df)}\n")
    f.write(f"   - Columns: {', '.join(france_prediction_df.columns)} (25 columns including 'sequence')\n")
    f.write(f"   - Structure: {len(france_prediction_df[france_prediction_df['data_type'] == 'actual'])} actual rows + {len(france_prediction_df[france_prediction_df['data_type'] == 'predicted'])} predicted rows\n\n")

    f.write("DATA INTEGRITY GUARANTEES:\n")
    f.write("-"*50 + "\n")
    f.write("• France data processed using identical logic to training task\n")
    f.write("• Same five-key join system: (gameid, possessioneventid, eventtime, sequence, period)\n")
    f.write("• Missing players handled with (-500, -500) coordinates\n")
    f.write("• No spatial normalization applied - coordinates used as-is\n")
    f.write("• Temporal context features extracted identically to training\n")
    f.write("• Batch size 64 used for inference (same as training)\n")

print(f"   ✅ Analysis report saved to: {report_path}")

total_time = time.time() - start_time
print(f"\n✅ STEP 4 COMPLETE: Model inference and prediction generation finished")
print(f"   📊 France test performance: MSE={france_metrics['mse']:.4f}, MAE={france_metrics['mae']:.4f}, R²={france_metrics['r2']:.4f}")
print(f"   💾 All France artifacts saved to: {output_base_path}")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\n🎉 🎉 🎉 FRANCE TEAM EVALUATION COMPLETED SUCCESSFULLY! 🎉 🎉 🎉")
print(f"\n📥 FINAL FRANCE EVALUATION ARTIFACTS SAVED TO:")
print(f"   {output_base_path}")
print("\n📊 KEY OUTPUT FILES:")
print(f"   • Predicted Players France: {predicted_players_france_path} (25 columns including 'sequence')")
print(f"   • Performance Metrics: {metrics_path}")
print(f"   • Error Analysis: {error_path}")
print(f"   • Pitch Visualization: {pitch_path}")
print(f"   • Analysis Report: {report_path}")


== STEP 4: MODEL INFERENCE AND PREDICTION GENERATION FOR FRANCE DATA ==

🔮 Generating predictions for France data...
   Model input shape: (None, 4, 62)
   France data shape: (1617, 4, 62)
   Batch size for inference: 64 (same as training)
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
   ✅ Predictions generated: (1617, 44)

🔑 Recreating five join keys for France data integrity...
   ✅ Five join keys recreated successfully

📁 Creating France test files with original structure...
   ⚽ Ball features France test data saved: 2605 rows
   📋 Possession features France test data saved: 8085 rows
   👥 Players France test data saved: 57310 rows

🎯 Creating predicted players CSV with complete structure including sequence column...


Building France prediction CSV: 100%|██████████| 1617/1617 [05:49<00:00,  4.62it/s]



💾 Saving predicted players CSV with complete column structure...
   ✅ Predicted players France CSV saved: 213444 rows
      • Actual data rows: 177870
      • Predicted data rows: 35574
      • Columns included: gameid, gameeventid, possessioneventid, starttime, endtime, duration, eventtime, sequence, playerid, positiongrouptype, jerseynum, team, x, y, visibility, confidence, possessioneventtype, teamattackingdirection, period, teamname, is_predicted, data_type, sequence_id, timestep, global_sequence_id

📈 Calculating comprehensive metrics for France data...

📊 France Performance Metrics:
   Test MSE: 136.482101
   Test MAE: 9.122630
   Test RMSE: 11.682555
   Test R²: 0.564042

🎨 Creating analysis visualizations and reports...
   💾 Performance metrics saved to: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/France_Test/model_validation/performance_metrics.json
   ✅ Error an

In [ ]:
# Load baseline model metrics from training task
print("\n📊 Loading baseline model performance metrics...")
baseline_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/training_artifacts/evaluation_metrics.json"

try:
    with open(baseline_metrics_path, 'r') as f:
        baseline_metrics = json.load(f)
    print(f"   ✅ Baseline metrics loaded: MSE={baseline_metrics['test_loss']:.4f}, MAE={baseline_metrics['test_mae']:.4f}, R²=0.7746")
except Exception as e:
    print(f"   ⚠️  Error loading baseline metrics: {e}")
    print("   Using hardcoded baseline values from training task")
    baseline_metrics = {
        'test_loss': 82.441124,
        'test_mae': 6.763830,
        'test_rmse': 9.079709
    }
    # R² needs to be calculated or hardcoded
    baseline_r2 = 0.7746

# Create performance comparison table
print("\n📈 Performance Comparison: Baseline vs France Team")

# Create comparison DataFrame
performance_comparison = pd.DataFrame({
    'Model': ['Baseline Model', 'France Team'],
    'MSE': [baseline_metrics['test_loss'], france_metrics['mse']],
    'MAE': [baseline_metrics['test_mae'], france_metrics['mae']],
    'RMSE': [baseline_metrics['test_rmse'], france_metrics['rmse']],
    'R²': [0.7746, france_metrics['r2']],  # Baseline R² from training output
    'Dataset Size': [1505, 1617]  # Test set sizes
})

print("\n📊 Performance Comparison Table:")
print(performance_comparison.to_string(index=False))

# Calculate performance differences
mse_diff = france_metrics['mse'] - baseline_metrics['test_loss']
mae_diff = france_metrics['mae'] - baseline_metrics['test_mae']
r2_diff = france_metrics['r2'] - 0.7746

print(f"\n🔍 Performance Difference Analysis:")
print(f"   MSE increase: {mse_diff:.2f} ({(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)")
print(f"   MAE increase: {mae_diff:.2f} ({(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)")
print(f"   R² decrease: {r2_diff:.4f} ({(r2_diff/0.7746)*100:.1f}%)")

# Create comparison visualization
plt.figure(figsize=(15, 6))

# Plot 1: MSE and MAE comparison
plt.subplot(1, 2, 1)
metrics_to_plot = ['MSE', 'MAE']
x = np.arange(len(metrics_to_plot))
width = 0.35

baseline_values = [baseline_metrics['test_loss'], baseline_metrics['test_mae']]
france_values = [france_metrics['mse'], france_metrics['mae']]

plt.bar(x - width/2, baseline_values, width, label='Baseline Model', color='blue', alpha=0.7)
plt.bar(x + width/2, france_values, width, label='France Team', color='red', alpha=0.7)

plt.xlabel('Metrics')
plt.ylabel('Value')
plt.title('Performance Comparison: Baseline vs France')
plt.xticks(x, metrics_to_plot)
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: R² comparison
plt.subplot(1, 2, 2)
plt.bar(['Baseline Model', 'France Team'], [0.7746, france_metrics['r2']],
        color=['blue', 'red'], alpha=0.7)
plt.ylim(0, 1)
plt.ylabel('R² Score')
plt.title('R² Comparison')
plt.grid(True, alpha=0.3)

plt.tight_layout()
comparison_plot_path = os.path.join(output_base_path, "visualizations", "performance_comparison.png")
plt.savefig(comparison_plot_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Performance comparison visualization saved to: {comparison_plot_path}")
plt.close()

# Save comparison table
comparison_csv_path = os.path.join(output_base_path, "model_validation", "performance_comparison.csv")
performance_comparison.to_csv(comparison_csv_path, index=False)
print(f"   💾 Performance comparison table saved to: {comparison_csv_path}")

# Update analysis report with comparison
print("\n📝 Updating analysis report with performance comparison...")
with open(report_path, 'a') as f:
    f.write("\n\nPERFORMANCE COMPARISON: BASELINE MODEL vs FRANCE TEAM\n")
    f.write("-"*50 + "\n")
    f.write("Baseline Model (Test Set Performance):\n")
    f.write(f"   Test MSE: 82.4411\n")
    f.write(f"   Test MAE: 6.7638\n")
    f.write(f"   Test RMSE: 9.0797\n")
    f.write(f"   Test R²: 0.7746\n\n")

    f.write("France Team Performance:\n")
    f.write(f"   Test MSE: {france_metrics['mse']:.4f}\n")
    f.write(f"   Test MAE: {france_metrics['mae']:.4f}\n")
    f.write(f"   Test RMSE: {france_metrics['rmse']:.4f}\n")
    f.write(f"   Test R²: {france_metrics['r2']:.4f}\n\n")

    f.write("Performance Change:\n")
    f.write(f"   MSE increase: {mse_diff:.2f} (+{(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)\n")
    f.write(f"   MAE increase: {mae_diff:.2f} (+{(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)\n")
    f.write(f"   R² decrease: {r2_diff:.4f} ({(r2_diff/0.7746)*100:.1f}%)\n\n")

    f.write("Key Insight:\n")
    f.write("• The model shows reduced performance on France team data compared to the general test set\n")
    f.write("• This indicates domain shift - France's tactical patterns differ from the training data distribution\n")
    f.write("• Model generalization to specific national teams requires further investigation\n")


📊 Loading baseline model performance metrics...
   ✅ Baseline metrics loaded: MSE=82.4411, MAE=6.7638, R²=0.7746

📈 Performance Comparison: Baseline vs France Team

📊 Performance Comparison Table:
         Model        MSE     MAE      RMSE       R²  Dataset Size
Baseline Model  82.441124 6.76383  9.079709 0.774600          1505
   France Team 136.482101 9.12263 11.682555 0.564042          1617

🔍 Performance Difference Analysis:
   MSE increase: 54.04 (65.6%)
   MAE increase: 2.36 (34.9%)
   R² decrease: -0.2106 (-27.2%)
   ✅ Performance comparison visualization saved to: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/France_Test/visualizations/performance_comparison.png
   💾 Performance comparison table saved to: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Fra

# **Baseline_Model_Test_on_England_Team_Only**

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive
import matplotlib.pyplot as plt
import json
import time
from datetime import datetime
import logging

print("== STEP 1: ENVIRONMENT SETUP & MODEL LOADING FOR ENGLAND TEAM EVALUATION ==")

# Mount Google Drive
if not os.path.exists('/content/drive'):
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("Google Drive already mounted")

# Define dataset paths for England team
base_path_england = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/England"

# England data file paths
ball_england_path = os.path.join(base_path_england, "Ball_Features/Ball_Normalized_Filtered_England_Team_Only.csv")
players_england_path = os.path.join(base_path_england, "Players_Features/Normalized_Oredered_England_Team_Only.csv")
possession_england_path = os.path.join(base_path_england, "Possession_Features/England_Team_Only_Sequence_of_5_Possession_Features.csv")

# Output save path for England test results (CORRECTED PATH)
output_base_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/England_Test"

print("\n📁 England Data File Paths:")
print(f"Ball features path: {ball_england_path}")
print(f"Players features path: {players_england_path}")
print(f"Possession features path: {possession_england_path}")
print(f"Output save path: {output_base_path}")

# Create output directory structure
os.makedirs(output_base_path, exist_ok=True)
os.makedirs(os.path.join(output_base_path, "model_validation"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "predictions"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "visualizations"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "analysis"), exist_ok=True)
print(f"\n✅ Output directory structure created at: {output_base_path}")

# Check GPU availability
print("\n🔍 GPU Availability Check:")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"  ✅ {len(gpus)} GPU(s) available for inference")
    for i, gpu in enumerate(gpus):
        print(f"     GPU {i}: {gpu}")

    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("  ✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(f"  ❌ Error setting memory growth: {e}")
else:
    print("  ❌ No GPU available, using CPU for inference")

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
print(f"\n🌱 Random seed set to {seed} for reproducibility")

# Load pre-trained model from previous task
print("\n🧠 Loading pre-trained model from baseline model...")
model_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/model_checkpoints/best_model_epoch_99_val_loss_80.440567.keras"

try:
    england_model = tf.keras.models.load_model(model_path)
    print(f"   ✅ Model loaded successfully from: {model_path}")

    # Verify model architecture
    print("\n✅ Model architecture verification:")
    print(f"   Input shape: {england_model.input_shape}")
    print(f"   Output shape: {england_model.output_shape}")
    print(f"   Total parameters: {england_model.count_params():,}")

    # Save model summary
    model_summary_path = os.path.join(output_base_path, "model_validation", "loaded_model_summary.txt")
    os.makedirs(os.path.dirname(model_summary_path), exist_ok=True)
    with open(model_summary_path, 'w') as f:
        england_model.summary(print_fn=lambda x: f.write(x + '\n'))
    print(f"   📝 Model summary saved to: {model_summary_path}")

except Exception as e:
    print(f"   ❌ Error loading model: {e}")
    raise

# Verify model can handle expected input shape
expected_input_shape = (None, 4, 62)  # batch_size, timesteps, features
if england_model.input_shape != expected_input_shape:
    print(f"   ⚠️  WARNING: Model input shape {england_model.input_shape} doesn't match expected {expected_input_shape}")
    print("   This may cause errors during inference with England data")

# Verify output shape
expected_output_shape = (None, 44)  # batch_size, player coordinates
if england_model.output_shape != expected_output_shape:
    print(f"   ⚠️  WARNING: Model output shape {england_model.output_shape} doesn't match expected {expected_output_shape}")

print("\n✅ STEP 1 COMPLETE: Environment setup and model loading finished")
print("Ready for next step: England data loading and validation")
print(f"\n📊 Next step will process England team data using identical logic to France task")
print("All spatial coordinates used as-is (no normalization applied)")
print("Missing players handled with (-500, -500) coordinates as in training")
print("Batch size for inference: 64 (same as training)")

== STEP 1: ENVIRONMENT SETUP & MODEL LOADING FOR ENGLAND TEAM EVALUATION ==
Google Drive already mounted

📁 England Data File Paths:
Ball features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/England/Ball_Features/Ball_Normalized_Filtered_England_Team_Only.csv
Players features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/England/Players_Features/Normalized_Oredered_England_Team_Only.csv
Possession features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/England/Possession_Features/England_Team_Only_Sequence_of_5_Possession_Features.csv
Output save path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/England_Test

✅ Output directory structure created at: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Base

   📝 Model summary saved to: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/England_Test/model_validation/loaded_model_summary.txt

✅ STEP 1 COMPLETE: Environment setup and model loading finished
Ready for next step: England data loading and validation

📊 Next step will process England team data using identical logic to France task
All spatial coordinates used as-is (no normalization applied)
Missing players handled with (-500, -500) coordinates as in training
Batch size for inference: 64 (same as training)


In [ ]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("\n== STEP 2: ENGLAND DATA LOADING AND VALIDATION ==")
start_time = time.time()

# 1. Load England possession features dataset
print("\n📊 Loading England possession features dataset...")
england_sequence_df = pd.read_csv(
    possession_england_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'passerplayerid': 'float32',  # Use float32 to handle NaN values
        'receiverplayerid': 'float32',  # Use float32 to handle NaN values
        'passtype': 'int8',
        'passoutcometype': 'int8',
        'pressuretype': 'int8',
        'sequence_id': 'int32',
        'timestep': 'int8',
        'global_sequence_id': 'int32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'teamname', 'teamattackingdirection', 'passerplayerid', 'receiverplayerid',
             'passtype', 'passoutcometype', 'pressuretype', 'timestep', 'global_sequence_id', 'sequence_id']
)

print(f"   ✅ England possession features loaded: {len(england_sequence_df):,} rows, {england_sequence_df.shape[1]} columns")

# 2. Load England ball features dataset
print("\n⚽ Loading England ball features dataset...")
england_ball_df = pd.read_csv(
    ball_england_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'ball_x': 'float32',
        'ball_y': 'float32',
        'ball_z': 'float32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'ball_x', 'ball_y', 'ball_z']  # No sequence_id in this file
)

print(f"   ✅ England ball features loaded: {len(england_ball_df):,} rows, {england_ball_df.shape[1]} columns")

# 3. Load England players features dataset
print("\n👥 Loading England players features dataset...")
england_players_df = pd.read_csv(
    players_england_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'jerseynum': 'int8',
        'playerid': 'int32',
        'positiongrouptype': 'category',
        'x': 'float32',
        'y': 'float32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'jerseynum', 'team', 'visibility', 'confidence', 'x', 'y', 'playerid', 'positiongrouptype']  # No sequence_id in this file
)

print(f"   ✅ England players features loaded: {len(england_players_df):,} rows, {england_players_df.shape[1]} columns")

# 4. Data validation and basic statistics (identical to France task)
print("\n🔍 Data validation and basic statistics:")

# Create the five join keys for all datasets
print("   🔑 Creating five join keys (gameid, possessioneventid, eventtime, sequence, period)...")
england_sequence_df['five_key'] = england_sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

england_ball_df['five_key'] = england_ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

england_players_df['five_key'] = england_players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

print("   ✅ Five join keys created successfully")

# Check for missing values in critical columns
print("\n   Missing values check:")
critical_columns = ['gameid', 'possessioneventid', 'eventtime', 'sequence', 'period', 'global_sequence_id']
for col in critical_columns:
    if col in england_sequence_df.columns:
        missing_count = england_sequence_df[col].isna().sum()
        print(f"     England Sequence {col}: {missing_count} missing values")

# CORRECTED: Calculate unique game-sequence combinations for England possessions
print("\n   🔍 Calculating unique England possessions using (gameid, sequence) composite key...")
england_sequence_df['game_sequence_key'] = england_sequence_df.apply(lambda row: (row['gameid'], row['sequence']), axis=1)
unique_england_game_sequences = england_sequence_df['game_sequence_key'].nunique()
unique_england_global_sequences = england_sequence_df['global_sequence_id'].nunique()
total_england_timesteps = len(england_sequence_df)

print(f"\n   📊 England dataset summary:")
print(f"     Unique global sequences: {unique_england_global_sequences:,} (globally unique 5-timestep sequences)")
print(f"     Unique game-sequence combinations: {unique_england_game_sequences:,} (unique England possessions)")
print(f"     Total timesteps: {total_england_timesteps:,}")
print(f"     Average timesteps per global sequence: {total_england_timesteps/unique_england_global_sequences:.1f}")
print(f"     Average timesteps per possession: {total_england_timesteps/unique_england_game_sequences:.1f}")

# Check global_sequence_id distribution
england_global_seq_counts = england_sequence_df['global_sequence_id'].value_counts()
min_timesteps = england_global_seq_counts.min()
max_timesteps = england_global_seq_counts.max()
avg_timesteps = england_global_seq_counts.mean()

print(f"\n   🔢 England global sequence distribution:")
print(f"     Min timesteps per global sequence: {min_timesteps}")
print(f"     Max timesteps per global sequence: {max_timesteps}")
print(f"     Avg timesteps per global sequence: {avg_timesteps:.1f}")

# Check for the expected 5 timesteps per global sequence
england_expected_sequences = england_global_seq_counts[england_global_seq_counts == 5].shape[0]
england_unexpected_sequences = england_global_seq_counts[england_global_seq_counts != 5].shape[0]

print(f"\n   ⚠️ England global sequence validation (expecting 5 timesteps per sequence):")
print(f"     Sequences with exactly 5 timesteps: {england_expected_sequences:,} ({england_expected_sequences/unique_england_global_sequences*100:.1f}%)")
print(f"     Sequences with unexpected timestep count: {england_unexpected_sequences:,} ({england_unexpected_sequences/unique_england_global_sequences*100:.1f}%)")

if england_unexpected_sequences > 0:
    print("     🚨 WARNING: Some England global sequences don't have exactly 5 timesteps!")
    print("            This may require filtering before inference.")

# Store England datasets for next steps
ENGLAND_DATA = {
    'sequence_df': england_sequence_df,
    'ball_df': england_ball_df,
    'players_df': england_players_df
}

total_time = time.time() - start_time
print(f"\n✅ STEP 2 COMPLETE: England data loading and validation finished")
print(f"   ✅ All England datasets loaded successfully")
print(f"   ✅ Basic validation completed with CORRECTED sequence counting")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Feature engineering and sequence construction for England data")
print("✅ Using identical logic to France task for feature extraction")


== STEP 2: ENGLAND DATA LOADING AND VALIDATION ==

📊 Loading England possession features dataset...
   ✅ England possession features loaded: 7,735 rows, 16 columns

⚽ Loading England ball features dataset...
   ✅ England ball features loaded: 3,027 rows, 9 columns

👥 Loading England players features dataset...
   ✅ England players features loaded: 69,366 rows, 14 columns

🔍 Data validation and basic statistics:
   🔑 Creating five join keys (gameid, possessioneventid, eventtime, sequence, period)...
   ✅ Five join keys created successfully

   Missing values check:
     England Sequence gameid: 0 missing values
     England Sequence possessioneventid: 0 missing values
     England Sequence eventtime: 0 missing values
     England Sequence sequence: 0 missing values
     England Sequence period: 0 missing values
     England Sequence global_sequence_id: 0 missing values

   🔍 Calculating unique England possessions using (gameid, sequence) composite key...

   📊 England dataset summary:


In [ ]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("\n== STEP 3: FEATURE ENGINEERING AND SEQUENCE CONSTRUCTION FOR ENGLAND DATA ==")
start_time = time.time()

# 1. Create lookup dictionaries for faster joins (identical to France task logic)
print("\n🔧 Creating lookup dictionaries for faster data joining...")
start_sub = time.time()

# Create ball lookup dictionary: five_key -> ball features
england_ball_lookup = ENGLAND_DATA['ball_df'].set_index('five_key')[['ball_x', 'ball_y', 'ball_z']].to_dict('index')

# Create players lookup dictionary: five_key -> player positions
england_players_grouped = {}
for key, group in ENGLAND_DATA['players_df'].groupby('five_key'):
    england_players_grouped[key] = group[['x', 'y', 'playerid', 'positiongrouptype', 'jerseynum', 'team']].to_dict('records')

# Create next timestep lookup for temporal context
# First, sort by global_sequence_id and timestep
england_sequence_df_sorted = ENGLAND_DATA['sequence_df'].sort_values(['global_sequence_id', 'timestep'])
# Create shifted columns for next timestep within the same global sequence
england_sequence_df_sorted['next_timestep'] = england_sequence_df_sorted.groupby('global_sequence_id')['timestep'].shift(-1)
england_sequence_df_sorted['next_eventtime'] = england_sequence_df_sorted.groupby('global_sequence_id')['eventtime'].shift(-1)

# Create lookup for next timestep context
england_next_timestep_lookup = {}
for idx, row in england_sequence_df_sorted.iterrows():
    if not pd.isna(row['next_timestep']) and row['next_timestep'] == row['timestep'] + 1:
        current_key = (
            row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
        )
        next_key = (
            row['gameid'], row['possessioneventid'], row['next_eventtime'], row['sequence'], row['period']
        )
        england_next_timestep_lookup[current_key] = {
            'next_ball_key': next_key,
            'next_passerplayerid': row['passerplayerid'] if not pd.isna(row['passerplayerid']) else -1,
            'next_receiverplayerid': row['receiverplayerid'] if not pd.isna(row['receiverplayerid']) else -1
        }

sub_time = time.time() - start_sub
print(f"   ✅ Lookup dictionaries built in {sub_time:.2f} seconds")

# 2. Get unique global sequences for England data (already validated to have exactly 5 timesteps)
print("\n📊 Getting unique England global sequences...")
unique_england_global_sequences = ENGLAND_DATA['sequence_df']['global_sequence_id'].unique()
print(f"   📂 Total unique England global sequences: {len(unique_england_global_sequences):,}")

# 3. Feature engineering with validation - CORRECTED: Hard check sequence count matching
print("\n⚙️ Engineering features for England sequence of 5...")
start_sub = time.time()

# Initialize storage for England sequences
X_england_sequences = []  # Input sequences (4 timesteps × 62 features)
y_england_sequences = []  # Target sequences (44 player coordinates for timestep 5)
valid_england_global_sequences = []  # Store valid global sequence IDs

# Create progress bar for sequence processing
seq_progress = tqdm(total=len(unique_england_global_sequences), desc="Building England sequences", position=0, leave=True)

# Track global sequences that will be processed
processed_england_global_sequences = []

for global_seq_id in unique_england_global_sequences:
    # Get all timesteps for this global sequence
    seq_data = ENGLAND_DATA['sequence_df'][ENGLAND_DATA['sequence_df']['global_sequence_id'] == global_seq_id].sort_values('timestep')

    # Validate we have exactly 5 timesteps
    if len(seq_data) != 5:
        seq_progress.update(1)
        continue

    # Prepare input features (timesteps 1-4) and target (timestep 5)
    input_features = []
    has_missing_data = False

    # Process timesteps 1-4 for input
    for timestep in range(1, 5):  # Timesteps 1-4 for input
        row = seq_data[seq_data['timestep'] == timestep].iloc[0]

        # Create the five-key tuple for joining
        key = (row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period'])

        # Get ball features with fallback
        ball_features = england_ball_lookup.get(key, {'ball_x': 0.0, 'ball_y': 0.0, 'ball_z': 0.0})

        # Get player positions (44 features) with fallback
        player_positions = england_players_grouped.get(key, [])
        if len(player_positions) < 22:
            # Handle missing players by using (-500, -500) as default coordinates
            player_coords = np.zeros(44)
            for i in range(22):
                player_coords[i*2] = -500.0
                player_coords[i*2 + 1] = -500.0
            has_missing_data = True
        else:
            # Extract x,y coordinates for all 22 players in order
            player_coords = np.zeros(44)
            for i, player in enumerate(player_positions[:22]):  # Take first 22 players
                player_coords[i*2] = player['x']
                player_coords[i*2 + 1] = player['y']

        # Get event features (8 features)
        passer_id = row['passerplayerid'] if not pd.isna(row['passerplayerid']) else -1
        receiver_id = row['receiverplayerid'] if not pd.isna(row['receiverplayerid']) else -1

        # Get passer and receiver coordinates with fallback
        passer_coords = (-500.0, -500.0)  # Default for missing
        receiver_coords = (-500.0, -500.0)  # Default for missing

        if len(player_positions) >= 22:
            # Find passer and receiver in the player positions
            for player in player_positions:
                if player['playerid'] == passer_id:
                    passer_coords = (player['x'], player['y'])
                if player['playerid'] == receiver_id:
                    receiver_coords = (player['x'], player['y'])

        event_features = [
            row['passtype'] if not pd.isna(row['passtype']) else 0,
            row['passoutcometype'] if not pd.isna(row['passoutcometype']) else 0,
            row['pressuretype'] if not pd.isna(row['pressuretype']) else 0,
            row['period'],
            passer_coords[0], passer_coords[1],
            receiver_coords[0], receiver_coords[1]
        ]

        # Get next timestep context (7 features) for the next timestep in the sequence
        next_context = [0.0, 0.0, 0.0, -500.0, -500.0, -500.0, -500.0]  # Default values

        if key in england_next_timestep_lookup:
            next_info = england_next_timestep_lookup[key]
            next_ball_key = next_info['next_ball_key']
            next_ball = england_ball_lookup.get(next_ball_key, {'ball_x': 0.0, 'ball_y': 0.0, 'ball_z': 0.0})

            # Get next passer/receiver coordinates
            next_passer_coords = (-500.0, -500.0)
            next_receiver_coords = (-500.0, -500.0)

            if next_ball_key in england_players_grouped and len(england_players_grouped[next_ball_key]) >= 22:
                next_players = england_players_grouped[next_ball_key]
                for player in next_players:
                    if player['playerid'] == next_info['next_passerplayerid']:
                        next_passer_coords = (player['x'], player['y'])
                    if player['playerid'] == next_info['next_receiverplayerid']:
                        next_receiver_coords = (player['x'], player['y'])

            next_context = [
                next_ball['ball_x'], next_ball['ball_y'], next_ball['ball_z'],
                next_passer_coords[0], next_passer_coords[1],
                next_receiver_coords[0], next_receiver_coords[1]
            ]

        # Combine all features (44 + 8 + 3 + 7 = 62 features)
        timestep_features = np.concatenate([
            player_coords,
            event_features,
            [ball_features['ball_x'], ball_features['ball_y'], ball_features['ball_z']],
            next_context
        ])

        input_features.append(timestep_features)

    # Get target (timestep 5 player positions)
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    target_key = (timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
                 timestep5_row['sequence'], timestep5_row['period'])

    target_players = england_players_grouped.get(target_key, [])
    if len(target_players) >= 22 and not has_missing_data:
        target_coords = np.zeros(44)
        for i, player in enumerate(target_players[:22]):
            target_coords[i*2] = player['x']
            target_coords[i*2 + 1] = player['y']

        X_england_sequences.append(np.array(input_features))  # Shape: (4, 62)
        y_england_sequences.append(target_coords)  # Shape: (44,)
        valid_england_global_sequences.append(global_seq_id)
        processed_england_global_sequences.append(global_seq_id)

    seq_progress.update(1)

seq_progress.close()
sub_time = time.time() - start_sub
print(f"   ✅ Features engineered for {len(X_england_sequences):,}/{len(unique_england_global_sequences):,} England sequences ({len(X_england_sequences)/len(unique_england_global_sequences)*100:.1f}%)")
print(f"   ⏱️  Feature engineering time: {sub_time:.2f} seconds")

# 4. Convert to numpy arrays and validate shapes - CORRECTED: Hard validation
print("\n📊 Converting to numpy arrays and validating shapes...")
X_england = np.array(X_england_sequences)  # Shape: (num_sequences, 4, 62)
y_england = np.array(y_england_sequences)  # Shape: (num_sequences, 44)

print(f"\n✅ Final England dataset shapes:")
print(f"   Input (X_england): {X_england.shape} - (sequences, timesteps, features)")
print(f"   Target (y_england): {y_england.shape} - (sequences, player_coordinates)")
print(f"   Features per timestep: {X_england.shape[2]} (should be 62)")
print(f"   Player coordinates: {y_england.shape[1]} (should be 44)")

# HARD VALIDATION: Ensure we processed the expected number of sequences
expected_england_sequences = 1547  # From Step 2 validation
actual_england_sequences = len(X_england_sequences)
print(f"\n🔍 HARD SEQUENCE VALIDATION:")
print(f"   Expected global sequences: {expected_england_sequences:,}")
print(f"   Actually processed: {actual_england_sequences:,}")
print(f"   Processing rate: {actual_england_sequences/expected_england_sequences*100:.1f}%")

if actual_england_sequences < expected_england_sequences * 0.95:  # Less than 95% processed
    print("   ⚠️  WARNING: Significant sequence loss during feature engineering!")
    print(f"   Lost {expected_england_sequences - actual_england_sequences:,} sequences")
    print("   Check for missing player data or other filtering issues")

# Validate feature count
assert X_england.shape[2] == 62, f"Expected 62 features per timestep, got {X_england.shape[2]}"
assert y_england.shape[1] == 44, f"Expected 44 target coordinates, got {y_england.shape[1]}"

# Store for next steps
ENGLAND_SEQUENCE_DATA = {
    'X': X_england,
    'y': y_england,
    'valid_global_sequences': valid_england_global_sequences,
    'sequence_df': ENGLAND_DATA['sequence_df'],
    'processed_global_sequences': processed_england_global_sequences
}

total_time = time.time() - start_time
print(f"\n✅ STEP 3 COMPLETE: England feature engineering and sequence construction finished")
print(f"   ✅ Successfully processed {len(X_england_sequences):,} valid England sequences")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Model inference and prediction generation for England data")
print("Note: Using batch size 64 for inference (same as training)")
print("✅ Applied identical feature engineering logic as France task")
print("✅ Hard validation ensures sequence count consistency")


== STEP 3: FEATURE ENGINEERING AND SEQUENCE CONSTRUCTION FOR ENGLAND DATA ==

🔧 Creating lookup dictionaries for faster data joining...
   ✅ Lookup dictionaries built in 9.30 seconds

📊 Getting unique England global sequences...
   📂 Total unique England global sequences: 1,547

⚙️ Engineering features for England sequence of 5...


Building England sequences: 100%|██████████| 1547/1547 [00:04<00:00, 339.83it/s]

   ✅ Features engineered for 1,547/1,547 England sequences (100.0%)
   ⏱️  Feature engineering time: 4.55 seconds

📊 Converting to numpy arrays and validating shapes...

✅ Final England dataset shapes:
   Input (X_england): (1547, 4, 62) - (sequences, timesteps, features)
   Target (y_england): (1547, 44) - (sequences, player_coordinates)
   Features per timestep: 62 (should be 62)
   Player coordinates: 44 (should be 44)

🔍 HARD SEQUENCE VALIDATION:
   Expected global sequences: 1,547
   Actually processed: 1,547
   Processing rate: 100.0%

✅ STEP 3 COMPLETE: England feature engineering and sequence construction finished
   ✅ Successfully processed 1,547 valid England sequences
   ⏱️  Total execution time: 13.86 seconds

Next step: Model inference and prediction generation for England data
Note: Using batch size 64 for inference (same as training)
✅ Applied identical feature engineering logic as France task
✅ Hard validation ensures sequence count consistency


In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from tqdm import tqdm
import os
from datetime import datetime

print("\n== STEP 4: MODEL INFERENCE AND PREDICTION GENERATION FOR ENGLAND DATA ==")
start_time = time.time()

# 1. Generate predictions for England data using the pre-trained model
print("\n🔮 Generating predictions for England data...")
print(f"   Model input shape: {england_model.input_shape}")
print(f"   England data shape: {ENGLAND_SEQUENCE_DATA['X'].shape}")
print(f"   Batch size for inference: 64 (same as training)")

england_predictions = england_model.predict(
    ENGLAND_SEQUENCE_DATA['X'],
    batch_size=64,  # Same batch size as training
    verbose=1
)

print(f"   ✅ Predictions generated: {england_predictions.shape}")

# 2. Load baseline model metrics from training task for comparison
print("\n📊 Loading baseline model performance metrics...")
baseline_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/training_artifacts/evaluation_metrics.json"

try:
    with open(baseline_metrics_path, 'r') as f:
        baseline_metrics = json.load(f)
    print(f"   ✅ Baseline metrics loaded: MSE={baseline_metrics['test_loss']:.4f}, MAE={baseline_metrics['test_mae']:.4f}, R²=0.7746 (from training output)")
    baseline_r2 = 0.7746  # From training output
except Exception as e:
    print(f"   ⚠️  Error loading baseline metrics: {e}")
    print("   Using hardcoded baseline values from training task")
    baseline_metrics = {
        'test_loss': 82.441124,
        'test_mae': 6.763830,
        'test_rmse': 9.079709
    }
    baseline_r2 = 0.7746

# 3. Create the five join keys for data merging (recreate if needed)
print("\n🔑 Recreating five join keys for England data integrity...")
england_sequence_df = ENGLAND_DATA['sequence_df']
england_ball_df = ENGLAND_DATA['ball_df']
england_players_df = ENGLAND_DATA['players_df']

england_sequence_df['five_key'] = england_sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

england_ball_df['five_key'] = england_ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

england_players_df['five_key'] = england_players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

print("   ✅ Five join keys recreated successfully")

# 4. Get England test sequences and create test files
print("\n📁 Creating England test files with original structure...")

# 4.1 Get processed sequence data
england_test_global_ids = ENGLAND_SEQUENCE_DATA['processed_global_sequences']
england_test_sequence_data = england_sequence_df[england_sequence_df['global_sequence_id'].isin(england_test_global_ids)]
england_test_five_keys = england_test_sequence_data['five_key'].unique()

# 4.2 Ball features test data
england_test_ball_data = england_ball_df[england_ball_df['five_key'].isin(england_test_five_keys)]
ball_england_path = os.path.join(output_base_path, "predictions", "ball_features_england_test.csv")
os.makedirs(os.path.dirname(ball_england_path), exist_ok=True)
england_test_ball_data.to_csv(ball_england_path, index=False)
print(f"   ⚽ Ball features England test data saved: {len(england_test_ball_data)} rows")

# 4.3 Possession features test data
england_test_possession_data = england_sequence_df[england_sequence_df['global_sequence_id'].isin(england_test_global_ids)]
possession_england_path = os.path.join(output_base_path, "predictions", "possession_features_england_test.csv")
os.makedirs(os.path.dirname(possession_england_path), exist_ok=True)
england_test_possession_data.to_csv(possession_england_path, index=False)
print(f"   📋 Possession features England test data saved: {len(england_test_possession_data)} rows")

# 4.4 Players test data
england_test_players_data = england_players_df[england_players_df['five_key'].isin(england_test_five_keys)]
players_england_path = os.path.join(output_base_path, "predictions", "players_england_test.csv")
os.makedirs(os.path.dirname(players_england_path), exist_ok=True)
england_test_players_data.to_csv(players_england_path, index=False)
print(f"   👥 Players England test data saved: {len(england_test_players_data)} rows")

# 5. Create predicted players CSV with complete structure
print("\n🎯 Creating predicted players CSV with complete structure including sequence column...")

# Create list to store prediction rows
england_prediction_rows = []

# Create progress bar
progress = tqdm(total=len(england_test_global_ids), desc="Building England prediction CSV", position=0, leave=True)

for i, global_seq_id in enumerate(england_test_global_ids):
    # Get sequence data for this global sequence
    seq_data = england_sequence_df[england_sequence_df['global_sequence_id'] == global_seq_id].sort_values('timestep')

    if len(seq_data) != 5:  # Sequence of 5 has 5 timesteps
        progress.update(1)
        continue

    # Get predicted coordinates for timestep 5
    predicted_coords = england_predictions[i]

    # Process each timestep (1-4) for actual data
    for timestep in range(1, 5):  # Timesteps 1-4 for actual data
        timestep_row = seq_data[seq_data['timestep'] == timestep].iloc[0]
        key = (
            timestep_row['gameid'], timestep_row['possessioneventid'], timestep_row['eventtime'],
            timestep_row['sequence'], timestep_row['period']
        )

        # Get player data for this timestep
        players_for_timestep = england_players_df[england_players_df['five_key'] == key]

        if len(players_for_timestep) < 22:
            continue

        # Add actual player positions (22 players per timestep) with ALL required columns
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row for this player's event
            matching_seq_row = england_sequence_df[
                (england_sequence_df['gameid'] == player_row['gameid']) &
                (england_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (england_sequence_df['eventtime'] == player_row['eventtime']) &
                (england_sequence_df['sequence'] == player_row['sequence']) &
                (england_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not england_sequence_df.empty else timestep_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],  # Using eventtime as starttime
                'endtime': matching_seq_row['eventtime'],    # Using eventtime as endtime
                'duration': 0.0,  # Default duration
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': timestep,
                'global_sequence_id': timestep_row['global_sequence_id']
            }
            england_prediction_rows.append(row_dict)

    # Add timestep 5 actual data
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    key = (
        timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
        timestep5_row['sequence'], timestep5_row['period']
    )

    players_for_timestep = england_players_df[england_players_df['five_key'] == key]

    if len(players_for_timestep) >= 22:
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row
            matching_seq_row = england_sequence_df[
                (england_sequence_df['gameid'] == player_row['gameid']) &
                (england_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (england_sequence_df['eventtime'] == player_row['eventtime']) &
                (england_sequence_df['sequence'] == player_row['sequence']) &
                (england_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not england_sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            england_prediction_rows.append(row_dict)

    # Add timestep 5 predicted data
    if len(players_for_timestep) >= 22:
        for j in range(22):
            player_row = players_for_timestep.iloc[j]
            # Get matching sequence row
            matching_seq_row = england_sequence_df[
                (england_sequence_df['gameid'] == player_row['gameid']) &
                (england_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (england_sequence_df['eventtime'] == player_row['eventtime']) &
                (england_sequence_df['sequence'] == player_row['sequence']) &
                (england_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not england_sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': predicted_coords[j*2],
                'y': predicted_coords[j*2 + 1],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 1,
                'data_type': 'predicted',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            england_prediction_rows.append(row_dict)

    progress.update(1)

progress.close()

# 6. Create and save prediction DataFrame with ALL required columns
print("\n💾 Saving predicted players CSV with complete column structure...")
england_prediction_df = pd.DataFrame(england_prediction_rows)

# Define EXACT column order as requested
required_columns = [
    'gameid', 'gameeventid', 'possessioneventid', 'starttime', 'endtime', 'duration', 'eventtime', 'sequence',
    'playerid', 'positiongrouptype', 'jerseynum', 'team', 'x', 'y', 'visibility', 'confidence',
    'possessioneventtype', 'teamattackingdirection', 'period', 'teamname',
    'is_predicted', 'data_type', 'sequence_id', 'timestep', 'global_sequence_id'
]

# Ensure all required columns exist with proper defaults
for col in required_columns:
    if col not in england_prediction_df.columns:
        if col in ['gameid', 'gameeventid', 'possessioneventid', 'playerid', 'jerseynum', 'period', 'sequence', 'sequence_id', 'timestep', 'global_sequence_id', 'is_predicted']:
            england_prediction_df[col] = 0
        elif col in ['x', 'y', 'starttime', 'endtime', 'duration']:
            england_prediction_df[col] = 0.0
        elif col in ['positiongrouptype', 'team', 'visibility', 'confidence', 'possessioneventtype', 'teamattackingdirection', 'teamname', 'data_type']:
            england_prediction_df[col] = 'Unknown'
        else:
            england_prediction_df[col] = 'missing'

# Reorder columns to EXACT required structure
england_prediction_df = england_prediction_df[required_columns]

predicted_players_england_path = os.path.join(output_base_path, "predictions", "predicted_players_england.csv")
os.makedirs(os.path.dirname(predicted_players_england_path), exist_ok=True)
england_prediction_df.to_csv(predicted_players_england_path, index=False)
print(f"   ✅ Predicted players England CSV saved: {len(england_prediction_df)} rows")
print(f"      • Actual data rows: {len(england_prediction_df[england_prediction_df['data_type'] == 'actual'])}")
print(f"      • Predicted data rows: {len(england_prediction_df[england_prediction_df['data_type'] == 'predicted'])}")
print(f"      • Columns included: {', '.join(england_prediction_df.columns)}")

# 7. Calculate comprehensive metrics for England data
print("\n📈 Calculating comprehensive metrics for England data...")

def calculate_metrics(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))
    r2 = r2_score(y_true.flatten(), y_pred.flatten())
    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

england_metrics = calculate_metrics(ENGLAND_SEQUENCE_DATA['y'], england_predictions)

print("\n📊 England Performance Metrics:")
print(f"   Test MSE: {england_metrics['mse']:.6f}")
print(f"   Test MAE: {england_metrics['mae']:.6f}")
print(f"   Test RMSE: {england_metrics['rmse']:.6f}")
print(f"   Test R²: {england_metrics['r2']:.6f}")

# 8. Create performance comparison with baseline model
print("\n📈 Creating performance comparison: Baseline Model vs England Team")

# Create comparison DataFrame
performance_comparison = pd.DataFrame({
    'Model': ['Baseline Model', 'England Team'],
    'MSE': [baseline_metrics['test_loss'], england_metrics['mse']],
    'MAE': [baseline_metrics['test_mae'], england_metrics['mae']],
    'RMSE': [baseline_metrics['test_rmse'], england_metrics['rmse']],
    'R²': [baseline_r2, england_metrics['r2']],
    'Dataset Size': [1505, 1547]  # Test set sizes
})

print("\n📊 Performance Comparison Table:")
print(performance_comparison.to_string(index=False))

# Calculate performance differences
mse_diff = england_metrics['mse'] - baseline_metrics['test_loss']
mae_diff = england_metrics['mae'] - baseline_metrics['test_mae']
r2_diff = england_metrics['r2'] - baseline_r2

print(f"\n🔍 Performance Difference Analysis:")
print(f"   MSE increase: {mse_diff:.2f} ({(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)")
print(f"   MAE increase: {mae_diff:.2f} ({(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)")
print(f"   R² decrease: {r2_diff:.4f} ({(r2_diff/baseline_r2)*100:.1f}%)")

# 9. Save metrics and create visualizations
print("\n🎨 Creating analysis visualizations and reports...")

# Save performance metrics
metrics_path = os.path.join(output_base_path, "model_validation", "performance_metrics.json")
os.makedirs(os.path.dirname(metrics_path), exist_ok=True)
with open(metrics_path, 'w') as f:
    json.dump(england_metrics, f, indent=2)
print(f"   💾 Performance metrics saved to: {metrics_path}")

# Save comparison table
comparison_csv_path = os.path.join(output_base_path, "model_validation", "performance_comparison.csv")
os.makedirs(os.path.dirname(comparison_csv_path), exist_ok=True)
performance_comparison.to_csv(comparison_csv_path, index=False)
print(f"   💾 Performance comparison table saved to: {comparison_csv_path}")

# Create error analysis visualization
plt.figure(figsize=(15, 6))

# Calculate errors for England data
errors = np.abs(ENGLAND_SEQUENCE_DATA['y'] - england_predictions)
player_errors = errors.reshape(-1, 22, 2)  # (samples, players, coordinates)
avg_player_errors = np.mean(player_errors, axis=(0, 2))  # Average error per player

plt.subplot(1, 2, 1)
plt.bar(range(1, 23), avg_player_errors, color='skyblue')
plt.title('Average Error per Player Position (England Data)')
plt.xlabel('Player Position (1-22)')
plt.ylabel('MAE')
plt.xticks(range(1, 23), [f'P{i}' for i in range(1, 23)], rotation=45)

plt.subplot(1, 2, 2)
all_errors = errors.flatten()
plt.hist(all_errors, bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
plt.axvline(np.mean(all_errors), color='red', linestyle='dashed', linewidth=2, label=f'Mean: {np.mean(all_errors):.2f}')
plt.title('Error Distribution (England Data)')
plt.xlabel('Absolute Error')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
error_path = os.path.join(output_base_path, "visualizations", "england_error_analysis.png")
os.makedirs(os.path.dirname(error_path), exist_ok=True)
plt.savefig(error_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Error analysis visualization saved to: {error_path}")
plt.close()

# Create comparison visualization
plt.figure(figsize=(15, 6))

# Plot 1: MSE and MAE comparison
plt.subplot(1, 2, 1)
metrics_to_plot = ['MSE', 'MAE']
x = np.arange(len(metrics_to_plot))
width = 0.35

baseline_values = [baseline_metrics['test_loss'], baseline_metrics['test_mae']]
england_values = [england_metrics['mse'], england_metrics['mae']]

plt.bar(x - width/2, baseline_values, width, label='Baseline Model', color='blue', alpha=0.7)
plt.bar(x + width/2, england_values, width, label='England Team', color='red', alpha=0.7)

plt.xlabel('Metrics')
plt.ylabel('Value')
plt.title('Performance Comparison: Baseline vs England')
plt.xticks(x, metrics_to_plot)
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: R² comparison
plt.subplot(1, 2, 2)
plt.bar(['Baseline Model', 'England Team'], [baseline_r2, england_metrics['r2']],
        color=['blue', 'red'], alpha=0.7)
plt.ylim(0, 1)
plt.ylabel('R² Score')
plt.title('R² Comparison')
plt.grid(True, alpha=0.3)

plt.tight_layout()
comparison_plot_path = os.path.join(output_base_path, "visualizations", "performance_comparison.png")
os.makedirs(os.path.dirname(comparison_plot_path), exist_ok=True)
plt.savefig(comparison_plot_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Performance comparison visualization saved to: {comparison_plot_path}")
plt.close()

# 10. Generate pitch visualization with actual vs predicted for England data
plt.figure(figsize=(20, 8))

# Select a few representative England sequences to visualize
num_examples = min(4, len(england_test_global_ids))
example_indices = np.random.choice(len(england_test_global_ids), num_examples, replace=False)

for idx, example_idx in enumerate(example_indices):
    global_seq_id = england_test_global_ids[example_idx]
    actual_coords = ENGLAND_SEQUENCE_DATA['y'][example_idx]
    pred_coords = england_predictions[example_idx]

    ax = plt.subplot(1, num_examples, idx+1)

    # Create pitch
    ax.set_xlim(-55, 55)
    ax.set_ylim(-35, 35)
    ax.set_aspect('equal')
    ax.set_title(f'England Sequence {global_seq_id}', fontsize=10)

    # Draw pitch markings
    ax.plot([-52.5, 52.5], [-34, -34], 'k-')  # Bottom
    ax.plot([-52.5, 52.5], [34, 34], 'k-')    # Top
    ax.plot([-52.5, -52.5], [-34, 34], 'k-')  # Left
    ax.plot([52.5, 52.5], [-34, 34], 'k-')    # Right
    ax.plot([0, 0], [-34, 34], 'k--')        # Center line

    # Plot actual positions (blue)
    actual_x = actual_coords[::2]
    actual_y = actual_coords[1::2]
    ax.scatter(actual_x[:11], actual_y[:11], c='blue', s=50, alpha=0.7, label='Actual Home')
    ax.scatter(actual_x[11:], actual_y[11:], c='red', s=50, alpha=0.7, label='Actual Away')

    # Plot predicted positions (green)
    pred_x = pred_coords[::2]
    pred_y = pred_coords[1::2]
    ax.scatter(pred_x[:11], pred_y[:11], c='lightgreen', s=50, marker='x', label='Predicted Home')
    ax.scatter(pred_x[11:], pred_y[11:], c='pink', s=50, marker='x', label='Predicted Away')

    # Draw error vectors
    for j in range(22):
        dx = pred_x[j] - actual_x[j]
        dy = pred_y[j] - actual_y[j]
        ax.arrow(actual_x[j], actual_y[j], dx, dy, color='black', alpha=0.5, width=0.1)

    # Turn off axis ticks and labels for cleaner look
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
pitch_path = os.path.join(output_base_path, "visualizations", "england_actual_vs_predicted_formations.png")
os.makedirs(os.path.dirname(pitch_path), exist_ok=True)
plt.savefig(pitch_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Pitch visualization saved to: {pitch_path}")
plt.close()

# 11. Generate comprehensive analysis report for England data
print("\n📝 Generating comprehensive analysis report for England data...")

report_path = os.path.join(output_base_path, "analysis", f"england_team_analysis_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
os.makedirs(os.path.dirname(report_path), exist_ok=True)

with open(report_path, 'w') as f:
    f.write("="*80 + "\n")
    f.write("ENGLAND TEAM FORMATION PREDICTION - COMPREHENSIVE ANALYSIS REPORT\n")
    f.write("="*80 + "\n\n")

    f.write(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

    f.write("MODEL INFORMATION:\n")
    f.write("-"*50 + "\n")
    f.write(f"Pre-trained model loaded from: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/model_checkpoints/best_model_epoch_99_val_loss_80.440567.keras\n")
    f.write(f"Model input shape: (None, 4, 62)\n")
    f.write(f"Model output shape: (None, 44)\n")
    f.write(f"Total parameters: 167,404\n")
    f.write(f"Batch size used for inference: 64\n\n")

    f.write("ENGLAND DATA SUMMARY:\n")
    f.write("-"*50 + "\n")
    f.write(f"Total unique global sequences: 1,547\n")
    f.write(f"Total timesteps processed: 7,735\n")
    f.write(f"Average timesteps per possession: 36.0\n")
    f.write(f"Sequences successfully processed: 1,547 (100.0%)\n\n")

    f.write("PERFORMANCE COMPARISON: BASELINE MODEL vs ENGLAND TEAM\n")
    f.write("-"*50 + "\n")
    f.write("Baseline Model (Test Set Performance):\n")
    f.write(f"   Test MSE: {baseline_metrics['test_loss']:.4f}\n")
    f.write(f"   Test MAE: {baseline_metrics['test_mae']:.4f}\n")
    f.write(f"   Test RMSE: {baseline_metrics['test_rmse']:.4f}\n")
    f.write(f"   Test R²: {baseline_r2:.4f}\n\n")

    f.write("England Team Performance:\n")
    f.write(f"   Test MSE: {england_metrics['mse']:.4f}\n")
    f.write(f"   Test MAE: {england_metrics['mae']:.4f}\n")
    f.write(f"   Test RMSE: {england_metrics['rmse']:.4f}\n")
    f.write(f"   Test R²: {england_metrics['r2']:.4f}\n\n")

    f.write("Performance Change:\n")
    f.write(f"   MSE : {mse_diff:.2f} (+{(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)\n")
    f.write(f"   MAE : {mae_diff:.2f} (+{(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)\n")
    f.write(f"   R² : {r2_diff:.4f} ({(r2_diff/baseline_r2)*100:.1f}%)\n\n")


    f.write("EXPORTED FILES:\n")
    f.write("-"*50 + "\n")
    f.write(f"1. Ball Features England Test: {ball_england_path}\n")
    f.write(f"   - Rows: {len(england_test_ball_data)}\n")
    f.write(f"   - Columns: {', '.join(england_test_ball_data.columns)}\n\n")
    f.write(f"2. Possession Features England Test: {possession_england_path}\n")
    f.write(f"   - Rows: {len(england_test_possession_data)}\n")
    f.write(f"   - Columns: {', '.join(england_test_possession_data.columns)}\n\n")
    f.write(f"3. Players England Test: {players_england_path}\n")
    f.write(f"   - Rows: {len(england_test_players_data)}\n")
    f.write(f"   - Columns: {', '.join(england_test_players_data.columns)}\n\n")
    f.write(f"4. Predicted Players England: {predicted_players_england_path}\n")
    f.write(f"   - Rows: {len(england_prediction_df)}\n")
    f.write(f"   - Columns: {', '.join(england_prediction_df.columns)} (25 columns including 'sequence')\n")
    f.write(f"   - Structure: {len(england_prediction_df[england_prediction_df['data_type'] == 'actual'])} actual rows + {len(england_prediction_df[england_prediction_df['data_type'] == 'predicted'])} predicted rows\n\n")

    f.write("DATA INTEGRITY GUARANTEES:\n")
    f.write("-"*50 + "\n")
    f.write("• England data processed using identical logic to France task\n")
    f.write("• Same five-key join system: (gameid, possessioneventid, eventtime, sequence, period)\n")
    f.write("• Missing players handled with (-500, -500) coordinates\n")
    f.write("• No spatial normalization applied - coordinates used as-is\n")
    f.write("• Temporal context features extracted identically to training\n")
    f.write("• Batch size 64 used for inference (same as training)\n")

print(f"   ✅ Analysis report saved to: {report_path}")

total_time = time.time() - start_time
print(f"\n✅ STEP 4 COMPLETE: Model inference and prediction generation finished")
print(f"   📊 England test performance: MSE={england_metrics['mse']:.4f}, MAE={england_metrics['mae']:.4f}, R²={england_metrics['r2']:.4f}")
print(f"   ➡️  Performance comparison: MSE increased by {mse_diff:.2f} (+{(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)")
print(f"   💾 All England artifacts saved to: {output_base_path}")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\n🎉 🎉 🎉 ENGLAND TEAM EVALUATION COMPLETED SUCCESSFULLY! 🎉 🎉 🎉")
print(f"\n📥 FINAL ENGLAND EVALUATION ARTIFACTS SAVED TO:")
print(f"   {output_base_path}")
print("\n📊 KEY OUTPUT FILES:")
print(f"   • Predicted Players England: {predicted_players_england_path} (25 columns including 'sequence')")
print(f"   • Performance Metrics: {metrics_path}")
print(f"   • Performance Comparison: {comparison_csv_path}")
print(f"   • Error Analysis: {error_path}")
print(f"   • Performance Comparison Visualization: {comparison_plot_path}")
print(f"   • Pitch Visualization: {pitch_path}")
print(f"   • Analysis Report: {report_path}")


== STEP 4: MODEL INFERENCE AND PREDICTION GENERATION FOR ENGLAND DATA ==

🔮 Generating predictions for England data...
   Model input shape: (None, 4, 62)
   England data shape: (1547, 4, 62)
   Batch size for inference: 64 (same as training)
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step
   ✅ Predictions generated: (1547, 44)

📊 Loading baseline model performance metrics...
   ✅ Baseline metrics loaded: MSE=82.4411, MAE=6.7638, R²=0.7746 (from training output)

🔑 Recreating five join keys for England data integrity...
   ✅ Five join keys recreated successfully

📁 Creating England test files with original structure...
   ⚽ Ball features England test data saved: 2407 rows
   📋 Possession features England test data saved: 7735 rows
   👥 Players England test data saved: 52954 rows

🎯 Creating predicted players CSV with complete structure including sequence column...


Building England prediction CSV: 100%|██████████| 1547/1547 [05:18<00:00,  4.86it/s]



💾 Saving predicted players CSV with complete column structure...
   ✅ Predicted players England CSV saved: 204204 rows
      • Actual data rows: 170170
      • Predicted data rows: 34034
      • Columns included: gameid, gameeventid, possessioneventid, starttime, endtime, duration, eventtime, sequence, playerid, positiongrouptype, jerseynum, team, x, y, visibility, confidence, possessioneventtype, teamattackingdirection, period, teamname, is_predicted, data_type, sequence_id, timestep, global_sequence_id

📈 Calculating comprehensive metrics for England data...

📊 England Performance Metrics:
   Test MSE: 137.155408
   Test MAE: 9.230365
   Test RMSE: 11.711337
   Test R²: 0.565174

📈 Creating performance comparison: Baseline Model vs England Team

📊 Performance Comparison Table:
         Model        MSE      MAE      RMSE       R²  Dataset Size
Baseline Model  82.441124 6.763830  9.079709 0.774600          1505
  England Team 137.155408 9.230365 11.711337 0.565174          1547

🔍 Pe

# **Baseline_Model_Test_on_Argentina_Team_Only**

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive
import matplotlib.pyplot as plt
import json
import time
from datetime import datetime
import logging

print("== STEP 1: ENVIRONMENT SETUP & MODEL LOADING FOR ARGENTINA TEAM EVALUATION ==")

# Mount Google Drive
if not os.path.exists('/content/drive'):
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("Google Drive already mounted")

# Define dataset paths for Argentina team
base_path_argentina = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/Argentina"

# Argentina data file paths
ball_argentina_path = os.path.join(base_path_argentina, "Ball_Features/Ball_Normalized_Filtered_Argentina_Team_Only.csv")
players_argentina_path = os.path.join(base_path_argentina, "Players_Features/Normalized_Ordered_Argentina_Team_Only.csv")
possession_argentina_path = os.path.join(base_path_argentina, "Possession_Features/Argentina_Team_Only_Sequence_of_5_Possession_Features.csv")

# Output save path for Argentina test results
output_base_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Argentina_Test"

print("\n📁 Argentina Data File Paths:")
print(f"Ball features path: {ball_argentina_path}")
print(f"Players features path: {players_argentina_path}")
print(f"Possession features path: {possession_argentina_path}")
print(f"Output save path: {output_base_path}")

# Create output directory structure
os.makedirs(output_base_path, exist_ok=True)
os.makedirs(os.path.join(output_base_path, "model_validation"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "predictions"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "visualizations"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "analysis"), exist_ok=True)
print(f"\n✅ Output directory structure created at: {output_base_path}")

# Check GPU availability
print("\n🔍 GPU Availability Check:")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"  ✅ {len(gpus)} GPU(s) available for inference")
    for i, gpu in enumerate(gpus):
        print(f"     GPU {i}: {gpu}")

    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("  ✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(f"  ❌ Error setting memory growth: {e}")
else:
    print("  ❌ No GPU available, using CPU for inference")

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
print(f"\n🌱 Random seed set to {seed} for reproducibility")

# Load pre-trained model from previous task
print("\n🧠 Loading pre-trained model from baseline model...")
model_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/model_checkpoints/best_model_epoch_99_val_loss_80.440567.keras"

try:
    argentina_model = tf.keras.models.load_model(model_path)
    print(f"   ✅ Model loaded successfully from: {model_path}")

    # Verify model architecture
    print("\n✅ Model architecture verification:")
    print(f"   Input shape: {argentina_model.input_shape}")
    print(f"   Output shape: {argentina_model.output_shape}")
    print(f"   Total parameters: {argentina_model.count_params():,}")

    # Save model summary
    model_summary_path = os.path.join(output_base_path, "model_validation", "loaded_model_summary.txt")
    os.makedirs(os.path.dirname(model_summary_path), exist_ok=True)
    with open(model_summary_path, 'w') as f:
        argentina_model.summary(print_fn=lambda x: f.write(x + '\n'))
    print(f"   📝 Model summary saved to: {model_summary_path}")

except Exception as e:
    print(f"   ❌ Error loading model: {e}")
    raise

# Verify model can handle expected input shape
expected_input_shape = (None, 4, 62)  # batch_size, timesteps, features
if argentina_model.input_shape != expected_input_shape:
    print(f"   ⚠️  WARNING: Model input shape {argentina_model.input_shape} doesn't match expected {expected_input_shape}")
    print("   This may cause errors during inference with Argentina data")

# Verify output shape
expected_output_shape = (None, 44)  # batch_size, player coordinates
if argentina_model.output_shape != expected_output_shape:
    print(f"   ⚠️  WARNING: Model output shape {argentina_model.output_shape} doesn't match expected {expected_output_shape}")

print("\n✅ STEP 1 COMPLETE: Environment setup and model loading finished")
print("Ready for next step: Argentina data loading and validation")
print(f"\n📊 Next step will process Argentina team data using identical logic to England task")
print("All spatial coordinates used as-is (no normalization applied)")
print("Missing players handled with (-500, -500) coordinates as in training")
print("Batch size for inference: 64 (same as training)")
print("✅ Using ALL available Argentina data (no sampling to fixed sequence count)")
print("✅ CORRECTED: Output path uses Argentina_Test directory")

== STEP 1: ENVIRONMENT SETUP & MODEL LOADING FOR ARGENTINA TEAM EVALUATION ==
Google Drive already mounted

📁 Argentina Data File Paths:
Ball features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/Argentina/Ball_Features/Ball_Normalized_Filtered_Argentina_Team_Only.csv
Players features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/Argentina/Players_Features/Normalized_Ordered_Argentina_Team_Only.csv
Possession features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/Argentina/Possession_Features/Argentina_Team_Only_Sequence_of_5_Possession_Features.csv
Output save path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Argentina_Test

✅ Output directory structure created at: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Cro

   📝 Model summary saved to: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Argentina_Test/model_validation/loaded_model_summary.txt

✅ STEP 1 COMPLETE: Environment setup and model loading finished
Ready for next step: Argentina data loading and validation

📊 Next step will process Argentina team data using identical logic to England task
All spatial coordinates used as-is (no normalization applied)
Missing players handled with (-500, -500) coordinates as in training
Batch size for inference: 64 (same as training)
✅ Using ALL available Argentina data (no sampling to fixed sequence count)
✅ CORRECTED: Output path uses Argentina_Test directory


In [ ]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("\n== STEP 2: ARGENTINA DATA LOADING AND VALIDATION ==")
start_time = time.time()

# 1. Load Argentina possession features dataset (ALL DATA)
print("\n📊 Loading Argentina possession features dataset...")
argentina_sequence_df = pd.read_csv(
    possession_argentina_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'passerplayerid': 'float32',  # Use float32 to handle NaN values
        'receiverplayerid': 'float32',  # Use float32 to handle NaN values
        'passtype': 'int8',
        'passoutcometype': 'int8',
        'pressuretype': 'int8',
        'sequence_id': 'int32',
        'timestep': 'int8',
        'global_sequence_id': 'int32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'teamname', 'teamattackingdirection', 'passerplayerid', 'receiverplayerid',
             'passtype', 'passoutcometype', 'pressuretype', 'timestep', 'global_sequence_id', 'sequence_id']
)

print(f"   ✅ Argentina possession features loaded: {len(argentina_sequence_df):,} rows, {argentina_sequence_df.shape[1]} columns")

# 2. Load Argentina ball features dataset (ALL DATA)
print("\n⚽ Loading Argentina ball features dataset...")
argentina_ball_df = pd.read_csv(
    ball_argentina_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'ball_x': 'float32',
        'ball_y': 'float32',
        'ball_z': 'float32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'ball_x', 'ball_y', 'ball_z']  # No sequence_id in this file
)

print(f"   ✅ Argentina ball features loaded: {len(argentina_ball_df):,} rows, {argentina_ball_df.shape[1]} columns")

# 3. Load Argentina players features dataset (ALL DATA)
print("\n👥 Loading Argentina players features dataset...")
argentina_players_df = pd.read_csv(
    players_argentina_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'jerseynum': 'int8',
        'playerid': 'int32',
        'positiongrouptype': 'category',
        'x': 'float32',
        'y': 'float32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'jerseynum', 'team', 'visibility', 'confidence', 'x', 'y', 'playerid', 'positiongrouptype']  # No sequence_id in this file
)

print(f"   ✅ Argentina players features loaded: {len(argentina_players_df):,} rows, {argentina_players_df.shape[1]} columns")

# 4. Data validation and basic statistics (identical to England task, using ALL data)
print("\n🔍 Data validation and basic statistics:")

# Create the five join keys for all datasets
print("   🔑 Creating five join keys (gameid, possessioneventid, eventtime, sequence, period)...")
argentina_sequence_df['five_key'] = argentina_sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

argentina_ball_df['five_key'] = argentina_ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

argentina_players_df['five_key'] = argentina_players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

print("   ✅ Five join keys created successfully")

# Check for missing values in critical columns
print("\n   Missing values check:")
critical_columns = ['gameid', 'possessioneventid', 'eventtime', 'sequence', 'period', 'global_sequence_id']
for col in critical_columns:
    if col in argentina_sequence_df.columns:
        missing_count = argentina_sequence_df[col].isna().sum()
        print(f"     Argentina Sequence {col}: {missing_count} missing values")

# CORRECTED: Calculate unique game-sequence combinations for Argentina possessions (using ALL data)
print("\n   🔍 Calculating unique Argentina possessions using (gameid, sequence) composite key...")
argentina_sequence_df['game_sequence_key'] = argentina_sequence_df.apply(lambda row: (row['gameid'], row['sequence']), axis=1)
unique_argentina_game_sequences = argentina_sequence_df['game_sequence_key'].nunique()
unique_argentina_global_sequences = argentina_sequence_df['global_sequence_id'].nunique()
total_argentina_timesteps = len(argentina_sequence_df)

print(f"\n   📊 Argentina dataset summary (ALL available data):")
print(f"     Unique global sequences: {unique_argentina_global_sequences:,} (globally unique 5-timestep sequences)")
print(f"     Unique game-sequence combinations: {unique_argentina_game_sequences:,} (unique Argentina possessions)")
print(f"     Total timesteps: {total_argentina_timesteps:,}")
print(f"     Average timesteps per global sequence: {total_argentina_timesteps/unique_argentina_global_sequences:.1f}")
print(f"     Average timesteps per possession: {total_argentina_timesteps/unique_argentina_game_sequences:.1f}")

# Check global_sequence_id distribution
argentina_global_seq_counts = argentina_sequence_df['global_sequence_id'].value_counts()
min_timesteps = argentina_global_seq_counts.min()
max_timesteps = argentina_global_seq_counts.max()
avg_timesteps = argentina_global_seq_counts.mean()

print(f"\n   🔢 Argentina global sequence distribution:")
print(f"     Min timesteps per global sequence: {min_timesteps}")
print(f"     Max timesteps per global sequence: {max_timesteps}")
print(f"     Avg timesteps per global sequence: {avg_timesteps:.1f}")

# Check for the expected 5 timesteps per global sequence
argentina_expected_sequences = argentina_global_seq_counts[argentina_global_seq_counts == 5].shape[0]
argentina_unexpected_sequences = argentina_global_seq_counts[argentina_global_seq_counts != 5].shape[0]

print(f"\n   ⚠️ Argentina global sequence validation (expecting 5 timesteps per sequence):")
print(f"     Sequences with exactly 5 timesteps: {argentina_expected_sequences:,} ({argentina_expected_sequences/unique_argentina_global_sequences*100:.1f}%)")
print(f"     Sequences with unexpected timestep count: {argentina_unexpected_sequences:,} ({argentina_unexpected_sequences/unique_argentina_global_sequences*100:.1f}%)")

if argentina_unexpected_sequences > 0:
    print("     🚨 WARNING: Some Argentina global sequences don't have exactly 5 timesteps!")
    print("            This may require filtering before inference.")

# Store Argentina datasets for next steps
ARGENTINA_DATA = {
    'sequence_df': argentina_sequence_df,
    'ball_df': argentina_ball_df,
    'players_df': argentina_players_df
}

total_time = time.time() - start_time
print(f"\n✅ STEP 2 COMPLETE: Argentina data loading and validation finished")
print(f"   ✅ All Argentina datasets loaded successfully (ALL available data used)")
print(f"   ✅ Basic validation completed with CORRECTED sequence counting")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Feature engineering and sequence construction for Argentina data")
print("✅ Using identical logic to England task for feature extraction")
print(f"✅ Processing all {unique_argentina_global_sequences:,} available global sequences (no sampling)")


== STEP 2: ARGENTINA DATA LOADING AND VALIDATION ==

📊 Loading Argentina possession features dataset...
   ✅ Argentina possession features loaded: 10,080 rows, 16 columns

⚽ Loading Argentina ball features dataset...
   ✅ Argentina ball features loaded: 4,282 rows, 9 columns

👥 Loading Argentina players features dataset...
   ✅ Argentina players features loaded: 99,946 rows, 14 columns

🔍 Data validation and basic statistics:
   🔑 Creating five join keys (gameid, possessioneventid, eventtime, sequence, period)...
   ✅ Five join keys created successfully

   Missing values check:
     Argentina Sequence gameid: 0 missing values
     Argentina Sequence possessioneventid: 0 missing values
     Argentina Sequence eventtime: 0 missing values
     Argentina Sequence sequence: 0 missing values
     Argentina Sequence period: 0 missing values
     Argentina Sequence global_sequence_id: 0 missing values

   🔍 Calculating unique Argentina possessions using (gameid, sequence) composite key...

 

In [ ]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("\n== STEP 3: FEATURE ENGINEERING AND SEQUENCE CONSTRUCTION FOR ARGENTINA DATA ==")
start_time = time.time()

# 1. Create lookup dictionaries for faster joins (identical to England task logic)
print("\n🔧 Creating lookup dictionaries for faster data joining...")
start_sub = time.time()

# Create ball lookup dictionary: five_key -> ball features
argentina_ball_lookup = ARGENTINA_DATA['ball_df'].set_index('five_key')[['ball_x', 'ball_y', 'ball_z']].to_dict('index')

# Create players lookup dictionary: five_key -> player positions
argentina_players_grouped = {}
for key, group in ARGENTINA_DATA['players_df'].groupby('five_key'):
    argentina_players_grouped[key] = group[['x', 'y', 'playerid', 'positiongrouptype', 'jerseynum', 'team']].to_dict('records')

# Create next timestep lookup for temporal context
# First, sort by global_sequence_id and timestep
argentina_sequence_df_sorted = ARGENTINA_DATA['sequence_df'].sort_values(['global_sequence_id', 'timestep'])
# Create shifted columns for next timestep within the same global sequence
argentina_sequence_df_sorted['next_timestep'] = argentina_sequence_df_sorted.groupby('global_sequence_id')['timestep'].shift(-1)
argentina_sequence_df_sorted['next_eventtime'] = argentina_sequence_df_sorted.groupby('global_sequence_id')['eventtime'].shift(-1)

# Create lookup for next timestep context
argentina_next_timestep_lookup = {}
for idx, row in argentina_sequence_df_sorted.iterrows():
    if not pd.isna(row['next_timestep']) and row['next_timestep'] == row['timestep'] + 1:
        current_key = (
            row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
        )
        next_key = (
            row['gameid'], row['possessioneventid'], row['next_eventtime'], row['sequence'], row['period']
        )
        argentina_next_timestep_lookup[current_key] = {
            'next_ball_key': next_key,
            'next_passerplayerid': row['passerplayerid'] if not pd.isna(row['passerplayerid']) else -1,
            'next_receiverplayerid': row['receiverplayerid'] if not pd.isna(row['receiverplayerid']) else -1
        }

sub_time = time.time() - start_sub
print(f"   ✅ Lookup dictionaries built in {sub_time:.2f} seconds")

# 2. Get unique global sequences for Argentina data (already validated to have exactly 5 timesteps)
print("\n📊 Getting unique Argentina global sequences...")
unique_argentina_global_sequences = ARGENTINA_DATA['sequence_df']['global_sequence_id'].unique()
print(f"   📂 Total unique Argentina global sequences: {len(unique_argentina_global_sequences):,}")

# 3. Feature engineering with validation - CORRECTED: Hard check sequence count matching
print("\n⚙️ Engineering features for Argentina sequence of 5...")
start_sub = time.time()

# Initialize storage for Argentina sequences
X_argentina_sequences = []  # Input sequences (4 timesteps × 62 features)
y_argentina_sequences = []  # Target sequences (44 player coordinates for timestep 5)
valid_argentina_global_sequences = []  # Store valid global sequence IDs

# Create progress bar for sequence processing
seq_progress = tqdm(total=len(unique_argentina_global_sequences), desc="Building Argentina sequences", position=0, leave=True)

# Track global sequences that will be processed
processed_argentina_global_sequences = []

for global_seq_id in unique_argentina_global_sequences:
    # Get all timesteps for this global sequence
    seq_data = ARGENTINA_DATA['sequence_df'][ARGENTINA_DATA['sequence_df']['global_sequence_id'] == global_seq_id].sort_values('timestep')

    # Validate we have exactly 5 timesteps
    if len(seq_data) != 5:
        seq_progress.update(1)
        continue

    # Prepare input features (timesteps 1-4) and target (timestep 5)
    input_features = []
    has_missing_data = False

    # Process timesteps 1-4 for input
    for timestep in range(1, 5):  # Timesteps 1-4 for input
        row = seq_data[seq_data['timestep'] == timestep].iloc[0]

        # Create the five-key tuple for joining
        key = (row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period'])

        # Get ball features with fallback
        ball_features = argentina_ball_lookup.get(key, {'ball_x': 0.0, 'ball_y': 0.0, 'ball_z': 0.0})

        # Get player positions (44 features) with fallback
        player_positions = argentina_players_grouped.get(key, [])
        if len(player_positions) < 22:
            # Handle missing players by using (-500, -500) as default coordinates
            player_coords = np.zeros(44)
            for i in range(22):
                player_coords[i*2] = -500.0
                player_coords[i*2 + 1] = -500.0
            has_missing_data = True
        else:
            # Extract x,y coordinates for all 22 players in order
            player_coords = np.zeros(44)
            for i, player in enumerate(player_positions[:22]):  # Take first 22 players
                player_coords[i*2] = player['x']
                player_coords[i*2 + 1] = player['y']

        # Get event features (8 features)
        passer_id = row['passerplayerid'] if not pd.isna(row['passerplayerid']) else -1
        receiver_id = row['receiverplayerid'] if not pd.isna(row['receiverplayerid']) else -1

        # Get passer and receiver coordinates with fallback
        passer_coords = (-500.0, -500.0)  # Default for missing
        receiver_coords = (-500.0, -500.0)  # Default for missing

        if len(player_positions) >= 22:
            # Find passer and receiver in the player positions
            for player in player_positions:
                if player['playerid'] == passer_id:
                    passer_coords = (player['x'], player['y'])
                if player['playerid'] == receiver_id:
                    receiver_coords = (player['x'], player['y'])

        event_features = [
            row['passtype'] if not pd.isna(row['passtype']) else 0,
            row['passoutcometype'] if not pd.isna(row['passoutcometype']) else 0,
            row['pressuretype'] if not pd.isna(row['pressuretype']) else 0,
            row['period'],
            passer_coords[0], passer_coords[1],
            receiver_coords[0], receiver_coords[1]
        ]

        # Get next timestep context (7 features) for the next timestep in the sequence
        next_context = [0.0, 0.0, 0.0, -500.0, -500.0, -500.0, -500.0]  # Default values

        if key in argentina_next_timestep_lookup:
            next_info = argentina_next_timestep_lookup[key]
            next_ball_key = next_info['next_ball_key']
            next_ball = argentina_ball_lookup.get(next_ball_key, {'ball_x': 0.0, 'ball_y': 0.0, 'ball_z': 0.0})

            # Get next passer/receiver coordinates
            next_passer_coords = (-500.0, -500.0)
            next_receiver_coords = (-500.0, -500.0)

            if next_ball_key in argentina_players_grouped and len(argentina_players_grouped[next_ball_key]) >= 22:
                next_players = argentina_players_grouped[next_ball_key]
                for player in next_players:
                    if player['playerid'] == next_info['next_passerplayerid']:
                        next_passer_coords = (player['x'], player['y'])
                    if player['playerid'] == next_info['next_receiverplayerid']:
                        next_receiver_coords = (player['x'], player['y'])

            next_context = [
                next_ball['ball_x'], next_ball['ball_y'], next_ball['ball_z'],
                next_passer_coords[0], next_passer_coords[1],
                next_receiver_coords[0], next_receiver_coords[1]
            ]

        # Combine all features (44 + 8 + 3 + 7 = 62 features)
        timestep_features = np.concatenate([
            player_coords,
            event_features,
            [ball_features['ball_x'], ball_features['ball_y'], ball_features['ball_z']],
            next_context
        ])

        input_features.append(timestep_features)

    # Get target (timestep 5 player positions)
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    target_key = (timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
                 timestep5_row['sequence'], timestep5_row['period'])

    target_players = argentina_players_grouped.get(target_key, [])
    if len(target_players) >= 22 and not has_missing_data:
        target_coords = np.zeros(44)
        for i, player in enumerate(target_players[:22]):
            target_coords[i*2] = player['x']
            target_coords[i*2 + 1] = player['y']

        X_argentina_sequences.append(np.array(input_features))  # Shape: (4, 62)
        y_argentina_sequences.append(target_coords)  # Shape: (44,)
        valid_argentina_global_sequences.append(global_seq_id)
        processed_argentina_global_sequences.append(global_seq_id)

    seq_progress.update(1)

seq_progress.close()
sub_time = time.time() - start_sub
print(f"   ✅ Features engineered for {len(X_argentina_sequences):,}/{len(unique_argentina_global_sequences):,} Argentina sequences ({len(X_argentina_sequences)/len(unique_argentina_global_sequences)*100:.1f}%)")
print(f"   ⏱️  Feature engineering time: {sub_time:.2f} seconds")

# 4. Convert to numpy arrays and validate shapes - CORRECTED: Hard validation
print("\n📊 Converting to numpy arrays and validating shapes...")
X_argentina = np.array(X_argentina_sequences)  # Shape: (num_sequences, 4, 62)
y_argentina = np.array(y_argentina_sequences)  # Shape: (num_sequences, 44)

print(f"\n✅ Final Argentina dataset shapes:")
print(f"   Input (X_argentina): {X_argentina.shape} - (sequences, timesteps, features)")
print(f"   Target (y_argentina): {y_argentina.shape} - (sequences, player_coordinates)")
print(f"   Features per timestep: {X_argentina.shape[2]} (should be 62)")
print(f"   Player coordinates: {y_argentina.shape[1]} (should be 44)")

# HARD VALIDATION: Ensure we processed the expected number of sequences
expected_argentina_sequences = 2016  # From Step 2 validation
actual_argentina_sequences = len(X_argentina_sequences)
print(f"\n🔍 HARD SEQUENCE VALIDATION:")
print(f"   Expected global sequences: {expected_argentina_sequences:,}")
print(f"   Actually processed: {actual_argentina_sequences:,}")
print(f"   Processing rate: {actual_argentina_sequences/expected_argentina_sequences*100:.1f}%")

if actual_argentina_sequences < expected_argentina_sequences * 0.95:  # Less than 95% processed
    print("   ⚠️  WARNING: Significant sequence loss during feature engineering!")
    print(f"   Lost {expected_argentina_sequences - actual_argentina_sequences:,} sequences")
    print("   Check for missing player data or other filtering issues")

# Validate feature count
assert X_argentina.shape[2] == 62, f"Expected 62 features per timestep, got {X_argentina.shape[2]}"
assert y_argentina.shape[1] == 44, f"Expected 44 target coordinates, got {y_argentina.shape[1]}"

# Store for next steps
ARGENTINA_SEQUENCE_DATA = {
    'X': X_argentina,
    'y': y_argentina,
    'valid_global_sequences': valid_argentina_global_sequences,
    'sequence_df': ARGENTINA_DATA['sequence_df'],
    'processed_global_sequences': processed_argentina_global_sequences
}

total_time = time.time() - start_time
print(f"\n✅ STEP 3 COMPLETE: Argentina feature engineering and sequence construction finished")
print(f"   ✅ Successfully processed {len(X_argentina_sequences):,} valid Argentina sequences")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Model inference and prediction generation for Argentina data")
print("Note: Using batch size 64 for inference (same as training)")
print("✅ Applied identical feature engineering logic as England task")
print("✅ Hard validation ensures sequence count consistency")
print(f"✅ Using ALL {len(X_argentina_sequences):,} available sequences (no sampling)")


== STEP 3: FEATURE ENGINEERING AND SEQUENCE CONSTRUCTION FOR ARGENTINA DATA ==

🔧 Creating lookup dictionaries for faster data joining...
   ✅ Lookup dictionaries built in 11.67 seconds

📊 Getting unique Argentina global sequences...
   📂 Total unique Argentina global sequences: 2,016

⚙️ Engineering features for Argentina sequence of 5...


Building Argentina sequences: 100%|██████████| 2016/2016 [00:06<00:00, 328.12it/s]

   ✅ Features engineered for 2,016/2,016 Argentina sequences (100.0%)
   ⏱️  Feature engineering time: 6.15 seconds

📊 Converting to numpy arrays and validating shapes...

✅ Final Argentina dataset shapes:
   Input (X_argentina): (2016, 4, 62) - (sequences, timesteps, features)
   Target (y_argentina): (2016, 44) - (sequences, player_coordinates)
   Features per timestep: 62 (should be 62)
   Player coordinates: 44 (should be 44)

🔍 HARD SEQUENCE VALIDATION:
   Expected global sequences: 2,016
   Actually processed: 2,016
   Processing rate: 100.0%

✅ STEP 3 COMPLETE: Argentina feature engineering and sequence construction finished
   ✅ Successfully processed 2,016 valid Argentina sequences
   ⏱️  Total execution time: 17.83 seconds

Next step: Model inference and prediction generation for Argentina data
Note: Using batch size 64 for inference (same as training)
✅ Applied identical feature engineering logic as England task
✅ Hard validation ensures sequence count consistency
✅ Using AL

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from tqdm import tqdm
import os
from datetime import datetime

print("\n== STEP 4: MODEL INFERENCE AND PREDICTION GENERATION FOR ARGENTINA DATA ==")
start_time = time.time()

# 1. Generate predictions for Argentina data using the pre-trained model
print("\n🔮 Generating predictions for Argentina data...")
print(f"   Model input shape: {argentina_model.input_shape}")
print(f"   Argentina data shape: {ARGENTINA_SEQUENCE_DATA['X'].shape}")
print(f"   Batch size for inference: 64 (same as training)")

argentina_predictions = argentina_model.predict(
    ARGENTINA_SEQUENCE_DATA['X'],
    batch_size=64,  # Same batch size as training
    verbose=1
)

print(f"   ✅ Predictions generated: {argentina_predictions.shape}")

# 2. Load baseline model metrics from training task for comparison
print("\n📊 Loading baseline model performance metrics...")
baseline_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/training_artifacts/evaluation_metrics.json"

try:
    with open(baseline_metrics_path, 'r') as f:
        baseline_metrics = json.load(f)
    print(f"   ✅ Baseline metrics loaded: MSE={baseline_metrics['test_loss']:.4f}, MAE={baseline_metrics['test_mae']:.4f}, R²=0.7746 (from training output)")
    baseline_r2 = 0.7746  # From training output
except Exception as e:
    print(f"   ⚠️  Error loading baseline metrics: {e}")
    print("   Using hardcoded baseline values from training task")
    baseline_metrics = {
        'test_loss': 82.441124,
        'test_mae': 6.763830,
        'test_rmse': 9.079709
    }
    baseline_r2 = 0.7746

# 3. Create the five join keys for data merging (recreate if needed)
print("\n🔑 Recreating five join keys for Argentina data integrity...")
argentina_sequence_df = ARGENTINA_DATA['sequence_df']
argentina_ball_df = ARGENTINA_DATA['ball_df']
argentina_players_df = ARGENTINA_DATA['players_df']

argentina_sequence_df['five_key'] = argentina_sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

argentina_ball_df['five_key'] = argentina_ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

argentina_players_df['five_key'] = argentina_players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

print("   ✅ Five join keys recreated successfully")

# 4. Get Argentina test sequences and create test files
print("\n📁 Creating Argentina test files with original structure...")

# 4.1 Get processed sequence data
argentina_test_global_ids = ARGENTINA_SEQUENCE_DATA['processed_global_sequences']
argentina_test_sequence_data = argentina_sequence_df[argentina_sequence_df['global_sequence_id'].isin(argentina_test_global_ids)]
argentina_test_five_keys = argentina_test_sequence_data['five_key'].unique()

# 4.2 Ball features test data
argentina_test_ball_data = argentina_ball_df[argentina_ball_df['five_key'].isin(argentina_test_five_keys)]
ball_argentina_path = os.path.join(output_base_path, "predictions", "ball_features_argentina_test.csv")
os.makedirs(os.path.dirname(ball_argentina_path), exist_ok=True)
argentina_test_ball_data.to_csv(ball_argentina_path, index=False)
print(f"   ⚽ Ball features Argentina test data saved: {len(argentina_test_ball_data)} rows")

# 4.3 Possession features test data
argentina_test_possession_data = argentina_sequence_df[argentina_sequence_df['global_sequence_id'].isin(argentina_test_global_ids)]
possession_argentina_path = os.path.join(output_base_path, "predictions", "possession_features_argentina_test.csv")
os.makedirs(os.path.dirname(possession_argentina_path), exist_ok=True)
argentina_test_possession_data.to_csv(possession_argentina_path, index=False)
print(f"   📋 Possession features Argentina test data saved: {len(argentina_test_possession_data)} rows")

# 4.4 Players test data
argentina_test_players_data = argentina_players_df[argentina_players_df['five_key'].isin(argentina_test_five_keys)]
players_argentina_path = os.path.join(output_base_path, "predictions", "players_argentina_test.csv")
os.makedirs(os.path.dirname(players_argentina_path), exist_ok=True)
argentina_test_players_data.to_csv(players_argentina_path, index=False)
print(f"   👥 Players Argentina test data saved: {len(argentina_test_players_data)} rows")

# 5. Create predicted players CSV with complete structure
print("\n🎯 Creating predicted players CSV with complete structure including sequence column...")

# Create list to store prediction rows
argentina_prediction_rows = []

# Create progress bar
progress = tqdm(total=len(argentina_test_global_ids), desc="Building Argentina prediction CSV", position=0, leave=True)

for i, global_seq_id in enumerate(argentina_test_global_ids):
    # Get sequence data for this global sequence
    seq_data = argentina_sequence_df[argentina_sequence_df['global_sequence_id'] == global_seq_id].sort_values('timestep')

    if len(seq_data) != 5:  # Sequence of 5 has 5 timesteps
        progress.update(1)
        continue

    # Get predicted coordinates for timestep 5
    predicted_coords = argentina_predictions[i]

    # Process each timestep (1-4) for actual data
    for timestep in range(1, 5):  # Timesteps 1-4 for actual data
        timestep_row = seq_data[seq_data['timestep'] == timestep].iloc[0]
        key = (
            timestep_row['gameid'], timestep_row['possessioneventid'], timestep_row['eventtime'],
            timestep_row['sequence'], timestep_row['period']
        )

        # Get player data for this timestep
        players_for_timestep = argentina_players_df[argentina_players_df['five_key'] == key]

        if len(players_for_timestep) < 22:
            continue

        # Add actual player positions (22 players per timestep) with ALL required columns
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row for this player's event
            matching_seq_row = argentina_sequence_df[
                (argentina_sequence_df['gameid'] == player_row['gameid']) &
                (argentina_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (argentina_sequence_df['eventtime'] == player_row['eventtime']) &
                (argentina_sequence_df['sequence'] == player_row['sequence']) &
                (argentina_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not argentina_sequence_df.empty else timestep_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],  # Using eventtime as starttime
                'endtime': matching_seq_row['eventtime'],    # Using eventtime as endtime
                'duration': 0.0,  # Default duration
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': timestep,
                'global_sequence_id': timestep_row['global_sequence_id']
            }
            argentina_prediction_rows.append(row_dict)

    # Add timestep 5 actual data
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    key = (
        timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
        timestep5_row['sequence'], timestep5_row['period']
    )

    players_for_timestep = argentina_players_df[argentina_players_df['five_key'] == key]

    if len(players_for_timestep) >= 22:
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row
            matching_seq_row = argentina_sequence_df[
                (argentina_sequence_df['gameid'] == player_row['gameid']) &
                (argentina_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (argentina_sequence_df['eventtime'] == player_row['eventtime']) &
                (argentina_sequence_df['sequence'] == player_row['sequence']) &
                (argentina_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not argentina_sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            argentina_prediction_rows.append(row_dict)

    # Add timestep 5 predicted data
    if len(players_for_timestep) >= 22:
        for j in range(22):
            player_row = players_for_timestep.iloc[j]
            # Get matching sequence row
            matching_seq_row = argentina_sequence_df[
                (argentina_sequence_df['gameid'] == player_row['gameid']) &
                (argentina_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (argentina_sequence_df['eventtime'] == player_row['eventtime']) &
                (argentina_sequence_df['sequence'] == player_row['sequence']) &
                (argentina_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not argentina_sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': predicted_coords[j*2],
                'y': predicted_coords[j*2 + 1],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 1,
                'data_type': 'predicted',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            argentina_prediction_rows.append(row_dict)

    progress.update(1)

progress.close()

# 6. Create and save prediction DataFrame with ALL required columns
print("\n💾 Saving predicted players CSV with complete column structure...")
argentina_prediction_df = pd.DataFrame(argentina_prediction_rows)

# Define EXACT column order as requested
required_columns = [
    'gameid', 'gameeventid', 'possessioneventid', 'starttime', 'endtime', 'duration', 'eventtime', 'sequence',
    'playerid', 'positiongrouptype', 'jerseynum', 'team', 'x', 'y', 'visibility', 'confidence',
    'possessioneventtype', 'teamattackingdirection', 'period', 'teamname',
    'is_predicted', 'data_type', 'sequence_id', 'timestep', 'global_sequence_id'
]

# Ensure all required columns exist with proper defaults
for col in required_columns:
    if col not in argentina_prediction_df.columns:
        if col in ['gameid', 'gameeventid', 'possessioneventid', 'playerid', 'jerseynum', 'period', 'sequence', 'sequence_id', 'timestep', 'global_sequence_id', 'is_predicted']:
            argentina_prediction_df[col] = 0
        elif col in ['x', 'y', 'starttime', 'endtime', 'duration']:
            argentina_prediction_df[col] = 0.0
        elif col in ['positiongrouptype', 'team', 'visibility', 'confidence', 'possessioneventtype', 'teamattackingdirection', 'teamname', 'data_type']:
            argentina_prediction_df[col] = 'Unknown'
        else:
            argentina_prediction_df[col] = 'missing'

# Reorder columns to EXACT required structure
argentina_prediction_df = argentina_prediction_df[required_columns]

predicted_players_argentina_path = os.path.join(output_base_path, "predictions", "predicted_players_argentina.csv")
os.makedirs(os.path.dirname(predicted_players_argentina_path), exist_ok=True)
argentina_prediction_df.to_csv(predicted_players_argentina_path, index=False)
print(f"   ✅ Predicted players Argentina CSV saved: {len(argentina_prediction_df)} rows")
print(f"      • Actual data rows: {len(argentina_prediction_df[argentina_prediction_df['data_type'] == 'actual'])}")
print(f"      • Predicted data rows: {len(argentina_prediction_df[argentina_prediction_df['data_type'] == 'predicted'])}")
print(f"      • Columns included: {', '.join(argentina_prediction_df.columns)}")

# 7. Calculate comprehensive metrics for Argentina data
print("\n📈 Calculating comprehensive metrics for Argentina data...")

def calculate_metrics(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))
    r2 = r2_score(y_true.flatten(), y_pred.flatten())
    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

argentina_metrics = calculate_metrics(ARGENTINA_SEQUENCE_DATA['y'], argentina_predictions)

print("\n📊 Argentina Performance Metrics:")
print(f"   Test MSE: {argentina_metrics['mse']:.6f}")
print(f"   Test MAE: {argentina_metrics['mae']:.6f}")
print(f"   Test RMSE: {argentina_metrics['rmse']:.6f}")
print(f"   Test R²: {argentina_metrics['r2']:.6f}")

# 8. Create performance comparison with baseline model
print("\n📈 Creating performance comparison: Baseline Model vs Argentina Team")

# Create comparison DataFrame
performance_comparison = pd.DataFrame({
    'Model': ['Baseline Model', 'Argentina Team'],
    'MSE': [baseline_metrics['test_loss'], argentina_metrics['mse']],
    'MAE': [baseline_metrics['test_mae'], argentina_metrics['mae']],
    'RMSE': [baseline_metrics['test_rmse'], argentina_metrics['rmse']],
    'R²': [baseline_r2, argentina_metrics['r2']],
    'Dataset Size': [1505, 2016]  # Test set sizes
})

print("\n📊 Performance Comparison Table:")
print(performance_comparison.to_string(index=False))

# Calculate performance differences
mse_diff = argentina_metrics['mse'] - baseline_metrics['test_loss']
mae_diff = argentina_metrics['mae'] - baseline_metrics['test_mae']
r2_diff = argentina_metrics['r2'] - baseline_r2

print(f"\n🔍 Performance Difference Analysis:")
print(f"   MSE increase: {mse_diff:.2f} ({(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)")
print(f"   MAE increase: {mae_diff:.2f} ({(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)")
print(f"   R² decrease: {r2_diff:.4f} ({(r2_diff/baseline_r2)*100:.1f}%)")

# 9. Save metrics and create visualizations
print("\n🎨 Creating analysis visualizations and reports...")

# Save performance metrics
metrics_path = os.path.join(output_base_path, "model_validation", "performance_metrics.json")
os.makedirs(os.path.dirname(metrics_path), exist_ok=True)
with open(metrics_path, 'w') as f:
    json.dump(argentina_metrics, f, indent=2)
print(f"   💾 Performance metrics saved to: {metrics_path}")

# Save comparison table
comparison_csv_path = os.path.join(output_base_path, "model_validation", "performance_comparison.csv")
os.makedirs(os.path.dirname(comparison_csv_path), exist_ok=True)
performance_comparison.to_csv(comparison_csv_path, index=False)
print(f"   💾 Performance comparison table saved to: {comparison_csv_path}")

# Create error analysis visualization
plt.figure(figsize=(15, 6))

# Calculate errors for Argentina data
errors = np.abs(ARGENTINA_SEQUENCE_DATA['y'] - argentina_predictions)
player_errors = errors.reshape(-1, 22, 2)  # (samples, players, coordinates)
avg_player_errors = np.mean(player_errors, axis=(0, 2))  # Average error per player

plt.subplot(1, 2, 1)
plt.bar(range(1, 23), avg_player_errors, color='skyblue')
plt.title('Average Error per Player Position (Argentina Data)')
plt.xlabel('Player Position (1-22)')
plt.ylabel('MAE')
plt.xticks(range(1, 23), [f'P{i}' for i in range(1, 23)], rotation=45)

plt.subplot(1, 2, 2)
all_errors = errors.flatten()
plt.hist(all_errors, bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
plt.axvline(np.mean(all_errors), color='red', linestyle='dashed', linewidth=2, label=f'Mean: {np.mean(all_errors):.2f}')
plt.title('Error Distribution (Argentina Data)')
plt.xlabel('Absolute Error')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
error_path = os.path.join(output_base_path, "visualizations", "argentina_error_analysis.png")
os.makedirs(os.path.dirname(error_path), exist_ok=True)
plt.savefig(error_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Error analysis visualization saved to: {error_path}")
plt.close()

# Create comparison visualization
plt.figure(figsize=(15, 6))

# Plot 1: MSE and MAE comparison
plt.subplot(1, 2, 1)
metrics_to_plot = ['MSE', 'MAE']
x = np.arange(len(metrics_to_plot))
width = 0.35

baseline_values = [baseline_metrics['test_loss'], baseline_metrics['test_mae']]
argentina_values = [argentina_metrics['mse'], argentina_metrics['mae']]

plt.bar(x - width/2, baseline_values, width, label='Baseline Model', color='blue', alpha=0.7)
plt.bar(x + width/2, argentina_values, width, label='Argentina Team', color='red', alpha=0.7)

plt.xlabel('Metrics')
plt.ylabel('Value')
plt.title('Performance Comparison: Baseline vs Argentina')
plt.xticks(x, metrics_to_plot)
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: R² comparison
plt.subplot(1, 2, 2)
plt.bar(['Baseline Model', 'Argentina Team'], [baseline_r2, argentina_metrics['r2']],
        color=['blue', 'red'], alpha=0.7)
plt.ylim(0, 1)
plt.ylabel('R² Score')
plt.title('R² Comparison')
plt.grid(True, alpha=0.3)

plt.tight_layout()
comparison_plot_path = os.path.join(output_base_path, "visualizations", "performance_comparison.png")
os.makedirs(os.path.dirname(comparison_plot_path), exist_ok=True)
plt.savefig(comparison_plot_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Performance comparison visualization saved to: {comparison_plot_path}")
plt.close()

# 10. Generate pitch visualization with actual vs predicted for Argentina data
plt.figure(figsize=(20, 8))

# Select a few representative Argentina sequences to visualize
num_examples = min(4, len(argentina_test_global_ids))
example_indices = np.random.choice(len(argentina_test_global_ids), num_examples, replace=False)

for idx, example_idx in enumerate(example_indices):
    global_seq_id = argentina_test_global_ids[example_idx]
    actual_coords = ARGENTINA_SEQUENCE_DATA['y'][example_idx]
    pred_coords = argentina_predictions[example_idx]

    ax = plt.subplot(1, num_examples, idx+1)

    # Create pitch
    ax.set_xlim(-55, 55)
    ax.set_ylim(-35, 35)
    ax.set_aspect('equal')
    ax.set_title(f'Argentina Sequence {global_seq_id}', fontsize=10)

    # Draw pitch markings
    ax.plot([-52.5, 52.5], [-34, -34], 'k-')  # Bottom
    ax.plot([-52.5, 52.5], [34, 34], 'k-')    # Top
    ax.plot([-52.5, -52.5], [-34, 34], 'k-')  # Left
    ax.plot([52.5, 52.5], [-34, 34], 'k-')    # Right
    ax.plot([0, 0], [-34, 34], 'k--')        # Center line

    # Plot actual positions (blue)
    actual_x = actual_coords[::2]
    actual_y = actual_coords[1::2]
    ax.scatter(actual_x[:11], actual_y[:11], c='blue', s=50, alpha=0.7, label='Actual Home')
    ax.scatter(actual_x[11:], actual_y[11:], c='red', s=50, alpha=0.7, label='Actual Away')

    # Plot predicted positions (green)
    pred_x = pred_coords[::2]
    pred_y = pred_coords[1::2]
    ax.scatter(pred_x[:11], pred_y[:11], c='lightgreen', s=50, marker='x', label='Predicted Home')
    ax.scatter(pred_x[11:], pred_y[11:], c='pink', s=50, marker='x', label='Predicted Away')

    # Draw error vectors
    for j in range(22):
        dx = pred_x[j] - actual_x[j]
        dy = pred_y[j] - actual_y[j]
        ax.arrow(actual_x[j], actual_y[j], dx, dy, color='black', alpha=0.5, width=0.1)

    # Turn off axis ticks and labels for cleaner look
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
pitch_path = os.path.join(output_base_path, "visualizations", "argentina_actual_vs_predicted_formations.png")
os.makedirs(os.path.dirname(pitch_path), exist_ok=True)
plt.savefig(pitch_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Pitch visualization saved to: {pitch_path}")
plt.close()

# 11. Generate comprehensive analysis report for Argentina data
print("\n📝 Generating comprehensive analysis report for Argentina data...")

report_path = os.path.join(output_base_path, "analysis", f"argentina_team_analysis_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
os.makedirs(os.path.dirname(report_path), exist_ok=True)

with open(report_path, 'w') as f:
    f.write("="*80 + "\n")
    f.write("ARGENTINA TEAM FORMATION PREDICTION - COMPREHENSIVE ANALYSIS REPORT\n")
    f.write("="*80 + "\n\n")

    f.write(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

    f.write("MODEL INFORMATION:\n")
    f.write("-"*50 + "\n")
    f.write(f"Pre-trained model loaded from: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/model_checkpoints/best_model_epoch_99_val_loss_80.440567.keras\n")
    f.write(f"Model input shape: (None, 4, 62)\n")
    f.write(f"Model output shape: (None, 44)\n")
    f.write(f"Total parameters: 167,404\n")
    f.write(f"Batch size used for inference: 64\n\n")

    f.write("ARGENTINA DATA SUMMARY:\n")
    f.write("-"*50 + "\n")
    f.write(f"Total unique global sequences: 2,016\n")
    f.write(f"Total timesteps processed: 10,080\n")
    f.write(f"Average timesteps per possession: 32.4\n")
    f.write(f"Sequences successfully processed: 2,016 (100.0%)\n\n")

    f.write("PERFORMANCE COMPARISON: BASELINE MODEL vs ARGENTINA TEAM\n")
    f.write("-"*50 + "\n")
    f.write("Baseline Model (Test Set Performance):\n")
    f.write(f"   Test MSE: {baseline_metrics['test_loss']:.4f}\n")
    f.write(f"   Test MAE: {baseline_metrics['test_mae']:.4f}\n")
    f.write(f"   Test RMSE: {baseline_metrics['test_rmse']:.4f}\n")
    f.write(f"   Test R²: {baseline_r2:.4f}\n\n")

    f.write("Argentina Team Performance:\n")
    f.write(f"   Test MSE: {argentina_metrics['mse']:.4f}\n")
    f.write(f"   Test MAE: {argentina_metrics['mae']:.4f}\n")
    f.write(f"   Test RMSE: {argentina_metrics['rmse']:.4f}\n")
    f.write(f"   Test R²: {argentina_metrics['r2']:.4f}\n\n")

    f.write("Performance Change:\n")
    f.write(f"   MSE : {mse_diff:.2f} (+{(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)\n")
    f.write(f"   MAE : {mae_diff:.2f} (+{(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)\n")
    f.write(f"   R²: {r2_diff:.4f} ({(r2_diff/baseline_r2)*100:.1f}%)\n\n")




    f.write("EXPORTED FILES:\n")
    f.write("-"*50 + "\n")
    f.write(f"1. Ball Features Argentina Test: {ball_argentina_path}\n")
    f.write(f"   - Rows: {len(argentina_test_ball_data)}\n")
    f.write(f"   - Columns: {', '.join(argentina_test_ball_data.columns)}\n\n")
    f.write(f"2. Possession Features Argentina Test: {possession_argentina_path}\n")
    f.write(f"   - Rows: {len(argentina_test_possession_data)}\n")
    f.write(f"   - Columns: {', '.join(argentina_test_possession_data.columns)}\n\n")
    f.write(f"3. Players Argentina Test: {players_argentina_path}\n")
    f.write(f"   - Rows: {len(argentina_test_players_data)}\n")
    f.write(f"   - Columns: {', '.join(argentina_test_players_data.columns)}\n\n")
    f.write(f"4. Predicted Players Argentina: {predicted_players_argentina_path}\n")
    f.write(f"   - Rows: {len(argentina_prediction_df)}\n")
    f.write(f"   - Columns: {', '.join(argentina_prediction_df.columns)} (25 columns including 'sequence')\n")
    f.write(f"   - Structure: {len(argentina_prediction_df[argentina_prediction_df['data_type'] == 'actual'])} actual rows + {len(argentina_prediction_df[argentina_prediction_df['data_type'] == 'predicted'])} predicted rows\n\n")

    f.write("DATA INTEGRITY GUARANTEES:\n")
    f.write("-"*50 + "\n")
    f.write("• Argentina data processed using identical logic to England task\n")
    f.write("• Same five-key join system: (gameid, possessioneventid, eventtime, sequence, period)\n")
    f.write("• Missing players handled with (-500, -500) coordinates\n")
    f.write("• No spatial normalization applied - coordinates used as-is\n")
    f.write("• Temporal context features extracted identically to training\n")
    f.write("• Batch size 64 used for inference (same as training)\n")

print(f"   ✅ Analysis report saved to: {report_path}")

total_time = time.time() - start_time
print(f"\n✅ STEP 4 COMPLETE: Model inference and prediction generation finished")
print(f"   📊 Argentina test performance: MSE={argentina_metrics['mse']:.4f}, MAE={argentina_metrics['mae']:.4f}, R²={argentina_metrics['r2']:.4f}")
print(f"   ➡️  Performance comparison: MSE increased by {mse_diff:.2f} (+{(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)")
print(f"   💾 All Argentina artifacts saved to: {output_base_path}")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\n🎉 🎉 🎉 ARGENTINA TEAM EVALUATION COMPLETED SUCCESSFULLY! 🎉 🎉 🎉")
print(f"\n📥 FINAL ARGENTINA EVALUATION ARTIFACTS SAVED TO:")
print(f"   {output_base_path}")
print("\n📊 KEY OUTPUT FILES:")
print(f"   • Predicted Players Argentina: {predicted_players_argentina_path} (25 columns including 'sequence')")
print(f"   • Performance Metrics: {metrics_path}")
print(f"   • Performance Comparison: {comparison_csv_path}")
print(f"   • Error Analysis: {error_path}")
print(f"   • Performance Comparison Visualization: {comparison_plot_path}")
print(f"   • Pitch Visualization: {pitch_path}")
print(f"   • Analysis Report: {report_path}")


== STEP 4: MODEL INFERENCE AND PREDICTION GENERATION FOR ARGENTINA DATA ==

🔮 Generating predictions for Argentina data...
   Model input shape: (None, 4, 62)
   Argentina data shape: (2016, 4, 62)
   Batch size for inference: 64 (same as training)
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
   ✅ Predictions generated: (2016, 44)

📊 Loading baseline model performance metrics...
   ✅ Baseline metrics loaded: MSE=82.4411, MAE=6.7638, R²=0.7746 (from training output)

🔑 Recreating five join keys for Argentina data integrity...
   ✅ Five join keys recreated successfully

📁 Creating Argentina test files with original structure...
   ⚽ Ball features Argentina test data saved: 3260 rows
   📋 Possession features Argentina test data saved: 10080 rows
   👥 Players Argentina test data saved: 71720 rows

🎯 Creating predicted players CSV with complete structure including sequence column...


Building Argentina prediction CSV: 100%|██████████| 2016/2016 [07:36<00:00,  4.41it/s]



💾 Saving predicted players CSV with complete column structure...
   ✅ Predicted players Argentina CSV saved: 266112 rows
      • Actual data rows: 221760
      • Predicted data rows: 44352
      • Columns included: gameid, gameeventid, possessioneventid, starttime, endtime, duration, eventtime, sequence, playerid, positiongrouptype, jerseynum, team, x, y, visibility, confidence, possessioneventtype, teamattackingdirection, period, teamname, is_predicted, data_type, sequence_id, timestep, global_sequence_id

📈 Calculating comprehensive metrics for Argentina data...

📊 Argentina Performance Metrics:
   Test MSE: 163.624350
   Test MAE: 9.922672
   Test RMSE: 12.791573
   Test R²: 0.491859

📈 Creating performance comparison: Baseline Model vs Argentina Team

📊 Performance Comparison Table:
         Model        MSE      MAE      RMSE       R²  Dataset Size
Baseline Model  82.441124 6.763830  9.079709 0.774600          1505
Argentina Team 163.624350 9.922672 12.791573 0.491859          20

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from tqdm import tqdm
import os
from datetime import datetime
import json

print("\n== STEP 4: MODEL INFERENCE AND PREDICTION GENERATION FOR ARGENTINA DATA ==")
start_time = time.time()

# 1. Generate predictions for Argentina data using the pre-trained model
print("\n🔮 Generating predictions for Argentina data...")
print(f"   Model input shape: {argentina_model.input_shape}")
print(f"   Argentina data shape: {ARGENTINA_SEQUENCE_DATA['X'].shape}")
print(f"   Batch size for inference: 64 (same as training)")

argentina_predictions = argentina_model.predict(
    ARGENTINA_SEQUENCE_DATA['X'],
    batch_size=64,  # Same batch size as training
    verbose=1
)

print(f"   ✅ Predictions generated: {argentina_predictions.shape}")

# 2. Load baseline model metrics from training task for comparison
print("\n📊 Loading baseline model performance metrics...")
baseline_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/training_artifacts/evaluation_metrics.json"

try:
    with open(baseline_metrics_path, 'r') as f:
        baseline_metrics = json.load(f)
    print(f"   ✅ Baseline metrics loaded: MSE={baseline_metrics['test_loss']:.4f}, MAE={baseline_metrics['test_mae']:.4f}, R²=0.7746 (from training output)")
    baseline_r2 = 0.7746  # From training output
except Exception as e:
    print(f"   ⚠️  Error loading baseline metrics: {e}")
    print("   Using hardcoded baseline values from training task")
    baseline_metrics = {
        'test_loss': 82.441124,
        'test_mae': 6.763830,
        'test_rmse': 9.079709
    }
    baseline_r2 = 0.7746

# 3. Create the five join keys for data merging (recreate if needed)
print("\n🔑 Recreating five join keys for Argentina data integrity...")
argentina_sequence_df = ARGENTINA_DATA['sequence_df']
argentina_ball_df = ARGENTINA_DATA['ball_df']
argentina_players_df = ARGENTINA_DATA['players_df']

argentina_sequence_df['five_key'] = argentina_sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

argentina_ball_df['five_key'] = argentina_ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

argentina_players_df['five_key'] = argentina_players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

print("   ✅ Five join keys recreated successfully")

# 4. Get Argentina test sequences and create test files
print("\n📁 Creating Argentina test files with original structure...")

# 4.1 Get processed sequence data
argentina_test_global_ids = ARGENTINA_SEQUENCE_DATA['processed_global_sequences']
argentina_test_sequence_data = argentina_sequence_df[argentina_sequence_df['global_sequence_id'].isin(argentina_test_global_ids)]
argentina_test_five_keys = argentina_test_sequence_data['five_key'].unique()

# 4.2 Ball features test data
argentina_test_ball_data = argentina_ball_df[argentina_ball_df['five_key'].isin(argentina_test_five_keys)]
ball_argentina_path = os.path.join(output_base_path, "predictions", "ball_features_argentina_test.csv")
os.makedirs(os.path.dirname(ball_argentina_path), exist_ok=True)
argentina_test_ball_data.to_csv(ball_argentina_path, index=False)
print(f"   ⚽ Ball features Argentina test data saved: {len(argentina_test_ball_data)} rows")

# 4.3 Possession features test data
argentina_test_possession_data = argentina_sequence_df[argentina_sequence_df['global_sequence_id'].isin(argentina_test_global_ids)]
possession_argentina_path = os.path.join(output_base_path, "predictions", "possession_features_argentina_test.csv")
os.makedirs(os.path.dirname(possession_argentina_path), exist_ok=True)
argentina_test_possession_data.to_csv(possession_argentina_path, index=False)
print(f"   📋 Possession features Argentina test data saved: {len(argentina_test_possession_data)} rows")

# 4.4 Players test data
argentina_test_players_data = argentina_players_df[argentina_players_df['five_key'].isin(argentina_test_five_keys)]
players_argentina_path = os.path.join(output_base_path, "predictions", "players_argentina_test.csv")
os.makedirs(os.path.dirname(players_argentina_path), exist_ok=True)
argentina_test_players_data.to_csv(players_argentina_path, index=False)
print(f"   👥 Players Argentina test data saved: {len(argentina_test_players_data)} rows")

# 5. Create predicted players CSV with complete structure
print("\n🎯 Creating predicted players CSV with complete structure including sequence column...")

# Create list to store prediction rows
argentina_prediction_rows = []

# Create progress bar
progress = tqdm(total=len(argentina_test_global_ids), desc="Building Argentina prediction CSV", position=0, leave=True)

for i, global_seq_id in enumerate(argentina_test_global_ids):
    # Get sequence data for this global sequence
    seq_data = argentina_sequence_df[argentina_sequence_df['global_sequence_id'] == global_seq_id].sort_values('timestep')

    if len(seq_data) != 5:  # Sequence of 5 has 5 timesteps
        progress.update(1)
        continue

    # Get predicted coordinates for timestep 5
    predicted_coords = argentina_predictions[i]

    # Process each timestep (1-4) for actual data
    for timestep in range(1, 5):  # Timesteps 1-4 for actual data
        timestep_row = seq_data[seq_data['timestep'] == timestep].iloc[0]
        key = (
            timestep_row['gameid'], timestep_row['possessioneventid'], timestep_row['eventtime'],
            timestep_row['sequence'], timestep_row['period']
        )

        # Get player data for this timestep
        players_for_timestep = argentina_players_df[argentina_players_df['five_key'] == key]

        if len(players_for_timestep) < 22:
            continue

        # Add actual player positions (22 players per timestep) with ALL required columns
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row for this player's event
            matching_seq_row = argentina_sequence_df[
                (argentina_sequence_df['gameid'] == player_row['gameid']) &
                (argentina_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (argentina_sequence_df['eventtime'] == player_row['eventtime']) &
                (argentina_sequence_df['sequence'] == player_row['sequence']) &
                (argentina_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not argentina_sequence_df.empty else timestep_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],  # Using eventtime as starttime
                'endtime': matching_seq_row['eventtime'],    # Using eventtime as endtime
                'duration': 0.0,  # Default duration
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': timestep,
                'global_sequence_id': timestep_row['global_sequence_id']
            }
            argentina_prediction_rows.append(row_dict)

    # Add timestep 5 actual data
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    key = (
        timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
        timestep5_row['sequence'], timestep5_row['period']
    )

    players_for_timestep = argentina_players_df[argentina_players_df['five_key'] == key]

    if len(players_for_timestep) >= 22:
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row
            matching_seq_row = argentina_sequence_df[
                (argentina_sequence_df['gameid'] == player_row['gameid']) &
                (argentina_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (argentina_sequence_df['eventtime'] == player_row['eventtime']) &
                (argentina_sequence_df['sequence'] == player_row['sequence']) &
                (argentina_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not argentina_sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            argentina_prediction_rows.append(row_dict)

    # Add timestep 5 predicted data
    if len(players_for_timestep) >= 22:
        for j in range(22):
            player_row = players_for_timestep.iloc[j]
            # Get matching sequence row
            matching_seq_row = argentina_sequence_df[
                (argentina_sequence_df['gameid'] == player_row['gameid']) &
                (argentina_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (argentina_sequence_df['eventtime'] == player_row['eventtime']) &
                (argentina_sequence_df['sequence'] == player_row['sequence']) &
                (argentina_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not argentina_sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': predicted_coords[j*2],
                'y': predicted_coords[j*2 + 1],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 1,
                'data_type': 'predicted',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            argentina_prediction_rows.append(row_dict)

    progress.update(1)

progress.close()

# 6. Create and save prediction DataFrame with ALL required columns
print("\n💾 Saving predicted players CSV with complete column structure...")
argentina_prediction_df = pd.DataFrame(argentina_prediction_rows)

# Define EXACT column order as requested
required_columns = [
    'gameid', 'gameeventid', 'possessioneventid', 'starttime', 'endtime', 'duration', 'eventtime', 'sequence',
    'playerid', 'positiongrouptype', 'jerseynum', 'team', 'x', 'y', 'visibility', 'confidence',
    'possessioneventtype', 'teamattackingdirection', 'period', 'teamname',
    'is_predicted', 'data_type', 'sequence_id', 'timestep', 'global_sequence_id'
]

# Ensure all required columns exist with proper defaults
for col in required_columns:
    if col not in argentina_prediction_df.columns:
        if col in ['gameid', 'gameeventid', 'possessioneventid', 'playerid', 'jerseynum', 'period', 'sequence', 'sequence_id', 'timestep', 'global_sequence_id', 'is_predicted']:
            argentina_prediction_df[col] = 0
        elif col in ['x', 'y', 'starttime', 'endtime', 'duration']:
            argentina_prediction_df[col] = 0.0
        elif col in ['positiongrouptype', 'team', 'visibility', 'confidence', 'possessioneventtype', 'teamattackingdirection', 'teamname', 'data_type']:
            argentina_prediction_df[col] = 'Unknown'
        else:
            argentina_prediction_df[col] = 'missing'

# Reorder columns to EXACT required structure
argentina_prediction_df = argentina_prediction_df[required_columns]

predicted_players_argentina_path = os.path.join(output_base_path, "predictions", "predicted_players_argentina.csv")
os.makedirs(os.path.dirname(predicted_players_argentina_path), exist_ok=True)
argentina_prediction_df.to_csv(predicted_players_argentina_path, index=False)
print(f"   ✅ Predicted players Argentina CSV saved: {len(argentina_prediction_df)} rows")
print(f"      • Actual data rows: {len(argentina_prediction_df[argentina_prediction_df['data_type'] == 'actual'])}")
print(f"      • Predicted data rows: {len(argentina_prediction_df[argentina_prediction_df['data_type'] == 'predicted'])}")
print(f"      • Columns included: {', '.join(argentina_prediction_df.columns)}")

# 7. Calculate comprehensive metrics for Argentina data
print("\n📈 Calculating comprehensive metrics for Argentina data...")

def calculate_metrics(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))
    r2 = r2_score(y_true.flatten(), y_pred.flatten())
    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

argentina_metrics = calculate_metrics(ARGENTINA_SEQUENCE_DATA['y'], argentina_predictions)

print("\n📊 Argentina Performance Metrics:")
print(f"   Test MSE: {argentina_metrics['mse']:.6f}")
print(f"   Test MAE: {argentina_metrics['mae']:.6f}")
print(f"   Test RMSE: {argentina_metrics['rmse']:.6f}")
print(f"   Test R²: {argentina_metrics['r2']:.6f}")

# 8. Create performance comparison with baseline model
print("\n📈 Creating performance comparison: Baseline Model vs Argentina Team")

# Create comparison DataFrame
performance_comparison = pd.DataFrame({
    'Model': ['Baseline Model', 'Argentina Team'],
    'MSE': [baseline_metrics['test_loss'], argentina_metrics['mse']],
    'MAE': [baseline_metrics['test_mae'], argentina_metrics['mae']],
    'RMSE': [baseline_metrics['test_rmse'], argentina_metrics['rmse']],
    'R²': [baseline_r2, argentina_metrics['r2']],
    'Dataset Size': [1505, 2016]  # Test set sizes
})

print("\n📊 Performance Comparison Table:")
print(performance_comparison.to_string(index=False))

# Calculate performance differences
mse_diff = argentina_metrics['mse'] - baseline_metrics['test_loss']
mae_diff = argentina_metrics['mae'] - baseline_metrics['test_mae']
r2_diff = argentina_metrics['r2'] - baseline_r2

print(f"\n🔍 Performance Difference Analysis:")
if mse_diff > 0:
    print(f"   MSE degradation: +{mse_diff:.2f} (+{(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)")
elif mse_diff < 0:
    print(f"   MSE improvement: {mse_diff:.2f} ({(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)")
else:
    print(f"   MSE unchanged: 0.00 (0.0%)")

if mae_diff > 0:
    print(f"   MAE degradation: +{mae_diff:.2f} (+{(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)")
elif mae_diff < 0:
    print(f"   MAE improvement: {mae_diff:.2f} ({(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)")
else:
    print(f"   MAE unchanged: 0.00 (0.0%)")

if r2_diff > 0:
    print(f"   R² improvement: +{r2_diff:.4f} (+{(r2_diff/baseline_r2)*100:.1f}%)")
elif r2_diff < 0:
    print(f"   R² degradation: {r2_diff:.4f} ({(r2_diff/baseline_r2)*100:.1f}%)")
else:
    print(f"   R² unchanged: 0.0000 (0.0%)")

# 9. Save metrics and create visualizations
print("\n🎨 Creating analysis visualizations and reports...")

# Save performance metrics
metrics_path = os.path.join(output_base_path, "model_validation", "performance_metrics.json")
os.makedirs(os.path.dirname(metrics_path), exist_ok=True)
with open(metrics_path, 'w') as f:
    json.dump(argentina_metrics, f, indent=2)
print(f"   💾 Performance metrics saved to: {metrics_path}")

# Save comparison table
comparison_csv_path = os.path.join(output_base_path, "model_validation", "performance_comparison.csv")
os.makedirs(os.path.dirname(comparison_csv_path), exist_ok=True)
performance_comparison.to_csv(comparison_csv_path, index=False)
print(f"   💾 Performance comparison table saved to: {comparison_csv_path}")

# Create error analysis visualization
plt.figure(figsize=(15, 6))

# Calculate errors for Argentina data
errors = np.abs(ARGENTINA_SEQUENCE_DATA['y'] - argentina_predictions)
player_errors = errors.reshape(-1, 22, 2)  # (samples, players, coordinates)
avg_player_errors = np.mean(player_errors, axis=(0, 2))  # Average error per player

plt.subplot(1, 2, 1)
plt.bar(range(1, 23), avg_player_errors, color='skyblue')
plt.title('Average Error per Player Position (Argentina Data)')
plt.xlabel('Player Position (1-22)')
plt.ylabel('MAE')
plt.xticks(range(1, 23), [f'P{i}' for i in range(1, 23)], rotation=45)

plt.subplot(1, 2, 2)
all_errors = errors.flatten()
plt.hist(all_errors, bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
plt.axvline(np.mean(all_errors), color='red', linestyle='dashed', linewidth=2, label=f'Mean: {np.mean(all_errors):.2f}')
plt.title('Error Distribution (Argentina Data)')
plt.xlabel('Absolute Error')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
error_path = os.path.join(output_base_path, "visualizations", "argentina_error_analysis.png")
os.makedirs(os.path.dirname(error_path), exist_ok=True)
plt.savefig(error_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Error analysis visualization saved to: {error_path}")
plt.close()

# Create comparison visualization
plt.figure(figsize=(15, 6))

# Plot 1: MSE and MAE comparison
plt.subplot(1, 2, 1)
metrics_to_plot = ['MSE', 'MAE']
x = np.arange(len(metrics_to_plot))
width = 0.35

baseline_values = [baseline_metrics['test_loss'], baseline_metrics['test_mae']]
argentina_values = [argentina_metrics['mse'], argentina_metrics['mae']]

plt.bar(x - width/2, baseline_values, width, label='Baseline Model', color='blue', alpha=0.7)
plt.bar(x + width/2, argentina_values, width, label='Argentina Team', color='red', alpha=0.7)

plt.xlabel('Metrics')
plt.ylabel('Value')
plt.title('Performance Comparison: Baseline vs Argentina')
plt.xticks(x, metrics_to_plot)
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: R² comparison
plt.subplot(1, 2, 2)
plt.bar(['Baseline Model', 'Argentina Team'], [baseline_r2, argentina_metrics['r2']],
        color=['blue', 'red'], alpha=0.7)
plt.ylim(0, 1)
plt.ylabel('R² Score')
plt.title('R² Comparison')
plt.grid(True, alpha=0.3)

plt.tight_layout()
comparison_plot_path = os.path.join(output_base_path, "visualizations", "performance_comparison.png")
os.makedirs(os.path.dirname(comparison_plot_path), exist_ok=True)
plt.savefig(comparison_plot_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Performance comparison visualization saved to: {comparison_plot_path}")
plt.close()

# 10. Generate pitch visualization with actual vs predicted for Argentina data
plt.figure(figsize=(20, 8))

# Select a few representative Argentina sequences to visualize
num_examples = min(4, len(argentina_test_global_ids))
example_indices = np.random.choice(len(argentina_test_global_ids), num_examples, replace=False)

for idx, example_idx in enumerate(example_indices):
    global_seq_id = argentina_test_global_ids[example_idx]
    actual_coords = ARGENTINA_SEQUENCE_DATA['y'][example_idx]
    pred_coords = argentina_predictions[example_idx]

    ax = plt.subplot(1, num_examples, idx+1)

    # Create pitch
    ax.set_xlim(-55, 55)
    ax.set_ylim(-35, 35)
    ax.set_aspect('equal')
    ax.set_title(f'Argentina Sequence {global_seq_id}', fontsize=10)

    # Draw pitch markings
    ax.plot([-52.5, 52.5], [-34, -34], 'k-')  # Bottom
    ax.plot([-52.5, 52.5], [34, 34], 'k-')    # Top
    ax.plot([-52.5, -52.5], [-34, 34], 'k-')  # Left
    ax.plot([52.5, 52.5], [-34, 34], 'k-')    # Right
    ax.plot([0, 0], [-34, 34], 'k--')        # Center line

    # Plot actual positions (blue)
    actual_x = actual_coords[::2]
    actual_y = actual_coords[1::2]
    ax.scatter(actual_x[:11], actual_y[:11], c='blue', s=50, alpha=0.7, label='Actual Home')
    ax.scatter(actual_x[11:], actual_y[11:], c='red', s=50, alpha=0.7, label='Actual Away')

    # Plot predicted positions (green)
    pred_x = pred_coords[::2]
    pred_y = pred_coords[1::2]
    ax.scatter(pred_x[:11], pred_y[:11], c='lightgreen', s=50, marker='x', label='Predicted Home')
    ax.scatter(pred_x[11:], pred_y[11:], c='pink', s=50, marker='x', label='Predicted Away')

    # Draw error vectors
    for j in range(22):
        dx = pred_x[j] - actual_x[j]
        dy = pred_y[j] - actual_y[j]
        ax.arrow(actual_x[j], actual_y[j], dx, dy, color='black', alpha=0.5, width=0.1)

    # Turn off axis ticks and labels for cleaner look
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
pitch_path = os.path.join(output_base_path, "visualizations", "argentina_actual_vs_predicted_formations.png")
os.makedirs(os.path.dirname(pitch_path), exist_ok=True)
plt.savefig(pitch_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Pitch visualization saved to: {pitch_path}")
plt.close()

# 11. Generate comprehensive analysis report for Argentina data with DYNAMIC analysis
print("\n📝 Generating comprehensive analysis report for Argentina data...")

report_path = os.path.join(output_base_path, "analysis", f"argentina_team_analysis_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
os.makedirs(os.path.dirname(report_path), exist_ok=True)

with open(report_path, 'w') as f:
    f.write("="*80 + "\n")
    f.write("ARGENTINA TEAM FORMATION PREDICTION - COMPREHENSIVE ANALYSIS REPORT\n")
    f.write("="*80 + "\n\n")

    f.write(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

    f.write("MODEL INFORMATION:\n")
    f.write("-"*50 + "\n")
    f.write(f"Pre-trained model loaded from: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/model_checkpoints/best_model_epoch_99_val_loss_80.440567.keras\n")
    f.write(f"Model input shape: (None, 4, 62)\n")
    f.write(f"Model output shape: (None, 44)\n")
    f.write(f"Total parameters: 167,404\n")
    f.write(f"Batch size used for inference: 64\n\n")

    f.write("ARGENTINA DATA SUMMARY:\n")
    f.write("-"*50 + "\n")
    f.write(f"Total unique global sequences: 2,016\n")
    f.write(f"Total timesteps processed: 10,080\n")
    f.write(f"Average timesteps per possession: 32.4\n")
    f.write(f"Sequences successfully processed: 2,016 (100.0%)\n\n")

    f.write("PERFORMANCE COMPARISON: BASELINE MODEL vs ARGENTINA TEAM\n")
    f.write("-"*50 + "\n")
    f.write("Baseline Model (Test Set Performance):\n")
    f.write(f"   Test MSE: {baseline_metrics['test_loss']:.4f}\n")
    f.write(f"   Test MAE: {baseline_metrics['test_mae']:.4f}\n")
    f.write(f"   Test RMSE: {baseline_metrics['test_rmse']:.4f}\n")
    f.write(f"   Test R²: {baseline_r2:.4f}\n\n")

    f.write("Argentina Team Performance:\n")
    f.write(f"   Test MSE: {argentina_metrics['mse']:.4f}\n")
    f.write(f"   Test MAE: {argentina_metrics['mae']:.4f}\n")
    f.write(f"   Test RMSE: {argentina_metrics['rmse']:.4f}\n")
    f.write(f"   Test R²: {argentina_metrics['r2']:.4f}\n\n")

    # DYNAMIC PERFORMANCE CHANGE ANALYSIS
    performance_change_text = "Performance Change:\n"
    performance_change_text += "-"*50 + "\n"

    # MSE change
    if mse_diff > 0:
        performance_change_text += f"   MSE degradation: +{mse_diff:.2f} (+{(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)\n"
    elif mse_diff < 0:
        performance_change_text += f"   MSE improvement: {mse_diff:.2f} ({(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)\n"
    else:
        performance_change_text += f"   MSE unchanged: 0.00 (0.0%)\n"

    # MAE change
    if mae_diff > 0:
        performance_change_text += f"   MAE degradation: +{mae_diff:.2f} (+{(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)\n"
    elif mae_diff < 0:
        performance_change_text += f"   MAE improvement: {mae_diff:.2f} ({(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)\n"
    else:
        performance_change_text += f"   MAE unchanged: 0.00 (0.0%)\n"

    # R² change
    if r2_diff > 0:
        performance_change_text += f"   R² improvement: +{r2_diff:.4f} (+{(r2_diff/baseline_r2)*100:.1f}%)\n"
    elif r2_diff < 0:
        performance_change_text += f"   R² degradation: {r2_diff:.4f} ({(r2_diff/baseline_r2)*100:.1f}%)\n"
    else:
        performance_change_text += f"   R² unchanged: 0.0000 (0.0%)\n\n"

    f.write(performance_change_text)

    # DYNAMIC KEY INSIGHTS
    insights_text = "Key Insights:\n"
    insights_text += "-"*50 + "\n"

    if mse_diff > 0 and mae_diff > 0 and r2_diff < 0:
        insights_text += "• The model shows reduced performance on Argentina team data compared to the general test set\n"
        insights_text += "• This indicates domain shift - Argentina's tactical patterns differ from the training data distribution\n"
        insights_text += "• Model generalization to specific national teams requires further investigation\n"
    elif mse_diff < 0 and mae_diff < 0 and r2_diff > 0:
        insights_text += "• The model shows improved performance on Argentina team data compared to the general test set\n"
        insights_text += "• This suggests Argentina's tactical patterns align well with the training data distribution\n"
        insights_text += "• The model generalizes effectively to Argentina's playing style\n"
    else:
        insights_text += "• Mixed performance on Argentina team data compared to baseline\n"
        if mse_diff > 0:
            insights_text += "• MSE degradation suggests higher variance in prediction errors\n"
        if mae_diff < 0:
            insights_text += "• MAE improvement indicates more consistent prediction accuracy\n"
        if r2_diff > 0:
            insights_text += "• R² improvement shows better explanation of formation variance\n"
        insights_text += "• Further analysis needed to understand performance patterns\n"

    insights_text += f"• Spatial interpretation: {argentina_metrics['mae']:.2f} MAE = ~{(argentina_metrics['mae'] * 0.68):.2f} meter average error per player on 105m pitch\n\n"
    f.write(insights_text)

    # DYNAMIC CROSS-TEAM PERFORMANCE SUMMARY (Option 1)
    try:
        print("\n🔄 Loading previous team metrics for cross-team comparison...")

        # Define paths to previous team metrics
        france_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/France_Test/model_validation/performance_metrics.json"
        england_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/England_Test/model_validation/performance_metrics.json"

        # Load previous team metrics
        france_metrics = {}
        england_metrics = {}
        team_metrics_loaded = 0

        if os.path.exists(france_metrics_path):
            with open(france_metrics_path, 'r') as f_france:
                france_metrics = json.load(f_france)
                team_metrics_loaded += 1
            print(f"   ✅ France metrics loaded from: {france_metrics_path}")
        else:
            print(f"   ⚠️  France metrics file not found: {france_metrics_path}")

        if os.path.exists(england_metrics_path):
            with open(england_metrics_path, 'r') as f_england:
                england_metrics = json.load(f_england)
                team_metrics_loaded += 1
            print(f"   ✅ England metrics loaded from: {england_metrics_path}")
        else:
            print(f"   ⚠️  England metrics file not found: {england_metrics_path}")

        # Create cross-team comparison
        f.write("CROSS-TEAM PERFORMANCE SUMMARY:\n")
        f.write("-"*50 + "\n")
        f.write("This Argentina evaluation completes the cross-team analysis with France and England:\n")

        # Add France metrics if available
        if france_metrics:
            f.write(f"   • France Team: {france_metrics['mae']:.2f} MAE, {france_metrics['r2']:.2f} R² (from previous evaluation)\n")
        else:
            f.write("   • France Team: Metrics unavailable (file not found)\n")

        # Add England metrics if available
        if england_metrics:
            f.write(f"   • England Team: {england_metrics['mae']:.2f} MAE, {england_metrics['r2']:.2f} R² (from previous evaluation)\n")
        else:
            f.write("   • England Team: Metrics unavailable (file not found)\n")

        # Add Argentina metrics
        f.write(f"   • Argentina Team: {argentina_metrics['mae']:.2f} MAE, {argentina_metrics['r2']:.2f} R² (current evaluation)\n\n")

        # Calculate and report performance degradation if possible
        degradation_calculated = False
        degradation_values = []

        if france_metrics and england_metrics:
            # Calculate performance degradation for each team vs baseline
            france_mae_diff = (france_metrics['mae'] - baseline_metrics['test_mae']) / baseline_metrics['test_mae'] * 100
            england_mae_diff = (england_metrics['mae'] - baseline_metrics['test_mae']) / baseline_metrics['test_mae'] * 100
            argentina_mae_diff = (argentina_metrics['mae'] - baseline_metrics['test_mae']) / baseline_metrics['test_mae'] * 100

            degradation_values = [france_mae_diff, england_mae_diff, argentina_mae_diff]
            avg_degradation = sum(degradation_values) / len(degradation_values)
            degradation_calculated = True

        if degradation_calculated:
            f.write(f"Key Finding: Cross-team analysis shows average MAE degradation of {avg_degradation:.1f}% compared to baseline\n")

            # Check if degradation is consistent across teams
            max_degradation = max(degradation_values)
            min_degradation = min(degradation_values)
            degradation_range = max_degradation - min_degradation

            if degradation_range < 5.0:  # Less than 5% variation
                f.write("• Performance degradation is remarkably consistent across all three national teams\n")
                f.write("• This suggests systematic model limitations when generalizing to top-tier national teams\n\n")
            else:
                f.write("• Performance degradation varies significantly across teams (range: {degradation_range:.1f}%)\n")
                f.write("• This indicates team-specific tactical patterns that affect model generalization differently\n\n")
        else:
            f.write("Key Finding: Cross-team degradation analysis requires complete metrics from all three teams\n")
            f.write("Current analysis shows Argentina performance compared to baseline model\n\n")

    except Exception as e:
        print(f"   ⚠️  Error during cross-team comparison: {str(e)}")
        f.write("CROSS-TEAM PERFORMANCE SUMMARY:\n")
        f.write("-"*50 + "\n")
        f.write("⚠️  Unable to load previous team metrics for comparison\n")
        f.write("   This Argentina evaluation completes the national team analysis, but cross-team comparison unavailable\n")
        f.write(f"   • Argentina Team: {argentina_metrics['mae']:.2f} MAE, {argentina_metrics['r2']:.2f} R² (current evaluation)\n\n")
        f.write(f"Note: Cross-team comparison failed with error: {str(e)}\n\n")

    f.write("EXPORTED FILES:\n")
    f.write("-"*50 + "\n")
    f.write(f"1. Ball Features Argentina Test: {ball_argentina_path}\n")
    f.write(f"   - Rows: {len(argentina_test_ball_data)}\n")
    f.write(f"   - Columns: {', '.join(argentina_test_ball_data.columns)}\n\n")
    f.write(f"2. Possession Features Argentina Test: {possession_argentina_path}\n")
    f.write(f"   - Rows: {len(argentina_test_possession_data)}\n")
    f.write(f"   - Columns: {', '.join(argentina_test_possession_data.columns)}\n\n")
    f.write(f"3. Players Argentina Test: {players_argentina_path}\n")
    f.write(f"   - Rows: {len(argentina_test_players_data)}\n")
    f.write(f"   - Columns: {', '.join(argentina_test_players_data.columns)}\n\n")
    f.write(f"4. Predicted Players Argentina: {predicted_players_argentina_path}\n")
    f.write(f"   - Rows: {len(argentina_prediction_df)}\n")
    f.write(f"   - Columns: {', '.join(argentina_prediction_df.columns)} (25 columns including 'sequence')\n")
    f.write(f"   - Structure: {len(argentina_prediction_df[argentina_prediction_df['data_type'] == 'actual'])} actual rows + {len(argentina_prediction_df[argentina_prediction_df['data_type'] == 'predicted'])} predicted rows\n\n")

    f.write("DATA INTEGRITY GUARANTEES:\n")
    f.write("-"*50 + "\n")
    f.write("• Argentina data processed using identical logic to England task\n")
    f.write("• Same five-key join system: (gameid, possessioneventid, eventtime, sequence, period)\n")
    f.write("• Missing players handled with (-500, -500) coordinates\n")
    f.write("• No spatial normalization applied - coordinates used as-is\n")
    f.write("• Temporal context features extracted identically to training\n")
    f.write("• Batch size 64 used for inference (same as training)\n")

print(f"   ✅ Analysis report saved to: {report_path}")

total_time = time.time() - start_time
print(f"\n✅ STEP 4 COMPLETE: Model inference and prediction generation finished")
print(f"   📊 Argentina test performance: MSE={argentina_metrics['mse']:.4f}, MAE={argentina_metrics['mae']:.4f}, R²={argentina_metrics['r2']:.4f}")
print(f"   💾 All Argentina artifacts saved to: {output_base_path}")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\n🎉 🎉 🎉 ARGENTINA TEAM EVALUATION COMPLETED SUCCESSFULLY! 🎉 🎉 🎉")
print(f"\n📥 FINAL ARGENTINA EVALUATION ARTIFACTS SAVED TO:")
print(f"   {output_base_path}")
print("\n📊 KEY OUTPUT FILES:")
print(f"   • Predicted Players Argentina: {predicted_players_argentina_path} (25 columns including 'sequence')")
print(f"   • Performance Metrics: {metrics_path}")
print(f"   • Performance Comparison: {comparison_csv_path}")
print(f"   • Error Analysis: {error_path}")
print(f"   • Performance Comparison Visualization: {comparison_plot_path}")
print(f"   • Pitch Visualization: {pitch_path}")
print(f"   • Analysis Report: {report_path}")


== STEP 4: MODEL INFERENCE AND PREDICTION GENERATION FOR ARGENTINA DATA ==

🔮 Generating predictions for Argentina data...
   Model input shape: (None, 4, 62)
   Argentina data shape: (2016, 4, 62)
   Batch size for inference: 64 (same as training)
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
   ✅ Predictions generated: (2016, 44)

📊 Loading baseline model performance metrics...
   ✅ Baseline metrics loaded: MSE=82.4411, MAE=6.7638, R²=0.7746 (from training output)

🔑 Recreating five join keys for Argentina data integrity...
   ✅ Five join keys recreated successfully

📁 Creating Argentina test files with original structure...
   ⚽ Ball features Argentina test data saved: 3260 rows
   📋 Possession features Argentina test data saved: 10080 rows
   👥 Players Argentina test data saved: 71720 rows

🎯 Creating predicted players CSV with complete structure including sequence column...


Building Argentina prediction CSV: 100%|██████████| 2016/2016 [07:45<00:00,  4.33it/s]



💾 Saving predicted players CSV with complete column structure...
   ✅ Predicted players Argentina CSV saved: 266112 rows
      • Actual data rows: 221760
      • Predicted data rows: 44352
      • Columns included: gameid, gameeventid, possessioneventid, starttime, endtime, duration, eventtime, sequence, playerid, positiongrouptype, jerseynum, team, x, y, visibility, confidence, possessioneventtype, teamattackingdirection, period, teamname, is_predicted, data_type, sequence_id, timestep, global_sequence_id

📈 Calculating comprehensive metrics for Argentina data...

📊 Argentina Performance Metrics:
   Test MSE: 163.624350
   Test MAE: 9.922672
   Test RMSE: 12.791573
   Test R²: 0.491859

📈 Creating performance comparison: Baseline Model vs Argentina Team

📊 Performance Comparison Table:
         Model        MSE      MAE      RMSE       R²  Dataset Size
Baseline Model  82.441124 6.763830  9.079709 0.774600          1505
Argentina Team 163.624350 9.922672 12.791573 0.491859          20

# **Baseline_Model_Test_on_Croatia_Team_Only**

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive
import matplotlib.pyplot as plt
import json
import time
from datetime import datetime
import logging

print("== STEP 1: ENVIRONMENT SETUP & MODEL LOADING FOR CROATIA TEAM EVALUATION ==")

# Mount Google Drive
if not os.path.exists('/content/drive'):
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("Google Drive already mounted")

# Define dataset paths for Croatia team
base_path_croatia = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/Croatia"

# Croatia data file paths
ball_croatia_path = os.path.join(base_path_croatia, "Ball_Features/Ball_Normalized_Filtered_Croatia_Team_Only.csv")
players_croatia_path = os.path.join(base_path_croatia, "Players_Features/Normalized_Ordered_Croatia_Team_Only.csv")
possession_croatia_path = os.path.join(base_path_croatia, "Possession_Features/Croatia_Team_Only_Sequence_of_5_Possession_Features.csv")

# Output save path for Croatia test results
output_base_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Croatia_Test"

print("\n📁 Croatia Data File Paths:")
print(f"Ball features path: {ball_croatia_path}")
print(f"Players features path: {players_croatia_path}")
print(f"Possession features path: {possession_croatia_path}")
print(f"Output save path: {output_base_path}")

# Create output directory structure
os.makedirs(output_base_path, exist_ok=True)
os.makedirs(os.path.join(output_base_path, "model_validation"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "predictions"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "visualizations"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "analysis"), exist_ok=True)
print(f"\n✅ Output directory structure created at: {output_base_path}")

# Check GPU availability
print("\n🔍 GPU Availability Check:")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"  ✅ {len(gpus)} GPU(s) available for inference")
    for i, gpu in enumerate(gpus):
        print(f"     GPU {i}: {gpu}")

    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("  ✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(f"  ❌ Error setting memory growth: {e}")
else:
    print("  ❌ No GPU available, using CPU for inference")

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
print(f"\n🌱 Random seed set to {seed} for reproducibility")

# Load pre-trained model from previous task
print("\n🧠 Loading pre-trained model from baseline model...")
model_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/model_checkpoints/best_model_epoch_99_val_loss_80.440567.keras"

try:
    croatia_model = tf.keras.models.load_model(model_path)
    print(f"   ✅ Model loaded successfully from: {model_path}")

    # Verify model architecture
    print("\n✅ Model architecture verification:")
    print(f"   Input shape: {croatia_model.input_shape}")
    print(f"   Output shape: {croatia_model.output_shape}")
    print(f"   Total parameters: {croatia_model.count_params():,}")

    # Save model summary
    model_summary_path = os.path.join(output_base_path, "model_validation", "loaded_model_summary.txt")
    os.makedirs(os.path.dirname(model_summary_path), exist_ok=True)
    with open(model_summary_path, 'w') as f:
        croatia_model.summary(print_fn=lambda x: f.write(x + '\n'))
    print(f"   📝 Model summary saved to: {model_summary_path}")

except Exception as e:
    print(f"   ❌ Error loading model: {e}")
    raise

# Verify model can handle expected input shape
expected_input_shape = (None, 4, 62)  # batch_size, timesteps, features
if croatia_model.input_shape != expected_input_shape:
    print(f"   ⚠️  WARNING: Model input shape {croatia_model.input_shape} doesn't match expected {expected_input_shape}")
    print("   This may cause errors during inference with Croatia data")

# Verify output shape
expected_output_shape = (None, 44)  # batch_size, player coordinates
if croatia_model.output_shape != expected_output_shape:
    print(f"   ⚠️  WARNING: Model output shape {croatia_model.output_shape} doesn't match expected {expected_output_shape}")

print("\n✅ STEP 1 COMPLETE: Environment setup and model loading finished")
print("Ready for next step: Croatia data loading and validation")
print(f"\n📊 Next step will process Croatia team data using identical logic to Argentina task")
print("Missing players handled with (-500, -500) coordinates as in training")
print("Batch size for inference: 64 (same as training)")
print("✅ Using ALL available Croatia data (no sampling to fixed sequence count)")
print("✅ CORRECTED: Output path uses Croatia_Test directory")

== STEP 1: ENVIRONMENT SETUP & MODEL LOADING FOR CROATIA TEAM EVALUATION ==
Google Drive already mounted

📁 Croatia Data File Paths:
Ball features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/Croatia/Ball_Features/Ball_Normalized_Filtered_Croatia_Team_Only.csv
Players features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/Croatia/Players_Features/Normalized_Ordered_Croatia_Team_Only.csv
Possession features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/Croatia/Possession_Features/Croatia_Team_Only_Sequence_of_5_Possession_Features.csv
Output save path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Croatia_Test

✅ Output directory structure created at: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Basel

   📝 Model summary saved to: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Croatia_Test/model_validation/loaded_model_summary.txt

✅ STEP 1 COMPLETE: Environment setup and model loading finished
Ready for next step: Croatia data loading and validation

📊 Next step will process Croatia team data using identical logic to Argentina task
Missing players handled with (-500, -500) coordinates as in training
Batch size for inference: 64 (same as training)
✅ Using ALL available Croatia data (no sampling to fixed sequence count)
✅ CORRECTED: Output path uses Croatia_Test directory


In [ ]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("\n== STEP 2: CROATIA DATA LOADING AND VALIDATION ==")
start_time = time.time()

# 1. Load Croatia possession features dataset (ALL DATA)
print("\n📊 Loading Croatia possession features dataset...")
croatia_sequence_df = pd.read_csv(
    possession_croatia_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'passerplayerid': 'float32',  # Use float32 to handle NaN values
        'receiverplayerid': 'float32',  # Use float32 to handle NaN values
        'passtype': 'int8',
        'passoutcometype': 'int8',
        'pressuretype': 'int8',
        'sequence_id': 'int32',
        'timestep': 'int8',
        'global_sequence_id': 'int32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'teamname', 'teamattackingdirection', 'passerplayerid', 'receiverplayerid',
             'passtype', 'passoutcometype', 'pressuretype', 'timestep', 'global_sequence_id', 'sequence_id']
)

print(f"   ✅ Croatia possession features loaded: {len(croatia_sequence_df):,} rows, {croatia_sequence_df.shape[1]} columns")

# 2. Load Croatia ball features dataset (ALL DATA)
print("\n⚽ Loading Croatia ball features dataset...")
croatia_ball_df = pd.read_csv(
    ball_croatia_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'ball_x': 'float32',
        'ball_y': 'float32',
        'ball_z': 'float32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'ball_x', 'ball_y', 'ball_z']  # No sequence_id in this file
)

print(f"   ✅ Croatia ball features loaded: {len(croatia_ball_df):,} rows, {croatia_ball_df.shape[1]} columns")

# 3. Load Croatia players features dataset (ALL DATA)
print("\n👥 Loading Croatia players features dataset...")
croatia_players_df = pd.read_csv(
    players_croatia_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'jerseynum': 'int8',
        'playerid': 'int32',
        'positiongrouptype': 'category',
        'x': 'float32',
        'y': 'float32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'jerseynum', 'team', 'visibility', 'confidence', 'x', 'y', 'playerid', 'positiongrouptype']  # No sequence_id in this file
)

print(f"   ✅ Croatia players features loaded: {len(croatia_players_df):,} rows, {croatia_players_df.shape[1]} columns")

# 4. Data validation and basic statistics (identical to Argentina task, using ALL data)
print("\n🔍 Data validation and basic statistics:")

# Create the five join keys for all datasets
print("   🔑 Creating five join keys (gameid, possessioneventid, eventtime, sequence, period)...")
croatia_sequence_df['five_key'] = croatia_sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

croatia_ball_df['five_key'] = croatia_ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

croatia_players_df['five_key'] = croatia_players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

print("   ✅ Five join keys created successfully")

# Check for missing values in critical columns
print("\n   Missing values check:")
critical_columns = ['gameid', 'possessioneventid', 'eventtime', 'sequence', 'period', 'global_sequence_id']
for col in critical_columns:
    if col in croatia_sequence_df.columns:
        missing_count = croatia_sequence_df[col].isna().sum()
        print(f"     Croatia Sequence {col}: {missing_count} missing values")

# CORRECTED: Calculate unique game-sequence combinations for Croatia possessions (using ALL data)
print("\n   🔍 Calculating unique Croatia possessions using (gameid, sequence) composite key...")
croatia_sequence_df['game_sequence_key'] = croatia_sequence_df.apply(lambda row: (row['gameid'], row['sequence']), axis=1)
unique_croatia_game_sequences = croatia_sequence_df['game_sequence_key'].nunique()
unique_croatia_global_sequences = croatia_sequence_df['global_sequence_id'].nunique()
total_croatia_timesteps = len(croatia_sequence_df)

print(f"\n   📊 Croatia dataset summary (ALL available data):")
print(f"     Unique global sequences: {unique_croatia_global_sequences:,} (globally unique 5-timestep sequences)")
print(f"     Unique game-sequence combinations: {unique_croatia_game_sequences:,} (unique Croatia possessions)")
print(f"     Total timesteps: {total_croatia_timesteps:,}")
print(f"     Average timesteps per global sequence: {total_croatia_timesteps/unique_croatia_global_sequences:.1f}")
print(f"     Average timesteps per possession: {total_croatia_timesteps/unique_croatia_game_sequences:.1f}")

# Check global_sequence_id distribution
croatia_global_seq_counts = croatia_sequence_df['global_sequence_id'].value_counts()
min_timesteps = croatia_global_seq_counts.min()
max_timesteps = croatia_global_seq_counts.max()
avg_timesteps = croatia_global_seq_counts.mean()

print(f"\n   🔢 Croatia global sequence distribution:")
print(f"     Min timesteps per global sequence: {min_timesteps}")
print(f"     Max timesteps per global sequence: {max_timesteps}")
print(f"     Avg timesteps per global sequence: {avg_timesteps:.1f}")

# Check for the expected 5 timesteps per global sequence
croatia_expected_sequences = croatia_global_seq_counts[croatia_global_seq_counts == 5].shape[0]
croatia_unexpected_sequences = croatia_global_seq_counts[croatia_global_seq_counts != 5].shape[0]

print(f"\n   ⚠️ Croatia global sequence validation (expecting 5 timesteps per sequence):")
print(f"     Sequences with exactly 5 timesteps: {croatia_expected_sequences:,} ({croatia_expected_sequences/unique_croatia_global_sequences*100:.1f}%)")
print(f"     Sequences with unexpected timestep count: {croatia_unexpected_sequences:,} ({croatia_unexpected_sequences/unique_croatia_global_sequences*100:.1f}%)")

if croatia_unexpected_sequences > 0:
    print("     🚨 WARNING: Some Croatia global sequences don't have exactly 5 timesteps!")
    print("            This may require filtering before inference.")

# Store Croatia datasets for next steps
CROATIA_DATA = {
    'sequence_df': croatia_sequence_df,
    'ball_df': croatia_ball_df,
    'players_df': croatia_players_df
}

total_time = time.time() - start_time
print(f"\n✅ STEP 2 COMPLETE: Croatia data loading and validation finished")
print(f"   ✅ All Croatia datasets loaded successfully (ALL available data used)")
print(f"   ✅ Basic validation completed with CORRECTED sequence counting")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Feature engineering and sequence construction for Croatia data")
print("✅ Using identical logic to Argentina task for feature extraction")
print(f"✅ Processing all {unique_croatia_global_sequences:,} available global sequences (no sampling)")


== STEP 2: CROATIA DATA LOADING AND VALIDATION ==

📊 Loading Croatia possession features dataset...
   ✅ Croatia possession features loaded: 8,470 rows, 16 columns

⚽ Loading Croatia ball features dataset...
   ✅ Croatia ball features loaded: 4,127 rows, 9 columns

👥 Loading Croatia players features dataset...
   ✅ Croatia players features loaded: 96,074 rows, 14 columns

🔍 Data validation and basic statistics:
   🔑 Creating five join keys (gameid, possessioneventid, eventtime, sequence, period)...
   ✅ Five join keys created successfully

   Missing values check:
     Croatia Sequence gameid: 0 missing values
     Croatia Sequence possessioneventid: 0 missing values
     Croatia Sequence eventtime: 0 missing values
     Croatia Sequence sequence: 0 missing values
     Croatia Sequence period: 0 missing values
     Croatia Sequence global_sequence_id: 0 missing values

   🔍 Calculating unique Croatia possessions using (gameid, sequence) composite key...

   📊 Croatia dataset summary (

In [ ]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("\n== STEP 3: FEATURE ENGINEERING AND SEQUENCE CONSTRUCTION FOR CROATIA DATA ==")
start_time = time.time()

# 1. Create lookup dictionaries for faster joins (identical to Argentina task logic)
print("\n🔧 Creating lookup dictionaries for faster data joining...")
start_sub = time.time()

# Create ball lookup dictionary: five_key -> ball features
croatia_ball_lookup = CROATIA_DATA['ball_df'].set_index('five_key')[['ball_x', 'ball_y', 'ball_z']].to_dict('index')

# Create players lookup dictionary: five_key -> player positions
croatia_players_grouped = {}
for key, group in CROATIA_DATA['players_df'].groupby('five_key'):
    croatia_players_grouped[key] = group[['x', 'y', 'playerid', 'positiongrouptype', 'jerseynum', 'team']].to_dict('records')

# Create next timestep lookup for temporal context
# First, sort by global_sequence_id and timestep
croatia_sequence_df_sorted = CROATIA_DATA['sequence_df'].sort_values(['global_sequence_id', 'timestep'])
# Create shifted columns for next timestep within the same global sequence
croatia_sequence_df_sorted['next_timestep'] = croatia_sequence_df_sorted.groupby('global_sequence_id')['timestep'].shift(-1)
croatia_sequence_df_sorted['next_eventtime'] = croatia_sequence_df_sorted.groupby('global_sequence_id')['eventtime'].shift(-1)

# Create lookup for next timestep context
croatia_next_timestep_lookup = {}
for idx, row in croatia_sequence_df_sorted.iterrows():
    if not pd.isna(row['next_timestep']) and row['next_timestep'] == row['timestep'] + 1:
        current_key = (
            row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
        )
        next_key = (
            row['gameid'], row['possessioneventid'], row['next_eventtime'], row['sequence'], row['period']
        )
        croatia_next_timestep_lookup[current_key] = {
            'next_ball_key': next_key,
            'next_passerplayerid': row['passerplayerid'] if not pd.isna(row['passerplayerid']) else -1,
            'next_receiverplayerid': row['receiverplayerid'] if not pd.isna(row['receiverplayerid']) else -1
        }

sub_time = time.time() - start_sub
print(f"   ✅ Lookup dictionaries built in {sub_time:.2f} seconds")

# 2. Get unique global sequences for Croatia data (already validated to have exactly 5 timesteps)
print("\n📊 Getting unique Croatia global sequences...")
unique_croatia_global_sequences = CROATIA_DATA['sequence_df']['global_sequence_id'].unique()
print(f"   📂 Total unique Croatia global sequences: {len(unique_croatia_global_sequences):,}")

# 3. Feature engineering with validation - CORRECTED: Hard check sequence count matching
print("\n⚙️ Engineering features for Croatia sequence of 5...")
start_sub = time.time()

# Initialize storage for Croatia sequences
X_croatia_sequences = []  # Input sequences (4 timesteps × 62 features)
y_croatia_sequences = []  # Target sequences (44 player coordinates for timestep 5)
valid_croatia_global_sequences = []  # Store valid global sequence IDs

# Create progress bar for sequence processing
seq_progress = tqdm(total=len(unique_croatia_global_sequences), desc="Building Croatia sequences", position=0, leave=True)

# Track global sequences that will be processed
processed_croatia_global_sequences = []

for global_seq_id in unique_croatia_global_sequences:
    # Get all timesteps for this global sequence
    seq_data = CROATIA_DATA['sequence_df'][CROATIA_DATA['sequence_df']['global_sequence_id'] == global_seq_id].sort_values('timestep')

    # Validate we have exactly 5 timesteps
    if len(seq_data) != 5:
        seq_progress.update(1)
        continue

    # Prepare input features (timesteps 1-4) and target (timestep 5)
    input_features = []
    has_missing_data = False

    # Process timesteps 1-4 for input
    for timestep in range(1, 5):  # Timesteps 1-4 for input
        row = seq_data[seq_data['timestep'] == timestep].iloc[0]

        # Create the five-key tuple for joining
        key = (row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period'])

        # Get ball features with fallback
        ball_features = croatia_ball_lookup.get(key, {'ball_x': 0.0, 'ball_y': 0.0, 'ball_z': 0.0})

        # Get player positions (44 features) with fallback
        player_positions = croatia_players_grouped.get(key, [])
        if len(player_positions) < 22:
            # Handle missing players by using (-500, -500) as default coordinates
            player_coords = np.zeros(44)
            for i in range(22):
                player_coords[i*2] = -500.0
                player_coords[i*2 + 1] = -500.0
            has_missing_data = True
        else:
            # Extract x,y coordinates for all 22 players in order
            player_coords = np.zeros(44)
            for i, player in enumerate(player_positions[:22]):  # Take first 22 players
                player_coords[i*2] = player['x']
                player_coords[i*2 + 1] = player['y']

        # Get event features (8 features)
        passer_id = row['passerplayerid'] if not pd.isna(row['passerplayerid']) else -1
        receiver_id = row['receiverplayerid'] if not pd.isna(row['receiverplayerid']) else -1

        # Get passer and receiver coordinates with fallback
        passer_coords = (-500.0, -500.0)  # Default for missing
        receiver_coords = (-500.0, -500.0)  # Default for missing

        if len(player_positions) >= 22:
            # Find passer and receiver in the player positions
            for player in player_positions:
                if player['playerid'] == passer_id:
                    passer_coords = (player['x'], player['y'])
                if player['playerid'] == receiver_id:
                    receiver_coords = (player['x'], player['y'])

        event_features = [
            row['passtype'] if not pd.isna(row['passtype']) else 0,
            row['passoutcometype'] if not pd.isna(row['passoutcometype']) else 0,
            row['pressuretype'] if not pd.isna(row['pressuretype']) else 0,
            row['period'],
            passer_coords[0], passer_coords[1],
            receiver_coords[0], receiver_coords[1]
        ]

        # Get next timestep context (7 features) for the next timestep in the sequence
        next_context = [0.0, 0.0, 0.0, -500.0, -500.0, -500.0, -500.0]  # Default values

        if key in croatia_next_timestep_lookup:
            next_info = croatia_next_timestep_lookup[key]
            next_ball_key = next_info['next_ball_key']
            next_ball = croatia_ball_lookup.get(next_ball_key, {'ball_x': 0.0, 'ball_y': 0.0, 'ball_z': 0.0})

            # Get next passer/receiver coordinates
            next_passer_coords = (-500.0, -500.0)
            next_receiver_coords = (-500.0, -500.0)

            if next_ball_key in croatia_players_grouped and len(croatia_players_grouped[next_ball_key]) >= 22:
                next_players = croatia_players_grouped[next_ball_key]
                for player in next_players:
                    if player['playerid'] == next_info['next_passerplayerid']:
                        next_passer_coords = (player['x'], player['y'])
                    if player['playerid'] == next_info['next_receiverplayerid']:
                        next_receiver_coords = (player['x'], player['y'])

            next_context = [
                next_ball['ball_x'], next_ball['ball_y'], next_ball['ball_z'],
                next_passer_coords[0], next_passer_coords[1],
                next_receiver_coords[0], next_receiver_coords[1]
            ]

        # Combine all features (44 + 8 + 3 + 7 = 62 features)
        timestep_features = np.concatenate([
            player_coords,
            event_features,
            [ball_features['ball_x'], ball_features['ball_y'], ball_features['ball_z']],
            next_context
        ])

        input_features.append(timestep_features)

    # Get target (timestep 5 player positions)
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    target_key = (timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
                 timestep5_row['sequence'], timestep5_row['period'])

    target_players = croatia_players_grouped.get(target_key, [])
    if len(target_players) >= 22 and not has_missing_data:  # CORRECTED: Fixed syntax error here
        target_coords = np.zeros(44)
        for i, player in enumerate(target_players[:22]):
            target_coords[i*2] = player['x']
            target_coords[i*2 + 1] = player['y']

        X_croatia_sequences.append(np.array(input_features))  # Shape: (4, 62)
        y_croatia_sequences.append(target_coords)  # Shape: (44,)
        valid_croatia_global_sequences.append(global_seq_id)
        processed_croatia_global_sequences.append(global_seq_id)

    seq_progress.update(1)

seq_progress.close()
sub_time = time.time() - start_sub
print(f"   ✅ Features engineered for {len(X_croatia_sequences):,}/{len(unique_croatia_global_sequences):,} Croatia sequences ({len(X_croatia_sequences)/len(unique_croatia_global_sequences)*100:.1f}%)")
print(f"   ⏱️  Feature engineering time: {sub_time:.2f} seconds")

# 4. Convert to numpy arrays and validate shapes - CORRECTED: Hard validation
print("\n📊 Converting to numpy arrays and validating shapes...")
X_croatia = np.array(X_croatia_sequences)  # Shape: (num_sequences, 4, 62)
y_croatia = np.array(y_croatia_sequences)  # Shape: (num_sequences, 44)

print(f"\n✅ Final Croatia dataset shapes:")
print(f"   Input (X_croatia): {X_croatia.shape} - (sequences, timesteps, features)")
print(f"   Target (y_croatia): {y_croatia.shape} - (sequences, player_coordinates)")
print(f"   Features per timestep: {X_croatia.shape[2]} (should be 62)")
print(f"   Player coordinates: {y_croatia.shape[1]} (should be 44)")

# HARD VALIDATION: Ensure we processed the expected number of sequences
expected_croatia_sequences = 1694  # From Step 2 validation
actual_croatia_sequences = len(X_croatia_sequences)
print(f"\n🔍 HARD SEQUENCE VALIDATION:")
print(f"   Expected global sequences: {expected_croatia_sequences:,}")
print(f"   Actually processed: {actual_croatia_sequences:,}")
print(f"   Processing rate: {actual_croatia_sequences/expected_croatia_sequences*100:.1f}%")

if actual_croatia_sequences < expected_croatia_sequences * 0.95:  # Less than 95% processed
    print("   ⚠️  WARNING: Significant sequence loss during feature engineering!")
    print(f"   Lost {expected_croatia_sequences - actual_croatia_sequences:,} sequences")
    print("   Check for missing player data or other filtering issues")

# Validate feature count
assert X_croatia.shape[2] == 62, f"Expected 62 features per timestep, got {X_croatia.shape[2]}"
assert y_croatia.shape[1] == 44, f"Expected 44 target coordinates, got {y_croatia.shape[1]}"

# Store for next steps
CROATIA_SEQUENCE_DATA = {
    'X': X_croatia,
    'y': y_croatia,
    'valid_global_sequences': valid_croatia_global_sequences,
    'sequence_df': CROATIA_DATA['sequence_df'],
    'processed_global_sequences': processed_croatia_global_sequences
}

total_time = time.time() - start_time
print(f"\n✅ STEP 3 COMPLETE: Croatia feature engineering and sequence construction finished")
print(f"   ✅ Successfully processed {len(X_croatia_sequences):,} valid Croatia sequences")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Model inference and prediction generation for Croatia data")
print("Note: Using batch size 64 for inference (same as training)")
print("✅ Applied identical feature engineering logic as Argentina task")
print("✅ Hard validation ensures sequence count consistency")
print(f"✅ Using ALL {len(X_croatia_sequences):,} available sequences (no sampling)")


== STEP 3: FEATURE ENGINEERING AND SEQUENCE CONSTRUCTION FOR CROATIA DATA ==

🔧 Creating lookup dictionaries for faster data joining...
   ✅ Lookup dictionaries built in 15.36 seconds

📊 Getting unique Croatia global sequences...
   📂 Total unique Croatia global sequences: 1,694

⚙️ Engineering features for Croatia sequence of 5...


Building Croatia sequences: 100%|██████████| 1694/1694 [00:09<00:00, 179.29it/s]

   ✅ Features engineered for 1,694/1,694 Croatia sequences (100.0%)
   ⏱️  Feature engineering time: 9.45 seconds

📊 Converting to numpy arrays and validating shapes...

✅ Final Croatia dataset shapes:
   Input (X_croatia): (1694, 4, 62) - (sequences, timesteps, features)
   Target (y_croatia): (1694, 44) - (sequences, player_coordinates)
   Features per timestep: 62 (should be 62)
   Player coordinates: 44 (should be 44)

🔍 HARD SEQUENCE VALIDATION:
   Expected global sequences: 1,694
   Actually processed: 1,694
   Processing rate: 100.0%

✅ STEP 3 COMPLETE: Croatia feature engineering and sequence construction finished
   ✅ Successfully processed 1,694 valid Croatia sequences
   ⏱️  Total execution time: 24.82 seconds

Next step: Model inference and prediction generation for Croatia data
Note: Using batch size 64 for inference (same as training)
✅ Applied identical feature engineering logic as Argentina task
✅ Hard validation ensures sequence count consistency
✅ Using ALL 1,694 avai

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from tqdm import tqdm
import os
from datetime import datetime
import json

print("\n== STEP 4: MODEL INFERENCE AND PREDICTION GENERATION FOR CROATIA DATA ==")
start_time = time.time()

# 1. Generate predictions for Croatia data using the pre-trained model
print("\n🔮 Generating predictions for Croatia data...")
print(f"   Model input shape: {croatia_model.input_shape}")
print(f"   Croatia data shape: {CROATIA_SEQUENCE_DATA['X'].shape}")
print(f"   Batch size for inference: 64 (same as training)")

croatia_predictions = croatia_model.predict(
    CROATIA_SEQUENCE_DATA['X'],
    batch_size=64,  # Same batch size as training
    verbose=1
)

print(f"   ✅ Predictions generated: {croatia_predictions.shape}")

# 2. Load baseline model metrics from training task for comparison
print("\n📊 Loading baseline model performance metrics...")
baseline_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/training_artifacts/evaluation_metrics.json"

try:
    with open(baseline_metrics_path, 'r') as f:
        baseline_metrics = json.load(f)
    print(f"   ✅ Baseline metrics loaded: MSE={baseline_metrics['test_loss']:.4f}, MAE={baseline_metrics['test_mae']:.4f}, R²=0.7746 (from training output)")
    baseline_r2 = 0.7746  # From training output
except Exception as e:
    print(f"   ⚠️  Error loading baseline metrics: {e}")
    print("   Using hardcoded baseline values from training task")
    baseline_metrics = {
        'test_loss': 82.441124,
        'test_mae': 6.763830,
        'test_rmse': 9.079709
    }
    baseline_r2 = 0.7746

# 3. Create the five join keys for data merging (recreate if needed)
print("\n🔑 Recreating five join keys for Croatia data integrity...")
croatia_sequence_df = CROATIA_DATA['sequence_df']
croatia_ball_df = CROATIA_DATA['ball_df']
croatia_players_df = CROATIA_DATA['players_df']

croatia_sequence_df['five_key'] = croatia_sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

croatia_ball_df['five_key'] = croatia_ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

croatia_players_df['five_key'] = croatia_players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

print("   ✅ Five join keys recreated successfully")

# 4. Get Croatia test sequences and create test files
print("\n📁 Creating Croatia test files with original structure...")

# 4.1 Get processed sequence data
croatia_test_global_ids = CROATIA_SEQUENCE_DATA['processed_global_sequences']
croatia_test_sequence_data = croatia_sequence_df[croatia_sequence_df['global_sequence_id'].isin(croatia_test_global_ids)]
croatia_test_five_keys = croatia_test_sequence_data['five_key'].unique()

# 4.2 Ball features test data
croatia_test_ball_data = croatia_ball_df[croatia_ball_df['five_key'].isin(croatia_test_five_keys)]
ball_croatia_path = os.path.join(output_base_path, "predictions", "ball_features_croatia_test.csv")
os.makedirs(os.path.dirname(ball_croatia_path), exist_ok=True)
croatia_test_ball_data.to_csv(ball_croatia_path, index=False)
print(f"   ⚽ Ball features Croatia test data saved: {len(croatia_test_ball_data)} rows")

# 4.3 Possession features test data
croatia_test_possession_data = croatia_sequence_df[croatia_sequence_df['global_sequence_id'].isin(croatia_test_global_ids)]
possession_croatia_path = os.path.join(output_base_path, "predictions", "possession_features_croatia_test.csv")
os.makedirs(os.path.dirname(possession_croatia_path), exist_ok=True)
croatia_test_possession_data.to_csv(possession_croatia_path, index=False)
print(f"   📋 Possession features Croatia test data saved: {len(croatia_test_possession_data)} rows")

# 4.4 Players test data
croatia_test_players_data = croatia_players_df[croatia_players_df['five_key'].isin(croatia_test_five_keys)]
players_croatia_path = os.path.join(output_base_path, "predictions", "players_croatia_test.csv")
os.makedirs(os.path.dirname(players_croatia_path), exist_ok=True)
croatia_test_players_data.to_csv(players_croatia_path, index=False)
print(f"   👥 Players Croatia test data saved: {len(croatia_test_players_data)} rows")

# 5. Create predicted players CSV with complete structure
print("\n🎯 Creating predicted players CSV with complete structure including sequence column...")

# Create list to store prediction rows
croatia_prediction_rows = []

# Create progress bar
progress = tqdm(total=len(croatia_test_global_ids), desc="Building Croatia prediction CSV", position=0, leave=True)

for i, global_seq_id in enumerate(croatia_test_global_ids):
    # Get sequence data for this global sequence
    seq_data = croatia_sequence_df[croatia_sequence_df['global_sequence_id'] == global_seq_id].sort_values('timestep')

    if len(seq_data) != 5:  # Sequence of 5 has 5 timesteps
        progress.update(1)
        continue

    # Get predicted coordinates for timestep 5
    predicted_coords = croatia_predictions[i]

    # Process each timestep (1-4) for actual data
    for timestep in range(1, 5):  # Timesteps 1-4 for actual data
        timestep_row = seq_data[seq_data['timestep'] == timestep].iloc[0]
        key = (
            timestep_row['gameid'], timestep_row['possessioneventid'], timestep_row['eventtime'],
            timestep_row['sequence'], timestep_row['period']
        )

        # Get player data for this timestep
        players_for_timestep = croatia_players_df[croatia_players_df['five_key'] == key]

        if len(players_for_timestep) < 22:
            continue

        # Add actual player positions (22 players per timestep) with ALL required columns
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row for this player's event
            matching_seq_row = croatia_sequence_df[
                (croatia_sequence_df['gameid'] == player_row['gameid']) &
                (croatia_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (croatia_sequence_df['eventtime'] == player_row['eventtime']) &
                (croatia_sequence_df['sequence'] == player_row['sequence']) &
                (croatia_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not croatia_sequence_df.empty else timestep_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],  # Using eventtime as starttime
                'endtime': matching_seq_row['eventtime'],    # Using eventtime as endtime
                'duration': 0.0,  # Default duration
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': timestep,
                'global_sequence_id': timestep_row['global_sequence_id']
            }
            croatia_prediction_rows.append(row_dict)

    # Add timestep 5 actual data
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    key = (
        timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
        timestep5_row['sequence'], timestep5_row['period']
    )

    players_for_timestep = croatia_players_df[croatia_players_df['five_key'] == key]

    if len(players_for_timestep) >= 22:
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row
            matching_seq_row = croatia_sequence_df[
                (croatia_sequence_df['gameid'] == player_row['gameid']) &
                (croatia_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (croatia_sequence_df['eventtime'] == player_row['eventtime']) &
                (croatia_sequence_df['sequence'] == player_row['sequence']) &
                (croatia_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not croatia_sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            croatia_prediction_rows.append(row_dict)

    # Add timestep 5 predicted data
    if len(players_for_timestep) >= 22:
        for j in range(22):
            player_row = players_for_timestep.iloc[j]
            # Get matching sequence row
            matching_seq_row = croatia_sequence_df[
                (croatia_sequence_df['gameid'] == player_row['gameid']) &
                (croatia_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (croatia_sequence_df['eventtime'] == player_row['eventtime']) &
                (croatia_sequence_df['sequence'] == player_row['sequence']) &
                (croatia_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not croatia_sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': predicted_coords[j*2],
                'y': predicted_coords[j*2 + 1],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 1,
                'data_type': 'predicted',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            croatia_prediction_rows.append(row_dict)

    progress.update(1)

progress.close()

# 6. Create and save prediction DataFrame with ALL required columns
print("\n💾 Saving predicted players CSV with complete column structure...")
croatia_prediction_df = pd.DataFrame(croatia_prediction_rows)

# Define EXACT column order as requested
required_columns = [
    'gameid', 'gameeventid', 'possessioneventid', 'starttime', 'endtime', 'duration', 'eventtime', 'sequence',
    'playerid', 'positiongrouptype', 'jerseynum', 'team', 'x', 'y', 'visibility', 'confidence',
    'possessioneventtype', 'teamattackingdirection', 'period', 'teamname',
    'is_predicted', 'data_type', 'sequence_id', 'timestep', 'global_sequence_id'
]

# Ensure all required columns exist with proper defaults
for col in required_columns:
    if col not in croatia_prediction_df.columns:
        if col in ['gameid', 'gameeventid', 'possessioneventid', 'playerid', 'jerseynum', 'period', 'sequence', 'sequence_id', 'timestep', 'global_sequence_id', 'is_predicted']:
            croatia_prediction_df[col] = 0
        elif col in ['x', 'y', 'starttime', 'endtime', 'duration']:
            croatia_prediction_df[col] = 0.0
        elif col in ['positiongrouptype', 'team', 'visibility', 'confidence', 'possessioneventtype', 'teamattackingdirection', 'teamname', 'data_type']:
            croatia_prediction_df[col] = 'Unknown'
        else:
            croatia_prediction_df[col] = 'missing'

# Reorder columns to EXACT required structure
croatia_prediction_df = croatia_prediction_df[required_columns]

predicted_players_croatia_path = os.path.join(output_base_path, "predictions", "predicted_players_croatia.csv")
os.makedirs(os.path.dirname(predicted_players_croatia_path), exist_ok=True)
croatia_prediction_df.to_csv(predicted_players_croatia_path, index=False)
print(f"   ✅ Predicted players Croatia CSV saved: {len(croatia_prediction_df)} rows")
print(f"      • Actual data rows: {len(croatia_prediction_df[croatia_prediction_df['data_type'] == 'actual'])}")
print(f"      • Predicted data rows: {len(croatia_prediction_df[croatia_prediction_df['data_type'] == 'predicted'])}")
print(f"      • Columns included: {', '.join(croatia_prediction_df.columns)}")

# 7. Calculate comprehensive metrics for Croatia data
print("\n📈 Calculating comprehensive metrics for Croatia data...")

def calculate_metrics(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))
    r2 = r2_score(y_true.flatten(), y_pred.flatten())
    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

croatia_metrics = calculate_metrics(CROATIA_SEQUENCE_DATA['y'], croatia_predictions)

print("\n📊 Croatia Performance Metrics:")
print(f"   Test MSE: {croatia_metrics['mse']:.6f}")
print(f"   Test MAE: {croatia_metrics['mae']:.6f}")
print(f"   Test RMSE: {croatia_metrics['rmse']:.6f}")
print(f"   Test R²: {croatia_metrics['r2']:.6f}")

# 8. Create performance comparison with baseline model (DYNAMIC ANALYSIS)
print("\n📈 Creating performance comparison: Baseline Model vs Croatia Team")

# Create comparison DataFrame
performance_comparison = pd.DataFrame({
    'Model': ['Baseline Model', 'Croatia Team'],
    'MSE': [baseline_metrics['test_loss'], croatia_metrics['mse']],
    'MAE': [baseline_metrics['test_mae'], croatia_metrics['mae']],
    'RMSE': [baseline_metrics['test_rmse'], croatia_metrics['rmse']],
    'R²': [baseline_r2, croatia_metrics['r2']],
    'Dataset Size': [1505, 1694]  # Test set sizes
})

print("\n📊 Performance Comparison Table:")
print(performance_comparison.to_string(index=False))

# Calculate performance differences
mse_diff = croatia_metrics['mse'] - baseline_metrics['test_loss']
mae_diff = croatia_metrics['mae'] - baseline_metrics['test_mae']
r2_diff = croatia_metrics['r2'] - baseline_r2

print(f"\n🔍 Performance Difference Analysis:")
if mse_diff > 0:
    print(f"   MSE degradation: +{mse_diff:.2f} (+{(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)")
elif mse_diff < 0:
    print(f"   MSE improvement: {mse_diff:.2f} ({(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)")
else:
    print(f"   MSE unchanged: 0.00 (0.0%)")

if mae_diff > 0:
    print(f"   MAE degradation: +{mae_diff:.2f} (+{(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)")
elif mae_diff < 0:
    print(f"   MAE improvement: {mae_diff:.2f} ({(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)")
else:
    print(f"   MAE unchanged: 0.00 (0.0%)")

if r2_diff > 0:
    print(f"   R² improvement: +{r2_diff:.4f} (+{(r2_diff/baseline_r2)*100:.1f}%)")
elif r2_diff < 0:
    print(f"   R² degradation: {r2_diff:.4f} ({(r2_diff/baseline_r2)*100:.1f}%)")
else:
    print(f"   R² unchanged: 0.0000 (0.0%)")

# 9. Save metrics and create visualizations
print("\n🎨 Creating analysis visualizations and reports...")

# Save performance metrics
metrics_path = os.path.join(output_base_path, "model_validation", "performance_metrics.json")
os.makedirs(os.path.dirname(metrics_path), exist_ok=True)
with open(metrics_path, 'w') as f:
    json.dump(croatia_metrics, f, indent=2)
print(f"   💾 Performance metrics saved to: {metrics_path}")

# Save comparison table
comparison_csv_path = os.path.join(output_base_path, "model_validation", "performance_comparison.csv")
os.makedirs(os.path.dirname(comparison_csv_path), exist_ok=True)
performance_comparison.to_csv(comparison_csv_path, index=False)
print(f"   💾 Performance comparison table saved to: {comparison_csv_path}")

# Create error analysis visualization
plt.figure(figsize=(15, 6))

# Calculate errors for Croatia data
errors = np.abs(CROATIA_SEQUENCE_DATA['y'] - croatia_predictions)
player_errors = errors.reshape(-1, 22, 2)  # (samples, players, coordinates)
avg_player_errors = np.mean(player_errors, axis=(0, 2))  # Average error per player

plt.subplot(1, 2, 1)
plt.bar(range(1, 23), avg_player_errors, color='skyblue')
plt.title('Average Error per Player Position (Croatia Data)')
plt.xlabel('Player Position (1-22)')
plt.ylabel('MAE')
plt.xticks(range(1, 23), [f'P{i}' for i in range(1, 23)], rotation=45)

plt.subplot(1, 2, 2)
all_errors = errors.flatten()
plt.hist(all_errors, bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
plt.axvline(np.mean(all_errors), color='red', linestyle='dashed', linewidth=2, label=f'Mean: {np.mean(all_errors):.2f}')
plt.title('Error Distribution (Croatia Data)')
plt.xlabel('Absolute Error')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
error_path = os.path.join(output_base_path, "visualizations", "croatia_error_analysis.png")
os.makedirs(os.path.dirname(error_path), exist_ok=True)
plt.savefig(error_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Error analysis visualization saved to: {error_path}")
plt.close()

# Create comparison visualization
plt.figure(figsize=(15, 6))

# Plot 1: MSE and MAE comparison
plt.subplot(1, 2, 1)
metrics_to_plot = ['MSE', 'MAE']
x = np.arange(len(metrics_to_plot))
width = 0.35

baseline_values = [baseline_metrics['test_loss'], baseline_metrics['test_mae']]
croatia_values = [croatia_metrics['mse'], croatia_metrics['mae']]

plt.bar(x - width/2, baseline_values, width, label='Baseline Model', color='blue', alpha=0.7)
plt.bar(x + width/2, croatia_values, width, label='Croatia Team', color='red', alpha=0.7)

plt.xlabel('Metrics')
plt.ylabel('Value')
plt.title('Performance Comparison: Baseline vs Croatia')
plt.xticks(x, metrics_to_plot)
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: R² comparison
plt.subplot(1, 2, 2)
plt.bar(['Baseline Model', 'Croatia Team'], [baseline_r2, croatia_metrics['r2']],
        color=['blue', 'red'], alpha=0.7)
plt.ylim(0, 1)
plt.ylabel('R² Score')
plt.title('R² Comparison')
plt.grid(True, alpha=0.3)

plt.tight_layout()
comparison_plot_path = os.path.join(output_base_path, "visualizations", "performance_comparison.png")
os.makedirs(os.path.dirname(comparison_plot_path), exist_ok=True)
plt.savefig(comparison_plot_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Performance comparison visualization saved to: {comparison_plot_path}")
plt.close()

# 10. Generate pitch visualization with actual vs predicted for Croatia data
plt.figure(figsize=(20, 8))

# Select a few representative Croatia sequences to visualize
num_examples = min(4, len(croatia_test_global_ids))
example_indices = np.random.choice(len(croatia_test_global_ids), num_examples, replace=False)

for idx, example_idx in enumerate(example_indices):
    global_seq_id = croatia_test_global_ids[example_idx]
    actual_coords = CROATIA_SEQUENCE_DATA['y'][example_idx]
    pred_coords = croatia_predictions[example_idx]

    ax = plt.subplot(1, num_examples, idx+1)

    # Create pitch
    ax.set_xlim(-55, 55)
    ax.set_ylim(-35, 35)
    ax.set_aspect('equal')
    ax.set_title(f'Croatia Sequence {global_seq_id}', fontsize=10)

    # Draw pitch markings
    ax.plot([-52.5, 52.5], [-34, -34], 'k-')  # Bottom
    ax.plot([-52.5, 52.5], [34, 34], 'k-')    # Top
    ax.plot([-52.5, -52.5], [-34, 34], 'k-')  # Left
    ax.plot([52.5, 52.5], [-34, 34], 'k-')    # Right
    ax.plot([0, 0], [-34, 34], 'k--')        # Center line

    # Plot actual positions (blue)
    actual_x = actual_coords[::2]
    actual_y = actual_coords[1::2]
    ax.scatter(actual_x[:11], actual_y[:11], c='blue', s=50, alpha=0.7, label='Actual Home')
    ax.scatter(actual_x[11:], actual_y[11:], c='red', s=50, alpha=0.7, label='Actual Away')

    # Plot predicted positions (green)
    pred_x = pred_coords[::2]
    pred_y = pred_coords[1::2]
    ax.scatter(pred_x[:11], pred_y[:11], c='lightgreen', s=50, marker='x', label='Predicted Home')
    ax.scatter(pred_x[11:], pred_y[11:], c='pink', s=50, marker='x', label='Predicted Away')

    # Draw error vectors
    for j in range(22):
        dx = pred_x[j] - actual_x[j]
        dy = pred_y[j] - actual_y[j]
        ax.arrow(actual_x[j], actual_y[j], dx, dy, color='black', alpha=0.5, width=0.1)

    # Turn off axis ticks and labels for cleaner look
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
pitch_path = os.path.join(output_base_path, "visualizations", "croatia_actual_vs_predicted_formations.png")
os.makedirs(os.path.dirname(pitch_path), exist_ok=True)
plt.savefig(pitch_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Pitch visualization saved to: {pitch_path}")
plt.close()

# 11. Generate comprehensive analysis report for Croatia data with DYNAMIC analysis
print("\n📝 Generating comprehensive analysis report for Croatia data...")

report_path = os.path.join(output_base_path, "analysis", f"croatia_team_analysis_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
os.makedirs(os.path.dirname(report_path), exist_ok=True)

with open(report_path, 'w') as f:
    f.write("="*80 + "\n")
    f.write("CROATIA TEAM FORMATION PREDICTION - COMPREHENSIVE ANALYSIS REPORT\n")
    f.write("="*80 + "\n\n")

    f.write(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

    f.write("MODEL INFORMATION:\n")
    f.write("-"*50 + "\n")
    f.write(f"Pre-trained model loaded from: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/model_checkpoints/best_model_epoch_99_val_loss_80.440567.keras\n")
    f.write(f"Model input shape: (None, 4, 62)\n")
    f.write(f"Model output shape: (None, 44)\n")
    f.write(f"Total parameters: 167,404\n")
    f.write(f"Batch size used for inference: 64\n\n")

    f.write("CROATIA DATA SUMMARY:\n")
    f.write("-"*50 + "\n")
    f.write(f"Total unique global sequences: 1,694\n")
    f.write(f"Total timesteps processed: 8,470\n")
    f.write(f"Average timesteps per possession: 26.5\n")
    f.write(f"Sequences successfully processed: 1,694 (100.0%)\n\n")

    f.write("PERFORMANCE COMPARISON: BASELINE MODEL vs CROATIA TEAM\n")
    f.write("-"*50 + "\n")
    f.write("Baseline Model (Test Set Performance):\n")
    f.write(f"   Test MSE: {baseline_metrics['test_loss']:.4f}\n")
    f.write(f"   Test MAE: {baseline_metrics['test_mae']:.4f}\n")
    f.write(f"   Test RMSE: {baseline_metrics['test_rmse']:.4f}\n")
    f.write(f"   Test R²: {baseline_r2:.4f}\n\n")

    f.write("Croatia Team Performance:\n")
    f.write(f"   Test MSE: {croatia_metrics['mse']:.4f}\n")
    f.write(f"   Test MAE: {croatia_metrics['mae']:.4f}\n")
    f.write(f"   Test RMSE: {croatia_metrics['rmse']:.4f}\n")
    f.write(f"   Test R²: {croatia_metrics['r2']:.4f}\n\n")

    # DYNAMIC PERFORMANCE CHANGE ANALYSIS
    performance_change_text = "Performance Change:\n"
    performance_change_text += "-"*50 + "\n"

    # MSE change
    if mse_diff > 0:
        performance_change_text += f"   MSE degradation: +{mse_diff:.2f} (+{(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)\n"
    elif mse_diff < 0:
        performance_change_text += f"   MSE improvement: {mse_diff:.2f} ({(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)\n"
    else:
        performance_change_text += f"   MSE unchanged: 0.00 (0.0%)\n"

    # MAE change
    if mae_diff > 0:
        performance_change_text += f"   MAE degradation: +{mae_diff:.2f} (+{(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)\n"
    elif mae_diff < 0:
        performance_change_text += f"   MAE improvement: {mae_diff:.2f} ({(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)\n"
    else:
        performance_change_text += f"   MAE unchanged: 0.00 (0.0%)\n"

    # R² change
    if r2_diff > 0:
        performance_change_text += f"   R² improvement: +{r2_diff:.4f} (+{(r2_diff/baseline_r2)*100:.1f}%)\n"
    elif r2_diff < 0:
        performance_change_text += f"   R² degradation: {r2_diff:.4f} ({(r2_diff/baseline_r2)*100:.1f}%)\n"
    else:
        performance_change_text += f"   R² unchanged: 0.0000 (0.0%)\n\n"

    f.write(performance_change_text)

    # DYNAMIC KEY INSIGHTS
    insights_text = "Key Insights:\n"
    insights_text += "-"*50 + "\n"

    if mse_diff > 0 and mae_diff > 0 and r2_diff < 0:
        insights_text += "• The model shows reduced performance on Croatia team data compared to the general test set\n"
        insights_text += "• This indicates domain shift - Croatia's tactical patterns differ from the training data distribution\n"
        insights_text += "• Model generalization to specific national teams requires further investigation\n"
    elif mse_diff < 0 and mae_diff < 0 and r2_diff > 0:
        insights_text += "• The model shows improved performance on Croatia team data compared to the general test set\n"
        insights_text += "• This suggests Croatia's tactical patterns align well with the training data distribution\n"
        insights_text += "• The model generalizes effectively to Croatia's playing style\n"
    else:
        insights_text += "• Mixed performance on Croatia team data compared to baseline\n"
        if mse_diff > 0:
            insights_text += "• MSE degradation suggests higher variance in prediction errors\n"
        if mae_diff < 0:
            insights_text += "• MAE improvement indicates more consistent prediction accuracy\n"
        if r2_diff > 0:
            insights_text += "• R² improvement shows better explanation of formation variance\n"
        insights_text += "• Further analysis needed to understand performance patterns\n"

    insights_text += f"• Spatial interpretation: {croatia_metrics['mae']:.2f} MAE = ~{(croatia_metrics['mae'] * 0.68):.2f} meter average error per player on 105m pitch\n\n"
    f.write(insights_text)

    # DYNAMIC CROSS-TEAM PERFORMANCE SUMMARY
    try:
        print("\n🔄 Loading previous team metrics for cross-team comparison...")

        # Define paths to previous team metrics
        france_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/France_Test/model_validation/performance_metrics.json"
        england_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/England_Test/model_validation/performance_metrics.json"
        argentina_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Argentina_Test/model_validation/performance_metrics.json"

        # Load previous team metrics
        france_metrics = {}
        england_metrics = {}
        argentina_metrics = {}
        team_metrics_loaded = 0

        if os.path.exists(france_metrics_path):
            with open(france_metrics_path, 'r') as f_france:
                france_metrics = json.load(f_france)
                team_metrics_loaded += 1
            print(f"   ✅ France metrics loaded from: {france_metrics_path}")
        else:
            print(f"   ⚠️  France metrics file not found: {france_metrics_path}")

        if os.path.exists(england_metrics_path):
            with open(england_metrics_path, 'r') as f_england:
                england_metrics = json.load(f_england)
                team_metrics_loaded += 1
            print(f"   ✅ England metrics loaded from: {england_metrics_path}")
        else:
            print(f"   ⚠️  England metrics file not found: {england_metrics_path}")

        if os.path.exists(argentina_metrics_path):
            with open(argentina_metrics_path, 'r') as f_argentina:
                argentina_metrics = json.load(f_argentina)
                team_metrics_loaded += 1
            print(f"   ✅ Argentina metrics loaded from: {argentina_metrics_path}")
        else:
            print(f"   ⚠️  Argentina metrics file not found: {argentina_metrics_path}")

        # Create cross-team comparison
        f.write("CROSS-TEAM PERFORMANCE SUMMARY:\n")
        f.write("-"*50 + "\n")
        f.write("This Croatia evaluation completes the cross-team analysis with France, England, and Argentina:\n")

        # Add France metrics if available
        if france_metrics:
            f.write(f"   • France Team: {france_metrics['mae']:.2f} MAE, {france_metrics['r2']:.2f} R² (from previous evaluation)\n")
        else:
            f.write("   • France Team: Metrics unavailable (file not found)\n")

        # Add England metrics if available
        if england_metrics:
            f.write(f"   • England Team: {england_metrics['mae']:.2f} MAE, {england_metrics['r2']:.2f} R² (from previous evaluation)\n")
        else:
            f.write("   • England Team: Metrics unavailable (file not found)\n")

        # Add Argentina metrics if available
        if argentina_metrics:
            f.write(f"   • Argentina Team: {argentina_metrics['mae']:.2f} MAE, {argentina_metrics['r2']:.2f} R² (from previous evaluation)\n")
        else:
            f.write("   • Argentina Team: Metrics unavailable (file not found)\n")

        # Add Croatia metrics
        f.write(f"   • Croatia Team: {croatia_metrics['mae']:.2f} MAE, {croatia_metrics['r2']:.2f} R² (current evaluation)\n\n")

        # Calculate and report performance degradation if possible
        degradation_calculated = False
        degradation_values = []

        if france_metrics and england_metrics and argentina_metrics:
            # Calculate performance degradation for each team vs baseline
            france_mae_diff = (france_metrics['mae'] - baseline_metrics['test_mae']) / baseline_metrics['test_mae'] * 100
            england_mae_diff = (england_metrics['mae'] - baseline_metrics['test_mae']) / baseline_metrics['test_mae'] * 100
            argentina_mae_diff = (argentina_metrics['mae'] - baseline_metrics['test_mae']) / baseline_metrics['test_mae'] * 100
            croatia_mae_diff = (croatia_metrics['mae'] - baseline_metrics['test_mae']) / baseline_metrics['test_mae'] * 100

            degradation_values = [france_mae_diff, england_mae_diff, argentina_mae_diff, croatia_mae_diff]
            avg_degradation = sum(degradation_values) / len(degradation_values)
            degradation_calculated = True

        if degradation_calculated:
            f.write(f"Key Finding: Cross-team analysis shows average MAE degradation of {avg_degradation:.1f}% compared to baseline\n")

            # Check if degradation is consistent across teams
            max_degradation = max(degradation_values)
            min_degradation = min(degradation_values)
            degradation_range = max_degradation - min_degradation

            if degradation_range < 5.0:  # Less than 5% variation
                f.write("• Performance degradation is remarkably consistent across all four national teams\n")
                f.write("• This suggests systematic model limitations when generalizing to top-tier national teams\n\n")
            else:
                f.write(f"• Performance degradation varies significantly across teams (range: {degradation_range:.1f}%)\n")
                f.write("• This indicates team-specific tactical patterns that affect model generalization differently\n\n")
        else:
            f.write("Key Finding: Cross-team degradation analysis requires complete metrics from all teams\n")
            f.write("Current analysis shows Croatia performance compared to baseline model\n\n")

    except Exception as e:
        print(f"   ⚠️  Error during cross-team comparison: {str(e)}")
        f.write("CROSS-TEAM PERFORMANCE SUMMARY:\n")
        f.write("-"*50 + "\n")
        f.write("⚠️  Unable to load previous team metrics for comparison\n")
        f.write("   This Croatia evaluation completes the national team analysis, but cross-team comparison unavailable\n")
        f.write(f"   • Croatia Team: {croatia_metrics['mae']:.2f} MAE, {croatia_metrics['r2']:.2f} R² (current evaluation)\n\n")
        f.write(f"Note: Cross-team comparison failed with error: {str(e)}\n\n")

    f.write("EXPORTED FILES:\n")
    f.write("-"*50 + "\n")
    f.write(f"1. Ball Features Croatia Test: {ball_croatia_path}\n")
    f.write(f"   - Rows: {len(croatia_test_ball_data)}\n")
    f.write(f"   - Columns: {', '.join(croatia_test_ball_data.columns)}\n\n")
    f.write(f"2. Possession Features Croatia Test: {possession_croatia_path}\n")
    f.write(f"   - Rows: {len(croatia_test_possession_data)}\n")
    f.write(f"   - Columns: {', '.join(croatia_test_possession_data.columns)}\n\n")
    f.write(f"3. Players Croatia Test: {players_croatia_path}\n")
    f.write(f"   - Rows: {len(croatia_test_players_data)}\n")
    f.write(f"   - Columns: {', '.join(croatia_test_players_data.columns)}\n\n")
    f.write(f"4. Predicted Players Croatia: {predicted_players_croatia_path}\n")
    f.write(f"   - Rows: {len(croatia_prediction_df)}\n")
    f.write(f"   - Columns: {', '.join(croatia_prediction_df.columns)} (25 columns including 'sequence')\n")
    f.write(f"   - Structure: {len(croatia_prediction_df[croatia_prediction_df['data_type'] == 'actual'])} actual rows + {len(croatia_prediction_df[croatia_prediction_df['data_type'] == 'predicted'])} predicted rows\n\n")

    f.write("DATA INTEGRITY GUARANTEES:\n")
    f.write("-"*50 + "\n")
    f.write("• Croatia data processed using identical logic to previous national team tasks\n")
    f.write("• Same five-key join system: (gameid, possessioneventid, eventtime, sequence, period)\n")
    f.write("• Missing players handled with (-500, -500) coordinates\n")
    f.write("• No spatial normalization applied - coordinates used as-is\n")
    f.write("• Temporal context features extracted identically to training\n")
    f.write("• Batch size 64 used for inference (same as training)\n")

print(f"   ✅ Analysis report saved to: {report_path}")

total_time = time.time() - start_time
print(f"\n✅ STEP 4 COMPLETE: Model inference and prediction generation finished")
print(f"   📊 Croatia test performance: MSE={croatia_metrics['mse']:.4f}, MAE={croatia_metrics['mae']:.4f}, R²={croatia_metrics['r2']:.4f}")
print(f"   💾 All Croatia artifacts saved to: {output_base_path}")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\n🎉 🎉 🎉 CROATIA TEAM EVALUATION COMPLETED SUCCESSFULLY! 🎉 🎉 🎉")
print(f"\n📥 FINAL CROATIA EVALUATION ARTIFACTS SAVED TO:")
print(f"   {output_base_path}")
print("\n📊 KEY OUTPUT FILES:")
print(f"   • Predicted Players Croatia: {predicted_players_croatia_path} (25 columns including 'sequence')")
print(f"   • Performance Metrics: {metrics_path}")
print(f"   • Performance Comparison: {comparison_csv_path}")
print(f"   • Error Analysis: {error_path}")
print(f"   • Performance Comparison Visualization: {comparison_plot_path}")
print(f"   • Pitch Visualization: {pitch_path}")
print(f"   • Analysis Report: {report_path}")


== STEP 4: MODEL INFERENCE AND PREDICTION GENERATION FOR CROATIA DATA ==

🔮 Generating predictions for Croatia data...
   Model input shape: (None, 4, 62)
   Croatia data shape: (1694, 4, 62)
   Batch size for inference: 64 (same as training)
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
   ✅ Predictions generated: (1694, 44)

📊 Loading baseline model performance metrics...
   ✅ Baseline metrics loaded: MSE=82.4411, MAE=6.7638, R²=0.7746 (from training output)

🔑 Recreating five join keys for Croatia data integrity...
   ✅ Five join keys recreated successfully

📁 Creating Croatia test files with original structure...
   ⚽ Ball features Croatia test data saved: 2974 rows
   📋 Possession features Croatia test data saved: 8470 rows
   👥 Players Croatia test data saved: 65428 rows

🎯 Creating predicted players CSV with complete structure including sequence column...


Building Croatia prediction CSV: 100%|██████████| 1694/1694 [06:24<00:00,  4.40it/s]



💾 Saving predicted players CSV with complete column structure...
   ✅ Predicted players Croatia CSV saved: 223608 rows
      • Actual data rows: 186340
      • Predicted data rows: 37268
      • Columns included: gameid, gameeventid, possessioneventid, starttime, endtime, duration, eventtime, sequence, playerid, positiongrouptype, jerseynum, team, x, y, visibility, confidence, possessioneventtype, teamattackingdirection, period, teamname, is_predicted, data_type, sequence_id, timestep, global_sequence_id

📈 Calculating comprehensive metrics for Croatia data...

📊 Croatia Performance Metrics:
   Test MSE: 150.205754
   Test MAE: 9.607228
   Test RMSE: 12.255846
   Test R²: 0.554682

📈 Creating performance comparison: Baseline Model vs Croatia Team

📊 Performance Comparison Table:
         Model        MSE      MAE      RMSE       R²  Dataset Size
Baseline Model  82.441124 6.763830  9.079709 0.774600          1505
  Croatia Team 150.205754 9.607228 12.255846 0.554682          1694

🔍 Pe

# **Baseline_Model_Test_on_Morocco_Team_Only**

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive
import matplotlib.pyplot as plt
import json
import time
from datetime import datetime
import logging

print("== STEP 1: ENVIRONMENT SETUP & MODEL LOADING FOR MOROCCO TEAM EVALUATION ==")

# Mount Google Drive
if not os.path.exists('/content/drive'):
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("Google Drive already mounted")

# Define dataset paths for Morocco team
base_path_morocco = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/Morocco"

# Morocco data file paths
ball_morocco_path = os.path.join(base_path_morocco, "Ball_Features/Ball_Normalized_Filtered_Morocco_Team_Only.csv")
players_morocco_path = os.path.join(base_path_morocco, "Players_Features/Normalized_Ordered_Morocco_Team_Only.csv")
possession_morocco_path = os.path.join(base_path_morocco, "Possession_Features/Morocco_Team_Only_Sequence_of_5_Possession_Features.csv")

# Output save path for Morocco test results
output_base_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Morocco_Test"

print("\n📁 Morocco Data File Paths:")
print(f"Ball features path: {ball_morocco_path}")
print(f"Players features path: {players_morocco_path}")
print(f"Possession features path: {possession_morocco_path}")
print(f"Output save path: {output_base_path}")

# Create output directory structure
os.makedirs(output_base_path, exist_ok=True)
os.makedirs(os.path.join(output_base_path, "model_validation"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "predictions"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "visualizations"), exist_ok=True)
os.makedirs(os.path.join(output_base_path, "analysis"), exist_ok=True)
print(f"\n✅ Output directory structure created at: {output_base_path}")

# Check GPU availability
print("\n🔍 GPU Availability Check:")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"  ✅ {len(gpus)} GPU(s) available for inference")
    for i, gpu in enumerate(gpus):
        print(f"     GPU {i}: {gpu}")

    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("  ✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(f"  ❌ Error setting memory growth: {e}")
else:
    print("  ❌ No GPU available, using CPU for inference")

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
print(f"\n🌱 Random seed set to {seed} for reproducibility")

# Load pre-trained model from previous task
print("\n🧠 Loading pre-trained model from baseline model...")
model_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/model_checkpoints/best_model_epoch_99_val_loss_80.440567.keras"

try:
    morocco_model = tf.keras.models.load_model(model_path)
    print(f"   ✅ Model loaded successfully from: {model_path}")

    # Verify model architecture
    print("\n✅ Model architecture verification:")
    print(f"   Input shape: {morocco_model.input_shape}")
    print(f"   Output shape: {morocco_model.output_shape}")
    print(f"   Total parameters: {morocco_model.count_params():,}")

    # Save model summary
    model_summary_path = os.path.join(output_base_path, "model_validation", "loaded_model_summary.txt")
    os.makedirs(os.path.dirname(model_summary_path), exist_ok=True)
    with open(model_summary_path, 'w') as f:
        morocco_model.summary(print_fn=lambda x: f.write(x + '\n'))
    print(f"   📝 Model summary saved to: {model_summary_path}")

except Exception as e:
    print(f"   ❌ Error loading model: {e}")
    raise

# Verify model can handle expected input shape
expected_input_shape = (None, 4, 62)  # batch_size, timesteps, features
if morocco_model.input_shape != expected_input_shape:
    print(f"   ⚠️  WARNING: Model input shape {morocco_model.input_shape} doesn't match expected {expected_input_shape}")
    print("   This may cause errors during inference with Morocco data")

# Verify output shape
expected_output_shape = (None, 44)  # batch_size, player coordinates
if morocco_model.output_shape != expected_output_shape:
    print(f"   ⚠️  WARNING: Model output shape {morocco_model.output_shape} doesn't match expected {expected_output_shape}")

print("\n✅ STEP 1 COMPLETE: Environment setup and model loading finished")
print("Ready for next step: Morocco data loading and validation")
print(f"\n📊 Next step will process Morocco team data using identical logic to Croatia task")
print("Missing players handled with (-500, -500) coordinates as in training")
print("Batch size for inference: 64 (same as training)")
print("✅ Using ALL available Morocco data (no sampling to fixed sequence count)")
print("✅ CORRECTED: Output path uses Morocco_Test directory")

== STEP 1: ENVIRONMENT SETUP & MODEL LOADING FOR MOROCCO TEAM EVALUATION ==
Google Drive already mounted

📁 Morocco Data File Paths:
Ball features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/Morocco/Ball_Features/Ball_Normalized_Filtered_Morocco_Team_Only.csv
Players features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/Morocco/Players_Features/Normalized_Ordered_Morocco_Team_Only.csv
Possession features path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/Morocco/Possession_Features/Morocco_Team_Only_Sequence_of_5_Possession_Features.csv
Output save path: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Morocco_Test

✅ Output directory structure created at: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Basel

   📝 Model summary saved to: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Morocco_Test/model_validation/loaded_model_summary.txt

✅ STEP 1 COMPLETE: Environment setup and model loading finished
Ready for next step: Morocco data loading and validation

📊 Next step will process Morocco team data using identical logic to Croatia task
Missing players handled with (-500, -500) coordinates as in training
Batch size for inference: 64 (same as training)
✅ Using ALL available Morocco data (no sampling to fixed sequence count)
✅ CORRECTED: Output path uses Morocco_Test directory


In [ ]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("\n== STEP 2: MOROCCO DATA LOADING AND VALIDATION ==")
start_time = time.time()

# 1. Load Morocco possession features dataset (ALL DATA)
print("\n📊 Loading Morocco possession features dataset...")
morocco_sequence_df = pd.read_csv(
    possession_morocco_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'passerplayerid': 'float32',  # Use float32 to handle NaN values
        'receiverplayerid': 'float32',  # Use float32 to handle NaN values
        'passtype': 'int8',
        'passoutcometype': 'int8',
        'pressuretype': 'int8',
        'sequence_id': 'int32',
        'timestep': 'int8',
        'global_sequence_id': 'int32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'teamname', 'teamattackingdirection', 'passerplayerid', 'receiverplayerid',
             'passtype', 'passoutcometype', 'pressuretype', 'timestep', 'global_sequence_id', 'sequence_id']
)

print(f"   ✅ Morocco possession features loaded: {len(morocco_sequence_df):,} rows, {morocco_sequence_df.shape[1]} columns")

# 2. Load Morocco ball features dataset (ALL DATA)
print("\n⚽ Loading Morocco ball features dataset...")
morocco_ball_df = pd.read_csv(
    ball_morocco_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'ball_x': 'float32',
        'ball_y': 'float32',
        'ball_z': 'float32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'ball_x', 'ball_y', 'ball_z']  # No sequence_id in this file
)

print(f"   ✅ Morocco ball features loaded: {len(morocco_ball_df):,} rows, {morocco_ball_df.shape[1]} columns")

# 3. Load Morocco players features dataset (ALL DATA)
print("\n👥 Loading Morocco players features dataset...")
morocco_players_df = pd.read_csv(
    players_morocco_path,
    dtype={
        'gameid': 'int32',
        'gameeventid': 'int32',
        'possessioneventid': 'int32',
        'sequence': 'int32',
        'period': 'int8',
        'jerseynum': 'int8',
        'playerid': 'int32',
        'positiongrouptype': 'category',
        'x': 'float32',
        'y': 'float32'
    },
    usecols=['gameid', 'gameeventid', 'possessioneventid', 'eventtime', 'sequence', 'period',
             'jerseynum', 'team', 'visibility', 'confidence', 'x', 'y', 'playerid', 'positiongrouptype']  # No sequence_id in this file
)

print(f"   ✅ Morocco players features loaded: {len(morocco_players_df):,} rows, {morocco_players_df.shape[1]} columns")

# 4. Data validation and basic statistics (identical to Croatia task, using ALL data)
print("\n🔍 Data validation and basic statistics:")

# Create the five join keys for all datasets
print("   🔑 Creating five join keys (gameid, possessioneventid, eventtime, sequence, period)...")
morocco_sequence_df['five_key'] = morocco_sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

morocco_ball_df['five_key'] = morocco_ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

morocco_players_df['five_key'] = morocco_players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

print("   ✅ Five join keys created successfully")

# Check for missing values in critical columns
print("\n   Missing values check:")
critical_columns = ['gameid', 'possessioneventid', 'eventtime', 'sequence', 'period', 'global_sequence_id']
for col in critical_columns:
    if col in morocco_sequence_df.columns:
        missing_count = morocco_sequence_df[col].isna().sum()
        print(f"     Morocco Sequence {col}: {missing_count} missing values")

# CORRECTED: Calculate unique game-sequence combinations for Morocco possessions (using ALL data)
print("\n   🔍 Calculating unique Morocco possessions using (gameid, sequence) composite key...")
morocco_sequence_df['game_sequence_key'] = morocco_sequence_df.apply(lambda row: (row['gameid'], row['sequence']), axis=1)
unique_morocco_game_sequences = morocco_sequence_df['game_sequence_key'].nunique()
unique_morocco_global_sequences = morocco_sequence_df['global_sequence_id'].nunique()
total_morocco_timesteps = len(morocco_sequence_df)

print(f"\n   📊 Morocco dataset summary (ALL available data):")
print(f"     Unique global sequences: {unique_morocco_global_sequences:,} (globally unique 5-timestep sequences)")
print(f"     Unique game-sequence combinations: {unique_morocco_game_sequences:,} (unique Morocco possessions)")
print(f"     Total timesteps: {total_morocco_timesteps:,}")
print(f"     Average timesteps per global sequence: {total_morocco_timesteps/unique_morocco_global_sequences:.1f}")
print(f"     Average timesteps per possession: {total_morocco_timesteps/unique_morocco_game_sequences:.1f}")

# Check global_sequence_id distribution
morocco_global_seq_counts = morocco_sequence_df['global_sequence_id'].value_counts()
min_timesteps = morocco_global_seq_counts.min()
max_timesteps = morocco_global_seq_counts.max()
avg_timesteps = morocco_global_seq_counts.mean()

print(f"\n   🔢 Morocco global sequence distribution:")
print(f"     Min timesteps per global sequence: {min_timesteps}")
print(f"     Max timesteps per global sequence: {max_timesteps}")
print(f"     Avg timesteps per global sequence: {avg_timesteps:.1f}")

# Check for the expected 5 timesteps per global sequence
morocco_expected_sequences = morocco_global_seq_counts[morocco_global_seq_counts == 5].shape[0]
morocco_unexpected_sequences = morocco_global_seq_counts[morocco_global_seq_counts != 5].shape[0]

print(f"\n   ⚠️ Morocco global sequence validation (expecting 5 timesteps per sequence):")
print(f"     Sequences with exactly 5 timesteps: {morocco_expected_sequences:,} ({morocco_expected_sequences/unique_morocco_global_sequences*100:.1f}%)")
print(f"     Sequences with unexpected timestep count: {morocco_unexpected_sequences:,} ({morocco_unexpected_sequences/unique_morocco_global_sequences*100:.1f}%)")

if morocco_unexpected_sequences > 0:
    print("     🚨 WARNING: Some Morocco global sequences don't have exactly 5 timesteps!")
    print("            This may require filtering before inference.")

# Store Morocco datasets for next steps
MOROCCO_DATA = {
    'sequence_df': morocco_sequence_df,
    'ball_df': morocco_ball_df,
    'players_df': morocco_players_df
}

total_time = time.time() - start_time
print(f"\n✅ STEP 2 COMPLETE: Morocco data loading and validation finished")
print(f"   ✅ All Morocco datasets loaded successfully (ALL available data used)")
print(f"   ✅ Basic validation completed with CORRECTED sequence counting")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Feature engineering and sequence construction for Morocco data")
print("✅ Using identical logic to Croatia task for feature extraction")
print(f"✅ Processing all {unique_morocco_global_sequences:,} available global sequences (no sampling)")


== STEP 2: MOROCCO DATA LOADING AND VALIDATION ==

📊 Loading Morocco possession features dataset...
   ✅ Morocco possession features loaded: 4,265 rows, 16 columns

⚽ Loading Morocco ball features dataset...
   ✅ Morocco ball features loaded: 2,623 rows, 9 columns

👥 Loading Morocco players features dataset...
   ✅ Morocco players features loaded: 61,748 rows, 14 columns

🔍 Data validation and basic statistics:
   🔑 Creating five join keys (gameid, possessioneventid, eventtime, sequence, period)...
   ✅ Five join keys created successfully

   Missing values check:
     Morocco Sequence gameid: 0 missing values
     Morocco Sequence possessioneventid: 0 missing values
     Morocco Sequence eventtime: 0 missing values
     Morocco Sequence sequence: 0 missing values
     Morocco Sequence period: 0 missing values
     Morocco Sequence global_sequence_id: 0 missing values

   🔍 Calculating unique Morocco possessions using (gameid, sequence) composite key...

   📊 Morocco dataset summary (

In [ ]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("\n== STEP 3: FEATURE ENGINEERING AND SEQUENCE CONSTRUCTION FOR MOROCCO DATA ==")
start_time = time.time()

# 1. Create lookup dictionaries for faster joins (identical to Croatia task logic)
print("\n🔧 Creating lookup dictionaries for faster data joining...")
start_sub = time.time()

# Create ball lookup dictionary: five_key -> ball features
morocco_ball_lookup = MOROCCO_DATA['ball_df'].set_index('five_key')[['ball_x', 'ball_y', 'ball_z']].to_dict('index')

# Create players lookup dictionary: five_key -> player positions
morocco_players_grouped = {}
for key, group in MOROCCO_DATA['players_df'].groupby('five_key'):
    morocco_players_grouped[key] = group[['x', 'y', 'playerid', 'positiongrouptype', 'jerseynum', 'team']].to_dict('records')

# Create next timestep lookup for temporal context
# First, sort by global_sequence_id and timestep
morocco_sequence_df_sorted = MOROCCO_DATA['sequence_df'].sort_values(['global_sequence_id', 'timestep'])
# Create shifted columns for next timestep within the same global sequence
morocco_sequence_df_sorted['next_timestep'] = morocco_sequence_df_sorted.groupby('global_sequence_id')['timestep'].shift(-1)
morocco_sequence_df_sorted['next_eventtime'] = morocco_sequence_df_sorted.groupby('global_sequence_id')['eventtime'].shift(-1)

# Create lookup for next timestep context
morocco_next_timestep_lookup = {}
for idx, row in morocco_sequence_df_sorted.iterrows():
    if not pd.isna(row['next_timestep']) and row['next_timestep'] == row['timestep'] + 1:
        current_key = (
            row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
        )
        next_key = (
            row['gameid'], row['possessioneventid'], row['next_eventtime'], row['sequence'], row['period']
        )
        morocco_next_timestep_lookup[current_key] = {
            'next_ball_key': next_key,
            'next_passerplayerid': row['passerplayerid'] if not pd.isna(row['passerplayerid']) else -1,
            'next_receiverplayerid': row['receiverplayerid'] if not pd.isna(row['receiverplayerid']) else -1
        }

sub_time = time.time() - start_sub
print(f"   ✅ Lookup dictionaries built in {sub_time:.2f} seconds")

# 2. Get unique global sequences for Morocco data (already validated to have exactly 5 timesteps)
print("\n📊 Getting unique Morocco global sequences...")
unique_morocco_global_sequences = MOROCCO_DATA['sequence_df']['global_sequence_id'].unique()
print(f"   📂 Total unique Morocco global sequences: {len(unique_morocco_global_sequences):,}")

# 3. Feature engineering with validation - CORRECTED: Hard check sequence count matching
print("\n⚙️ Engineering features for Morocco sequence of 5...")
start_sub = time.time()

# Initialize storage for Morocco sequences
X_morocco_sequences = []  # Input sequences (4 timesteps × 62 features)
y_morocco_sequences = []  # Target sequences (44 player coordinates for timestep 5)
valid_morocco_global_sequences = []  # Store valid global sequence IDs

# Create progress bar for sequence processing
seq_progress = tqdm(total=len(unique_morocco_global_sequences), desc="Building Morocco sequences", position=0, leave=True)

# Track global sequences that will be processed
processed_morocco_global_sequences = []

for global_seq_id in unique_morocco_global_sequences:
    # Get all timesteps for this global sequence
    seq_data = MOROCCO_DATA['sequence_df'][MOROCCO_DATA['sequence_df']['global_sequence_id'] == global_seq_id].sort_values('timestep')

    # Validate we have exactly 5 timesteps
    if len(seq_data) != 5:
        seq_progress.update(1)
        continue

    # Prepare input features (timesteps 1-4) and target (timestep 5)
    input_features = []
    has_missing_data = False

    # Process timesteps 1-4 for input
    for timestep in range(1, 5):  # Timesteps 1-4 for input
        row = seq_data[seq_data['timestep'] == timestep].iloc[0]

        # Create the five-key tuple for joining
        key = (row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period'])

        # Get ball features with fallback
        ball_features = morocco_ball_lookup.get(key, {'ball_x': 0.0, 'ball_y': 0.0, 'ball_z': 0.0})

        # Get player positions (44 features) with fallback
        player_positions = morocco_players_grouped.get(key, [])
        if len(player_positions) < 22:
            # Handle missing players by using (-500, -500) as default coordinates
            player_coords = np.zeros(44)
            for i in range(22):
                player_coords[i*2] = -500.0
                player_coords[i*2 + 1] = -500.0
            has_missing_data = True
        else:
            # Extract x,y coordinates for all 22 players in order
            player_coords = np.zeros(44)
            for i, player in enumerate(player_positions[:22]):  # Take first 22 players
                player_coords[i*2] = player['x']
                player_coords[i*2 + 1] = player['y']

        # Get event features (8 features)
        passer_id = row['passerplayerid'] if not pd.isna(row['passerplayerid']) else -1
        receiver_id = row['receiverplayerid'] if not pd.isna(row['receiverplayerid']) else -1

        # Get passer and receiver coordinates with fallback
        passer_coords = (-500.0, -500.0)  # Default for missing
        receiver_coords = (-500.0, -500.0)  # Default for missing

        if len(player_positions) >= 22:
            # Find passer and receiver in the player positions
            for player in player_positions:
                if player['playerid'] == passer_id:
                    passer_coords = (player['x'], player['y'])
                if player['playerid'] == receiver_id:
                    receiver_coords = (player['x'], player['y'])

        event_features = [
            row['passtype'] if not pd.isna(row['passtype']) else 0,
            row['passoutcometype'] if not pd.isna(row['passoutcometype']) else 0,
            row['pressuretype'] if not pd.isna(row['pressuretype']) else 0,
            row['period'],
            passer_coords[0], passer_coords[1],
            receiver_coords[0], receiver_coords[1]
        ]

        # Get next timestep context (7 features) for the next timestep in the sequence
        next_context = [0.0, 0.0, 0.0, -500.0, -500.0, -500.0, -500.0]  # Default values

        if key in morocco_next_timestep_lookup:
            next_info = morocco_next_timestep_lookup[key]
            next_ball_key = next_info['next_ball_key']
            next_ball = morocco_ball_lookup.get(next_ball_key, {'ball_x': 0.0, 'ball_y': 0.0, 'ball_z': 0.0})

            # Get next passer/receiver coordinates
            next_passer_coords = (-500.0, -500.0)
            next_receiver_coords = (-500.0, -500.0)

            if next_ball_key in morocco_players_grouped and len(morocco_players_grouped[next_ball_key]) >= 22:
                next_players = morocco_players_grouped[next_ball_key]
                for player in next_players:
                    if player['playerid'] == next_info['next_passerplayerid']:
                        next_passer_coords = (player['x'], player['y'])
                    if player['playerid'] == next_info['next_receiverplayerid']:
                        next_receiver_coords = (player['x'], player['y'])

            next_context = [
                next_ball['ball_x'], next_ball['ball_y'], next_ball['ball_z'],
                next_passer_coords[0], next_passer_coords[1],
                next_receiver_coords[0], next_receiver_coords[1]
            ]

        # Combine all features (44 + 8 + 3 + 7 = 62 features)
        timestep_features = np.concatenate([
            player_coords,
            event_features,
            [ball_features['ball_x'], ball_features['ball_y'], ball_features['ball_z']],
            next_context
        ])

        input_features.append(timestep_features)

    # Get target (timestep 5 player positions)
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    target_key = (timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
                 timestep5_row['sequence'], timestep5_row['period'])

    target_players = morocco_players_grouped.get(target_key, [])
    if len(target_players) >= 22 and not has_missing_data:
        target_coords = np.zeros(44)
        for i, player in enumerate(target_players[:22]):
            target_coords[i*2] = player['x']
            target_coords[i*2 + 1] = player['y']

        X_morocco_sequences.append(np.array(input_features))  # Shape: (4, 62)
        y_morocco_sequences.append(target_coords)  # Shape: (44,)
        valid_morocco_global_sequences.append(global_seq_id)
        processed_morocco_global_sequences.append(global_seq_id)

    seq_progress.update(1)

seq_progress.close()
sub_time = time.time() - start_sub
print(f"   ✅ Features engineered for {len(X_morocco_sequences):,}/{len(unique_morocco_global_sequences):,} Morocco sequences ({len(X_morocco_sequences)/len(unique_morocco_global_sequences)*100:.1f}%)")
print(f"   ⏱️  Feature engineering time: {sub_time:.2f} seconds")

# 4. Convert to numpy arrays and validate shapes - CORRECTED: Hard validation
print("\n📊 Converting to numpy arrays and validating shapes...")
X_morocco = np.array(X_morocco_sequences)  # Shape: (num_sequences, 4, 62)
y_morocco = np.array(y_morocco_sequences)  # Shape: (num_sequences, 44)

print(f"\n✅ Final Morocco dataset shapes:")
print(f"   Input (X_morocco): {X_morocco.shape} - (sequences, timesteps, features)")
print(f"   Target (y_morocco): {y_morocco.shape} - (sequences, player_coordinates)")
print(f"   Features per timestep: {X_morocco.shape[2]} (should be 62)")
print(f"   Player coordinates: {y_morocco.shape[1]} (should be 44)")

# HARD VALIDATION: Ensure we processed the expected number of sequences
expected_morocco_sequences = 853  # From Step 2 validation
actual_morocco_sequences = len(X_morocco_sequences)
print(f"\n🔍 HARD SEQUENCE VALIDATION:")
print(f"   Expected global sequences: {expected_morocco_sequences:,}")
print(f"   Actually processed: {actual_morocco_sequences:,}")
print(f"   Processing rate: {actual_morocco_sequences/expected_morocco_sequences*100:.1f}%")

if actual_morocco_sequences < expected_morocco_sequences * 0.95:  # Less than 95% processed
    print("   ⚠️  WARNING: Significant sequence loss during feature engineering!")
    print(f"   Lost {expected_morocco_sequences - actual_morocco_sequences:,} sequences")
    print("   Check for missing player data or other filtering issues")

# Validate feature count
assert X_morocco.shape[2] == 62, f"Expected 62 features per timestep, got {X_morocco.shape[2]}"
assert y_morocco.shape[1] == 44, f"Expected 44 target coordinates, got {y_morocco.shape[1]}"

# Store for next steps
MOROCCO_SEQUENCE_DATA = {
    'X': X_morocco,
    'y': y_morocco,
    'valid_global_sequences': valid_morocco_global_sequences,
    'sequence_df': MOROCCO_DATA['sequence_df'],
    'processed_global_sequences': processed_morocco_global_sequences
}

total_time = time.time() - start_time
print(f"\n✅ STEP 3 COMPLETE: Morocco feature engineering and sequence construction finished")
print(f"   ✅ Successfully processed {len(X_morocco_sequences):,} valid Morocco sequences")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\nNext step: Model inference and prediction generation for Morocco data")
print("Note: Using batch size 64 for inference (same as training)")
print("✅ Applied identical feature engineering logic as Croatia task")
print("✅ Hard validation ensures sequence count consistency")
print(f"✅ Using ALL {len(X_morocco_sequences):,} available sequences (no sampling)")


== STEP 3: FEATURE ENGINEERING AND SEQUENCE CONSTRUCTION FOR MOROCCO DATA ==

🔧 Creating lookup dictionaries for faster data joining...
   ✅ Lookup dictionaries built in 3.81 seconds

📊 Getting unique Morocco global sequences...
   📂 Total unique Morocco global sequences: 853

⚙️ Engineering features for Morocco sequence of 5...


Building Morocco sequences: 100%|██████████| 853/853 [00:07<00:00, 108.16it/s]

   ✅ Features engineered for 853/853 Morocco sequences (100.0%)
   ⏱️  Feature engineering time: 7.89 seconds

📊 Converting to numpy arrays and validating shapes...

✅ Final Morocco dataset shapes:
   Input (X_morocco): (853, 4, 62) - (sequences, timesteps, features)
   Target (y_morocco): (853, 44) - (sequences, player_coordinates)
   Features per timestep: 62 (should be 62)
   Player coordinates: 44 (should be 44)

🔍 HARD SEQUENCE VALIDATION:
   Expected global sequences: 853
   Actually processed: 853
   Processing rate: 100.0%

✅ STEP 3 COMPLETE: Morocco feature engineering and sequence construction finished
   ✅ Successfully processed 853 valid Morocco sequences
   ⏱️  Total execution time: 11.71 seconds

Next step: Model inference and prediction generation for Morocco data
Note: Using batch size 64 for inference (same as training)
✅ Applied identical feature engineering logic as Croatia task
✅ Hard validation ensures sequence count consistency
✅ Using ALL 853 available sequences 

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from tqdm import tqdm
import os
from datetime import datetime
import json

print("\n== STEP 4: MODEL INFERENCE AND PREDICTION GENERATION FOR MOROCCO DATA ==")
start_time = time.time()

# 1. Generate predictions for Morocco data using the pre-trained model
print("\n🔮 Generating predictions for Morocco data...")
print(f"   Model input shape: {morocco_model.input_shape}")
print(f"   Morocco data shape: {MOROCCO_SEQUENCE_DATA['X'].shape}")
print(f"   Batch size for inference: 64 (same as training)")

morocco_predictions = morocco_model.predict(
    MOROCCO_SEQUENCE_DATA['X'],
    batch_size=64,  # Same batch size as training
    verbose=1
)

print(f"   ✅ Predictions generated: {morocco_predictions.shape}")

# 2. Load baseline model metrics from training task for comparison
print("\n📊 Loading baseline model performance metrics...")
baseline_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/training_artifacts/evaluation_metrics.json"

try:
    with open(baseline_metrics_path, 'r') as f:
        baseline_metrics = json.load(f)
    print(f"   ✅ Baseline metrics loaded: MSE={baseline_metrics['test_loss']:.4f}, MAE={baseline_metrics['test_mae']:.4f}, R²=0.7746 (from training output)")
    baseline_r2 = 0.7746  # From training output
except Exception as e:
    print(f"   ⚠️  Error loading baseline metrics: {e}")
    print("   Using hardcoded baseline values from training task")
    baseline_metrics = {
        'test_loss': 82.441124,
        'test_mae': 6.763830,
        'test_rmse': 9.079709
    }
    baseline_r2 = 0.7746

# 3. Create the five join keys for data merging (recreate if needed)
print("\n🔑 Recreating five join keys for Morocco data integrity...")
morocco_sequence_df = MOROCCO_DATA['sequence_df']
morocco_ball_df = MOROCCO_DATA['ball_df']
morocco_players_df = MOROCCO_DATA['players_df']

morocco_sequence_df['five_key'] = morocco_sequence_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

morocco_ball_df['five_key'] = morocco_ball_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

morocco_players_df['five_key'] = morocco_players_df.apply(lambda row: (
    row['gameid'], row['possessioneventid'], row['eventtime'], row['sequence'], row['period']
), axis=1)

print("   ✅ Five join keys recreated successfully")

# 4. Get Morocco test sequences and create test files
print("\n📁 Creating Morocco test files with original structure...")

# 4.1 Get processed sequence data
morocco_test_global_ids = MOROCCO_SEQUENCE_DATA['processed_global_sequences']
morocco_test_sequence_data = morocco_sequence_df[morocco_sequence_df['global_sequence_id'].isin(morocco_test_global_ids)]
morocco_test_five_keys = morocco_test_sequence_data['five_key'].unique()

# 4.2 Ball features test data
morocco_test_ball_data = morocco_ball_df[morocco_ball_df['five_key'].isin(morocco_test_five_keys)]
ball_morocco_path = os.path.join(output_base_path, "predictions", "ball_features_morocco_test.csv")
os.makedirs(os.path.dirname(ball_morocco_path), exist_ok=True)
morocco_test_ball_data.to_csv(ball_morocco_path, index=False)
print(f"   ⚽ Ball features Morocco test data saved: {len(morocco_test_ball_data)} rows")

# 4.3 Possession features test data
morocco_test_possession_data = morocco_sequence_df[morocco_sequence_df['global_sequence_id'].isin(morocco_test_global_ids)]
possession_morocco_path = os.path.join(output_base_path, "predictions", "possession_features_morocco_test.csv")
os.makedirs(os.path.dirname(possession_morocco_path), exist_ok=True)
morocco_test_possession_data.to_csv(possession_morocco_path, index=False)
print(f"   📋 Possession features Morocco test data saved: {len(morocco_test_possession_data)} rows")

# 4.4 Players test data
morocco_test_players_data = morocco_players_df[morocco_players_df['five_key'].isin(morocco_test_five_keys)]
players_morocco_path = os.path.join(output_base_path, "predictions", "players_morocco_test.csv")
os.makedirs(os.path.dirname(players_morocco_path), exist_ok=True)
morocco_test_players_data.to_csv(players_morocco_path, index=False)
print(f"   👥 Players Morocco test data saved: {len(morocco_test_players_data)} rows")

# 5. Create predicted players CSV with complete structure
print("\n🎯 Creating predicted players CSV with complete structure including sequence column...")

# Create list to store prediction rows
morocco_prediction_rows = []

# Create progress bar
progress = tqdm(total=len(morocco_test_global_ids), desc="Building Morocco prediction CSV", position=0, leave=True)

for i, global_seq_id in enumerate(morocco_test_global_ids):
    # Get sequence data for this global sequence
    seq_data = morocco_sequence_df[morocco_sequence_df['global_sequence_id'] == global_seq_id].sort_values('timestep')

    if len(seq_data) != 5:  # Sequence of 5 has 5 timesteps
        progress.update(1)
        continue

    # Get predicted coordinates for timestep 5
    predicted_coords = morocco_predictions[i]

    # Process each timestep (1-4) for actual data
    for timestep in range(1, 5):  # Timesteps 1-4 for actual data
        timestep_row = seq_data[seq_data['timestep'] == timestep].iloc[0]
        key = (
            timestep_row['gameid'], timestep_row['possessioneventid'], timestep_row['eventtime'],
            timestep_row['sequence'], timestep_row['period']
        )

        # Get player data for this timestep
        players_for_timestep = morocco_players_df[morocco_players_df['five_key'] == key]

        if len(players_for_timestep) < 22:
            continue

        # Add actual player positions (22 players per timestep) with ALL required columns
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row for this player's event
            matching_seq_row = morocco_sequence_df[
                (morocco_sequence_df['gameid'] == player_row['gameid']) &
                (morocco_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (morocco_sequence_df['eventtime'] == player_row['eventtime']) &
                (morocco_sequence_df['sequence'] == player_row['sequence']) &
                (morocco_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not morocco_sequence_df.empty else timestep_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],  # Using eventtime as starttime
                'endtime': matching_seq_row['eventtime'],    # Using eventtime as endtime
                'duration': 0.0,  # Default duration
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': timestep,
                'global_sequence_id': timestep_row['global_sequence_id']
            }
            morocco_prediction_rows.append(row_dict)

    # Add timestep 5 actual data
    timestep5_row = seq_data[seq_data['timestep'] == 5].iloc[0]
    key = (
        timestep5_row['gameid'], timestep5_row['possessioneventid'], timestep5_row['eventtime'],
        timestep5_row['sequence'], timestep5_row['period']
    )

    players_for_timestep = morocco_players_df[morocco_players_df['five_key'] == key]

    if len(players_for_timestep) >= 22:
        for _, player_row in players_for_timestep.head(22).iterrows():
            # Get matching sequence row
            matching_seq_row = morocco_sequence_df[
                (morocco_sequence_df['gameid'] == player_row['gameid']) &
                (morocco_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (morocco_sequence_df['eventtime'] == player_row['eventtime']) &
                (morocco_sequence_df['sequence'] == player_row['sequence']) &
                (morocco_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not morocco_sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': player_row['x'],
                'y': player_row['y'],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 0,
                'data_type': 'actual',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            morocco_prediction_rows.append(row_dict)

    # Add timestep 5 predicted data
    if len(players_for_timestep) >= 22:
        for j in range(22):
            player_row = players_for_timestep.iloc[j]
            # Get matching sequence row
            matching_seq_row = morocco_sequence_df[
                (morocco_sequence_df['gameid'] == player_row['gameid']) &
                (morocco_sequence_df['possessioneventid'] == player_row['possessioneventid']) &
                (morocco_sequence_df['eventtime'] == player_row['eventtime']) &
                (morocco_sequence_df['sequence'] == player_row['sequence']) &
                (morocco_sequence_df['period'] == player_row['period'])
            ].iloc[0] if not morocco_sequence_df.empty else timestep5_row

            row_dict = {
                'gameid': player_row['gameid'],
                'gameeventid': matching_seq_row['gameeventid'],
                'possessioneventid': matching_seq_row['possessioneventid'],
                'starttime': matching_seq_row['eventtime'],
                'endtime': matching_seq_row['eventtime'],
                'duration': 0.0,
                'eventtime': matching_seq_row['eventtime'],
                'sequence': matching_seq_row['sequence'],
                'playerid': player_row['playerid'],
                'positiongrouptype': player_row['positiongrouptype'],
                'jerseynum': player_row['jerseynum'],
                'team': player_row['team'],
                'x': predicted_coords[j*2],
                'y': predicted_coords[j*2 + 1],
                'visibility': player_row['visibility'],
                'confidence': player_row['confidence'],
                'possessioneventtype': matching_seq_row.get('possessioneventtype', 'PA'),
                'teamattackingdirection': matching_seq_row.get('teamattackingdirection', 'R'),
                'period': matching_seq_row['period'],
                'teamname': matching_seq_row.get('teamname', 'Unknown'),
                'is_predicted': 1,
                'data_type': 'predicted',
                'sequence_id': matching_seq_row['sequence_id'],
                'timestep': 5,
                'global_sequence_id': timestep5_row['global_sequence_id']
            }
            morocco_prediction_rows.append(row_dict)

    progress.update(1)

progress.close()

# 6. Create and save prediction DataFrame with ALL required columns
print("\n💾 Saving predicted players CSV with complete column structure...")
morocco_prediction_df = pd.DataFrame(morocco_prediction_rows)

# Define EXACT column order as requested
required_columns = [
    'gameid', 'gameeventid', 'possessioneventid', 'starttime', 'endtime', 'duration', 'eventtime', 'sequence',
    'playerid', 'positiongrouptype', 'jerseynum', 'team', 'x', 'y', 'visibility', 'confidence',
    'possessioneventtype', 'teamattackingdirection', 'period', 'teamname',
    'is_predicted', 'data_type', 'sequence_id', 'timestep', 'global_sequence_id'
]

# Ensure all required columns exist with proper defaults
for col in required_columns:
    if col not in morocco_prediction_df.columns:
        if col in ['gameid', 'gameeventid', 'possessioneventid', 'playerid', 'jerseynum', 'period', 'sequence', 'sequence_id', 'timestep', 'global_sequence_id', 'is_predicted']:
            morocco_prediction_df[col] = 0
        elif col in ['x', 'y', 'starttime', 'endtime', 'duration']:
            morocco_prediction_df[col] = 0.0
        elif col in ['positiongrouptype', 'team', 'visibility', 'confidence', 'possessioneventtype', 'teamattackingdirection', 'teamname', 'data_type']:
            morocco_prediction_df[col] = 'Unknown'
        else:
            morocco_prediction_df[col] = 'missing'

# Reorder columns to EXACT required structure
morocco_prediction_df = morocco_prediction_df[required_columns]

predicted_players_morocco_path = os.path.join(output_base_path, "predictions", "predicted_players_morocco.csv")
os.makedirs(os.path.dirname(predicted_players_morocco_path), exist_ok=True)
morocco_prediction_df.to_csv(predicted_players_morocco_path, index=False)
print(f"   ✅ Predicted players Morocco CSV saved: {len(morocco_prediction_df)} rows")
print(f"      • Actual data rows: {len(morocco_prediction_df[morocco_prediction_df['data_type'] == 'actual'])}")
print(f"      • Predicted data rows: {len(morocco_prediction_df[morocco_prediction_df['data_type'] == 'predicted'])}")
print(f"      • Columns included: {', '.join(morocco_prediction_df.columns)}")

# 7. Calculate comprehensive metrics for Morocco data
print("\n📈 Calculating comprehensive metrics for Morocco data...")

def calculate_metrics(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))
    r2 = r2_score(y_true.flatten(), y_pred.flatten())
    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

morocco_metrics = calculate_metrics(MOROCCO_SEQUENCE_DATA['y'], morocco_predictions)

print("\n📊 Morocco Performance Metrics:")
print(f"   Test MSE: {morocco_metrics['mse']:.6f}")
print(f"   Test MAE: {morocco_metrics['mae']:.6f}")
print(f"   Test RMSE: {morocco_metrics['rmse']:.6f}")
print(f"   Test R²: {morocco_metrics['r2']:.6f}")

# 8. Create performance comparison with baseline model (DYNAMIC ANALYSIS)
print("\n📈 Creating performance comparison: Baseline Model vs Morocco Team")

# Create comparison DataFrame
performance_comparison = pd.DataFrame({
    'Model': ['Baseline Model', 'Morocco Team'],
    'MSE': [baseline_metrics['test_loss'], morocco_metrics['mse']],
    'MAE': [baseline_metrics['test_mae'], morocco_metrics['mae']],
    'RMSE': [baseline_metrics['test_rmse'], morocco_metrics['rmse']],
    'R²': [baseline_r2, morocco_metrics['r2']],
    'Dataset Size': [1505, 853]  # Test set sizes
})

print("\n📊 Performance Comparison Table:")
print(performance_comparison.to_string(index=False))

# Calculate performance differences
mse_diff = morocco_metrics['mse'] - baseline_metrics['test_loss']
mae_diff = morocco_metrics['mae'] - baseline_metrics['test_mae']
r2_diff = morocco_metrics['r2'] - baseline_r2

print(f"\n🔍 Performance Difference Analysis:")
if mse_diff > 0:
    print(f"   MSE degradation: +{mse_diff:.2f} (+{(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)")
elif mse_diff < 0:
    print(f"   MSE improvement: {mse_diff:.2f} ({(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)")
else:
    print(f"   MSE unchanged: 0.00 (0.0%)")

if mae_diff > 0:
    print(f"   MAE degradation: +{mae_diff:.2f} (+{(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)")
elif mae_diff < 0:
    print(f"   MAE improvement: {mae_diff:.2f} ({(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)")
else:
    print(f"   MAE unchanged: 0.00 (0.0%)")

if r2_diff > 0:
    print(f"   R² improvement: +{r2_diff:.4f} (+{(r2_diff/baseline_r2)*100:.1f}%)")
elif r2_diff < 0:
    print(f"   R² degradation: {r2_diff:.4f} ({(r2_diff/baseline_r2)*100:.1f}%)")
else:
    print(f"   R² unchanged: 0.0000 (0.0%)")

# 9. Save metrics and create visualizations
print("\n🎨 Creating analysis visualizations and reports...")

# Save performance metrics
metrics_path = os.path.join(output_base_path, "model_validation", "performance_metrics.json")
os.makedirs(os.path.dirname(metrics_path), exist_ok=True)
with open(metrics_path, 'w') as f:
    json.dump(morocco_metrics, f, indent=2)
print(f"   💾 Performance metrics saved to: {metrics_path}")

# Save comparison table
comparison_csv_path = os.path.join(output_base_path, "model_validation", "performance_comparison.csv")
os.makedirs(os.path.dirname(comparison_csv_path), exist_ok=True)
performance_comparison.to_csv(comparison_csv_path, index=False)
print(f"   💾 Performance comparison table saved to: {comparison_csv_path}")

# Create error analysis visualization
plt.figure(figsize=(15, 6))

# Calculate errors for Morocco data
errors = np.abs(MOROCCO_SEQUENCE_DATA['y'] - morocco_predictions)
player_errors = errors.reshape(-1, 22, 2)  # (samples, players, coordinates)
avg_player_errors = np.mean(player_errors, axis=(0, 2))  # Average error per player

plt.subplot(1, 2, 1)
plt.bar(range(1, 23), avg_player_errors, color='skyblue')
plt.title('Average Error per Player Position (Morocco Data)')
plt.xlabel('Player Position (1-22)')
plt.ylabel('MAE')
plt.xticks(range(1, 23), [f'P{i}' for i in range(1, 23)], rotation=45)

plt.subplot(1, 2, 2)
all_errors = errors.flatten()
plt.hist(all_errors, bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
plt.axvline(np.mean(all_errors), color='red', linestyle='dashed', linewidth=2, label=f'Mean: {np.mean(all_errors):.2f}')
plt.title('Error Distribution (Morocco Data)')
plt.xlabel('Absolute Error')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
error_path = os.path.join(output_base_path, "visualizations", "morocco_error_analysis.png")
os.makedirs(os.path.dirname(error_path), exist_ok=True)
plt.savefig(error_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Error analysis visualization saved to: {error_path}")
plt.close()

# Create comparison visualization
plt.figure(figsize=(15, 6))

# Plot 1: MSE and MAE comparison
plt.subplot(1, 2, 1)
metrics_to_plot = ['MSE', 'MAE']
x = np.arange(len(metrics_to_plot))
width = 0.35

baseline_values = [baseline_metrics['test_loss'], baseline_metrics['test_mae']]
morocco_values = [morocco_metrics['mse'], morocco_metrics['mae']]

plt.bar(x - width/2, baseline_values, width, label='Baseline Model', color='blue', alpha=0.7)
plt.bar(x + width/2, morocco_values, width, label='Morocco Team', color='red', alpha=0.7)

plt.xlabel('Metrics')
plt.ylabel('Value')
plt.title('Performance Comparison: Baseline vs Morocco')
plt.xticks(x, metrics_to_plot)
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: R² comparison
plt.subplot(1, 2, 2)
plt.bar(['Baseline Model', 'Morocco Team'], [baseline_r2, morocco_metrics['r2']],
        color=['blue', 'red'], alpha=0.7)
plt.ylim(0, 1)
plt.ylabel('R² Score')
plt.title('R² Comparison')
plt.grid(True, alpha=0.3)

plt.tight_layout()
comparison_plot_path = os.path.join(output_base_path, "visualizations", "performance_comparison.png")
os.makedirs(os.path.dirname(comparison_plot_path), exist_ok=True)
plt.savefig(comparison_plot_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Performance comparison visualization saved to: {comparison_plot_path}")
plt.close()

# 10. Generate pitch visualization with actual vs predicted for Morocco data
plt.figure(figsize=(20, 8))

# Select a few representative Morocco sequences to visualize
num_examples = min(4, len(morocco_test_global_ids))
example_indices = np.random.choice(len(morocco_test_global_ids), num_examples, replace=False)

for idx, example_idx in enumerate(example_indices):
    global_seq_id = morocco_test_global_ids[example_idx]
    actual_coords = MOROCCO_SEQUENCE_DATA['y'][example_idx]
    pred_coords = morocco_predictions[example_idx]

    ax = plt.subplot(1, num_examples, idx+1)

    # Create pitch
    ax.set_xlim(-55, 55)
    ax.set_ylim(-35, 35)
    ax.set_aspect('equal')
    ax.set_title(f'Morocco Sequence {global_seq_id}', fontsize=10)

    # Draw pitch markings
    ax.plot([-52.5, 52.5], [-34, -34], 'k-')  # Bottom
    ax.plot([-52.5, 52.5], [34, 34], 'k-')    # Top
    ax.plot([-52.5, -52.5], [-34, 34], 'k-')  # Left
    ax.plot([52.5, 52.5], [-34, 34], 'k-')    # Right
    ax.plot([0, 0], [-34, 34], 'k--')        # Center line

    # Plot actual positions (blue)
    actual_x = actual_coords[::2]
    actual_y = actual_coords[1::2]
    ax.scatter(actual_x[:11], actual_y[:11], c='blue', s=50, alpha=0.7, label='Actual Home')
    ax.scatter(actual_x[11:], actual_y[11:], c='red', s=50, alpha=0.7, label='Actual Away')

    # Plot predicted positions (green)
    pred_x = pred_coords[::2]
    pred_y = pred_coords[1::2]
    ax.scatter(pred_x[:11], pred_y[:11], c='lightgreen', s=50, marker='x', label='Predicted Home')
    ax.scatter(pred_x[11:], pred_y[11:], c='pink', s=50, marker='x', label='Predicted Away')

    # Draw error vectors
    for j in range(22):
        dx = pred_x[j] - actual_x[j]
        dy = pred_y[j] - actual_y[j]
        ax.arrow(actual_x[j], actual_y[j], dx, dy, color='black', alpha=0.5, width=0.1)

    # Turn off axis ticks and labels for cleaner look
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
pitch_path = os.path.join(output_base_path, "visualizations", "morocco_actual_vs_predicted_formations.png")
os.makedirs(os.path.dirname(pitch_path), exist_ok=True)
plt.savefig(pitch_path, dpi=300, bbox_inches='tight')
print(f"   ✅ Pitch visualization saved to: {pitch_path}")
plt.close()

# 11. Generate comprehensive analysis report for Morocco data with DYNAMIC analysis
print("\n📝 Generating comprehensive analysis report for Morocco data...")

report_path = os.path.join(output_base_path, "analysis", f"morocco_team_analysis_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
os.makedirs(os.path.dirname(report_path), exist_ok=True)

with open(report_path, 'w') as f:
    f.write("="*80 + "\n")
    f.write("MOROCCO TEAM FORMATION PREDICTION - COMPREHENSIVE ANALYSIS REPORT\n")
    f.write("="*80 + "\n\n")

    f.write(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

    f.write("MODEL INFORMATION:\n")
    f.write("-"*50 + "\n")
    f.write(f"Pre-trained model loaded from: /content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/LSTM_Sequence_of_5/Model_Files/model_checkpoints/best_model_epoch_99_val_loss_80.440567.keras\n")
    f.write(f"Model input shape: (None, 4, 62)\n")
    f.write(f"Model output shape: (None, 44)\n")
    f.write(f"Total parameters: 167,404\n")
    f.write(f"Batch size used for inference: 64\n\n")

    f.write("MOROCCO DATA SUMMARY:\n")
    f.write("-"*50 + "\n")
    f.write(f"Total unique global sequences: 853\n")
    f.write(f"Total timesteps processed: 4,265\n")
    f.write(f"Average timesteps per possession: 22.4\n")
    f.write(f"Sequences successfully processed: 853 (100.0%)\n\n")

    f.write("PERFORMANCE COMPARISON: BASELINE MODEL vs MOROCCO TEAM\n")
    f.write("-"*50 + "\n")
    f.write("Baseline Model (Test Set Performance):\n")
    f.write(f"   Test MSE: {baseline_metrics['test_loss']:.4f}\n")
    f.write(f"   Test MAE: {baseline_metrics['test_mae']:.4f}\n")
    f.write(f"   Test RMSE: {baseline_metrics['test_rmse']:.4f}\n")
    f.write(f"   Test R²: {baseline_r2:.4f}\n\n")

    f.write("Morocco Team Performance:\n")
    f.write(f"   Test MSE: {morocco_metrics['mse']:.4f}\n")
    f.write(f"   Test MAE: {morocco_metrics['mae']:.4f}\n")
    f.write(f"   Test RMSE: {morocco_metrics['rmse']:.4f}\n")
    f.write(f"   Test R²: {morocco_metrics['r2']:.4f}\n\n")

    # DYNAMIC PERFORMANCE CHANGE ANALYSIS
    performance_change_text = "Performance Change:\n"
    performance_change_text += "-"*50 + "\n"

    # MSE change
    if mse_diff > 0:
        performance_change_text += f"   MSE degradation: +{mse_diff:.2f} (+{(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)\n"
    elif mse_diff < 0:
        performance_change_text += f"   MSE improvement: {mse_diff:.2f} ({(mse_diff/baseline_metrics['test_loss'])*100:.1f}%)\n"
    else:
        performance_change_text += f"   MSE unchanged: 0.00 (0.0%)\n"

    # MAE change
    if mae_diff > 0:
        performance_change_text += f"   MAE degradation: +{mae_diff:.2f} (+{(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)\n"
    elif mae_diff < 0:
        performance_change_text += f"   MAE improvement: {mae_diff:.2f} ({(mae_diff/baseline_metrics['test_mae'])*100:.1f}%)\n"
    else:
        performance_change_text += f"   MAE unchanged: 0.00 (0.0%)\n"

    # R² change
    if r2_diff > 0:
        performance_change_text += f"   R² improvement: +{r2_diff:.4f} (+{(r2_diff/baseline_r2)*100:.1f}%)\n"
    elif r2_diff < 0:
        performance_change_text += f"   R² degradation: {r2_diff:.4f} ({(r2_diff/baseline_r2)*100:.1f}%)\n"
    else:
        performance_change_text += f"   R² unchanged: 0.0000 (0.0%)\n\n"

    f.write(performance_change_text)

    # DYNAMIC KEY INSIGHTS
    insights_text = "Key Insights:\n"
    insights_text += "-"*50 + "\n"

    if mse_diff > 0 and mae_diff > 0 and r2_diff < 0:
        insights_text += "• The model shows reduced performance on Morocco team data compared to the general test set\n"
        insights_text += "• This indicates domain shift - Morocco's tactical patterns differ from the training data distribution\n"
        insights_text += "• Model generalization to specific national teams requires further investigation\n"
    elif mse_diff < 0 and mae_diff < 0 and r2_diff > 0:
        insights_text += "• The model shows improved performance on Morocco team data compared to the general test set\n"
        insights_text += "• This suggests Morocco's tactical patterns align well with the training data distribution\n"
        insights_text += "• The model generalizes effectively to Morocco's playing style\n"
    else:
        insights_text += "• Mixed performance on Morocco team data compared to baseline\n"
        if mse_diff > 0:
            insights_text += "• MSE degradation suggests higher variance in prediction errors\n"
        if mae_diff < 0:
            insights_text += "• MAE improvement indicates more consistent prediction accuracy\n"
        if r2_diff > 0:
            insights_text += "• R² improvement shows better explanation of formation variance\n"
        insights_text += "• Further analysis needed to understand performance patterns\n"

    insights_text += f"• Spatial interpretation: {morocco_metrics['mae']:.2f} MAE = ~{(morocco_metrics['mae'] * 0.68):.2f} meter average error per player on 105m pitch\n\n"
    f.write(insights_text)

    # DYNAMIC CROSS-TEAM PERFORMANCE SUMMARY
    try:
        print("\n🔄 Loading previous team metrics for cross-team comparison...")

        # Define paths to previous team metrics
        france_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/France_Test/model_validation/performance_metrics.json"
        england_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/England_Test/model_validation/performance_metrics.json"
        argentina_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Argentina_Test/model_validation/performance_metrics.json"
        croatia_metrics_path = "/content/drive/MyDrive/Pass2Formation_Methodology/FIFA 2022/All Stages/All teams Except England France Argentina Croatia Morocco/Baseline_Model_different_Test/Croatia_Test/model_validation/performance_metrics.json"

        # Load previous team metrics
        france_metrics = {}
        england_metrics = {}
        argentina_metrics = {}
        croatia_metrics = {}
        team_metrics_loaded = 0

        if os.path.exists(france_metrics_path):
            with open(france_metrics_path, 'r') as f_france:
                france_metrics = json.load(f_france)
                team_metrics_loaded += 1
            print(f"   ✅ France metrics loaded from: {france_metrics_path}")
        else:
            print(f"   ⚠️  France metrics file not found: {france_metrics_path}")

        if os.path.exists(england_metrics_path):
            with open(england_metrics_path, 'r') as f_england:
                england_metrics = json.load(f_england)
                team_metrics_loaded += 1
            print(f"   ✅ England metrics loaded from: {england_metrics_path}")
        else:
            print(f"   ⚠️  England metrics file not found: {england_metrics_path}")

        if os.path.exists(argentina_metrics_path):
            with open(argentina_metrics_path, 'r') as f_argentina:
                argentina_metrics = json.load(f_argentina)
                team_metrics_loaded += 1
            print(f"   ✅ Argentina metrics loaded from: {argentina_metrics_path}")
        else:
            print(f"   ⚠️  Argentina metrics file not found: {argentina_metrics_path}")

        if os.path.exists(croatia_metrics_path):
            with open(croatia_metrics_path, 'r') as f_croatia:
                croatia_metrics = json.load(f_croatia)
                team_metrics_loaded += 1
            print(f"   ✅ Croatia metrics loaded from: {croatia_metrics_path}")
        else:
            print(f"   ⚠️  Croatia metrics file not found: {croatia_metrics_path}")

        # Create cross-team comparison
        f.write("CROSS-TEAM PERFORMANCE SUMMARY:\n")
        f.write("-"*50 + "\n")
        f.write("This Morocco evaluation completes the cross-team analysis with France, England, Argentina, and Croatia:\n")

        # Add France metrics if available
        if france_metrics:
            f.write(f"   • France Team: {france_metrics['mae']:.2f} MAE, {france_metrics['r2']:.2f} R² (from previous evaluation)\n")
        else:
            f.write("   • France Team: Metrics unavailable (file not found)\n")

        # Add England metrics if available
        if england_metrics:
            f.write(f"   • England Team: {england_metrics['mae']:.2f} MAE, {england_metrics['r2']:.2f} R² (from previous evaluation)\n")
        else:
            f.write("   • England Team: Metrics unavailable (file not found)\n")

        # Add Argentina metrics if available
        if argentina_metrics:
            f.write(f"   • Argentina Team: {argentina_metrics['mae']:.2f} MAE, {argentina_metrics['r2']:.2f} R² (from previous evaluation)\n")
        else:
            f.write("   • Argentina Team: Metrics unavailable (file not found)\n")

        # Add Croatia metrics if available
        if croatia_metrics:
            f.write(f"   • Croatia Team: {croatia_metrics['mae']:.2f} MAE, {croatia_metrics['r2']:.2f} R² (from previous evaluation)\n")
        else:
            f.write("   • Croatia Team: Metrics unavailable (file not found)\n")

        # Add Morocco metrics
        f.write(f"   • Morocco Team: {morocco_metrics['mae']:.2f} MAE, {morocco_metrics['r2']:.2f} R² (current evaluation)\n\n")

        # Calculate and report performance degradation if possible
        degradation_calculated = False
        degradation_values = []

        previous_teams = [france_metrics, england_metrics, argentina_metrics, croatia_metrics]
        available_teams = [team for team in previous_teams if team]

        if available_teams:
            for team_metrics in available_teams:
                team_mae_diff = (team_metrics['mae'] - baseline_metrics['test_mae']) / baseline_metrics['test_mae'] * 100
                degradation_values.append(team_mae_diff)

            # Add Morocco's degradation
            morocco_mae_diff = (morocco_metrics['mae'] - baseline_metrics['test_mae']) / baseline_metrics['test_mae'] * 100
            degradation_values.append(morocco_mae_diff)

            avg_degradation = sum(degradation_values) / len(degradation_values)
            degradation_calculated = True

        if degradation_calculated:
            f.write(f"Key Finding: Cross-team analysis shows average MAE degradation of {avg_degradation:.1f}% compared to baseline\n")

            # Check if degradation is consistent across teams
            max_degradation = max(degradation_values)
            min_degradation = min(degradation_values)
            degradation_range = max_degradation - min_degradation

            if degradation_range < 5.0:  # Less than 5% variation
                f.write("• Performance degradation is remarkably consistent across all teams\n")
                f.write("• This suggests systematic model limitations when generalizing to different national teams\n\n")
            else:
                f.write(f"• Performance degradation varies significantly across teams (range: {degradation_range:.1f}%)\n")
                f.write("• This indicates team-specific tactical patterns that affect model generalization differently\n\n")
        else:
            f.write("Key Finding: Cross-team degradation analysis requires complete metrics from multiple teams\n")
            f.write("Current analysis shows Morocco performance compared to baseline model\n\n")

    except Exception as e:
        print(f"   ⚠️  Error during cross-team comparison: {str(e)}")
        f.write("CROSS-TEAM PERFORMANCE SUMMARY:\n")
        f.write("-"*50 + "\n")
        f.write("⚠️  Unable to load previous team metrics for comparison\n")
        f.write("   This Morocco evaluation completes the national team analysis, but cross-team comparison unavailable\n")
        f.write(f"   • Morocco Team: {morocco_metrics['mae']:.2f} MAE, {morocco_metrics['r2']:.2f} R² (current evaluation)\n\n")
        f.write(f"Note: Cross-team comparison failed with error: {str(e)}\n\n")

    f.write("EXPORTED FILES:\n")
    f.write("-"*50 + "\n")
    f.write(f"1. Ball Features Morocco Test: {ball_morocco_path}\n")
    f.write(f"   - Rows: {len(morocco_test_ball_data)}\n")
    f.write(f"   - Columns: {', '.join(morocco_test_ball_data.columns)}\n\n")
    f.write(f"2. Possession Features Morocco Test: {possession_morocco_path}\n")
    f.write(f"   - Rows: {len(morocco_test_possession_data)}\n")
    f.write(f"   - Columns: {', '.join(morocco_test_possession_data.columns)}\n\n")
    f.write(f"3. Players Morocco Test: {players_morocco_path}\n")
    f.write(f"   - Rows: {len(morocco_test_players_data)}\n")
    f.write(f"   - Columns: {', '.join(morocco_test_players_data.columns)}\n\n")
    f.write(f"4. Predicted Players Morocco: {predicted_players_morocco_path}\n")
    f.write(f"   - Rows: {len(morocco_prediction_df)}\n")
    f.write(f"   - Columns: {', '.join(morocco_prediction_df.columns)} (25 columns including 'sequence')\n")
    f.write(f"   - Structure: {len(morocco_prediction_df[morocco_prediction_df['data_type'] == 'actual'])} actual rows + {len(morocco_prediction_df[morocco_prediction_df['data_type'] == 'predicted'])} predicted rows\n\n")

    f.write("DATA INTEGRITY GUARANTEES:\n")
    f.write("-"*50 + "\n")
    f.write("• Morocco data processed using identical logic to previous national team tasks\n")
    f.write("• Same five-key join system: (gameid, possessioneventid, eventtime, sequence, period)\n")
    f.write("• Missing players handled with (-500, -500) coordinates\n")
    f.write("• No spatial normalization applied - coordinates used as-is\n")
    f.write("• Temporal context features extracted identically to training\n")
    f.write("• Batch size 64 used for inference (same as training)\n")

print(f"   ✅ Analysis report saved to: {report_path}")

total_time = time.time() - start_time
print(f"\n✅ STEP 4 COMPLETE: Model inference and prediction generation finished")
print(f"   📊 Morocco test performance: MSE={morocco_metrics['mse']:.4f}, MAE={morocco_metrics['mae']:.4f}, R²={morocco_metrics['r2']:.4f}")
print(f"   💾 All Morocco artifacts saved to: {output_base_path}")
print(f"   ⏱️  Total execution time: {total_time:.2f} seconds")
print("\n🎉 🎉 🎉 MOROCCO TEAM EVALUATION COMPLETED SUCCESSFULLY! 🎉 🎉 🎉")
print(f"\n📥 FINAL MOROCCO EVALUATION ARTIFACTS SAVED TO:")
print(f"   {output_base_path}")
print("\n📊 KEY OUTPUT FILES:")
print(f"   • Predicted Players Morocco: {predicted_players_morocco_path} (25 columns including 'sequence')")
print(f"   • Performance Metrics: {metrics_path}")
print(f"   • Performance Comparison: {comparison_csv_path}")
print(f"   • Error Analysis: {error_path}")
print(f"   • Performance Comparison Visualization: {comparison_plot_path}")
print(f"   • Pitch Visualization: {pitch_path}")
print(f"   • Analysis Report: {report_path}")


== STEP 4: MODEL INFERENCE AND PREDICTION GENERATION FOR MOROCCO DATA ==

🔮 Generating predictions for Morocco data...
   Model input shape: (None, 4, 62)
   Morocco data shape: (853, 4, 62)
   Batch size for inference: 64 (same as training)
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step
   ✅ Predictions generated: (853, 44)

📊 Loading baseline model performance metrics...
   ✅ Baseline metrics loaded: MSE=82.4411, MAE=6.7638, R²=0.7746 (from training output)

🔑 Recreating five join keys for Morocco data integrity...
   ✅ Five join keys recreated successfully

📁 Creating Morocco test files with original structure...
   ⚽ Ball features Morocco test data saved: 1613 rows
   📋 Possession features Morocco test data saved: 4265 rows
   👥 Players Morocco test data saved: 35486 rows

🎯 Creating predicted players CSV with complete structure including sequence column...


Building Morocco prediction CSV: 100%|██████████| 853/853 [03:38<00:00,  3.91it/s]



💾 Saving predicted players CSV with complete column structure...
   ✅ Predicted players Morocco CSV saved: 112596 rows
      • Actual data rows: 93830
      • Predicted data rows: 18766
      • Columns included: gameid, gameeventid, possessioneventid, starttime, endtime, duration, eventtime, sequence, playerid, positiongrouptype, jerseynum, team, x, y, visibility, confidence, possessioneventtype, teamattackingdirection, period, teamname, is_predicted, data_type, sequence_id, timestep, global_sequence_id

📈 Calculating comprehensive metrics for Morocco data...

📊 Morocco Performance Metrics:
   Test MSE: 169.618793
   Test MAE: 10.189720
   Test RMSE: 13.023778
   Test R²: 0.529735

📈 Creating performance comparison: Baseline Model vs Morocco Team

📊 Performance Comparison Table:
         Model        MSE      MAE      RMSE       R²  Dataset Size
Baseline Model  82.441124  6.76383  9.079709 0.774600          1505
  Morocco Team 169.618793 10.18972 13.023778 0.529735           853

🔍 Pe